In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

Before LOWER(TRIM()) on `brand` join

In [164]:

check = client.query(
    """ 
    CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` AS (
        SELECT
            "Instore Order" AS order_context,
            start_txn_date,
            start_txn_time,
            "CRN" as shopper_identification_method,
            lylty_card_detail.crn AS shopper_identification_number,
            article_sales_summary.Article AS article,
            am.BrandDescription AS brand,
            am.SubcatDescription,
            article_sales_summary.prod_nbr,
            basket_key,
            tot_net_incld_gst
        
        -- transaction table
        FROM `gcp-wow-food-wlx-digaspt-dev.wdp_tables.article_sales_summary` AS article_sales_summary

        LEFT JOIN (
            SELECT 
                SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
                MAX(SubCategoryShortDescription) AS SubcatDescription,
                MAX(Brand) AS BrandDescription
            FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
            WHERE SalesOrganisation = "1005"
            AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
            AND Brand IS NOT NULL
            AND SubCategoryShortDescription IS NOT NULL
            GROUP BY 1
        ) am ON am.Article = article_sales_summary.Article
        
        -- loyalty card
        LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.wdp_tables.lylty_card_detail` AS lylty_card_detail
            ON article_sales_summary.lylty_card_nbr = lylty_card_detail.lylty_card_nbr
        

        WHERE
            article_sales_summary.start_txn_date >= DATE("2022-10-01")
        AND 
            article_sales_summary.start_txn_date <= CURRENT_DATE("Australia/Sydney")
        AND 
            LENGTH(article_sales_summary.lylty_card_nbr) > 3 
        AND 
            LENGTH(lylty_card_detail.crn) > 3 
        AND
            article_sales_summary.division_nbr IN (1005, 1030)
        AND 
            LOWER(article_sales_summary.void_flag) = 'n' 
        AND 
            LOWER(article_sales_summary.SalesChannelDescription) <> "online"
        
        
        UNION ALL 

        SELECT 
            "Online Order" AS order_context,
            CAST(oh.DisColDateTime AS DATE) AS start_txn_date,
            od.CreateDateTime AS start_txn_time, 
            "CRN" AS shopper_identification_method, 
            crn_map.CustomerRegistrationNumber AS shopper_identification_number,      
            CAST(od.StockCode as STRING) AS article,
            am2.BrandDescription AS brand,
            am2.SubcatDescription,
            "" AS prod_nbr, 
            CAST(OriginalOrderNumber AS STRING) AS basket_key, 
            od.OrderLineAmount as tot_net_incld_gst
    FROM `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_online_view_smkt.ecf_smkt_order_header_v` as oh
    INNER JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_online_view_smkt.ecf_smkt_order_detail_v` as od
        ON oh.OrderNumber = od.OrderNumber
    LEFT JOIN (
        SELECT 
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            MAX(SubCategoryShortDescription) AS SubcatDescription,
            MAX(Brand) AS BrandDescription
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
        AND Brand IS NOT NULL
        AND SubCategoryShortDescription IS NOT NULL
        GROUP BY 1
        ) am2 ON am2.Article = CAST(od.StockCode as STRING)
    LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_customer_view.customer_online_shopper_v` crn_map 
        ON CAST(crn_map.ShopperID AS STRING) = CAST(oh.CustomerNumber AS STRING)
    WHERE CAST(oh.DisColDateTime as DATE) >= DATE("2022-10-01")

);
    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [ ]:

check = client.query(
    """ 
    with step_one AS (
    SELECT 
        Article, 
        ARRAY_AGG(DISTINCT SubcatDescription IGNORE NULLS) AS SubcatDescriptions
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
    GROUP BY 1
    )
    SELECT 
        Article, 
        SubcatDescriptions, 
        ARRAY_LENGTH(SubcatDescriptions) AS n_SubcatDescriptions
    FROM step_one 
    ORDER BY 3 DESC
    LIMIT 500
    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

In [44]:
check = client.query(
"""    
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_vs_7_days` AS (
    SELECT 
        sd_sku.booking_id, 
        sd_brand.brands,
        --sd.asset_type,
        sd_sku.exposed_shoppers_7_days exposed_shoppers,
        sd_sku.converted_shoppers_7_days AS sku_level_converted_shoppers_7_days,
        sd_brand.converted_shoppers_7_days AS brand_level_converted_shoppers_7_days,
        SAFE_DIVIDE(sd_sku.converted_shoppers_7_days  , sd_sku.exposed_shoppers_7_days) AS sku_level_conversion_rate_7_days,
        SAFE_DIVIDE(sd_brand.converted_shoppers_7_days  , sd_sku.exposed_shoppers_7_days) AS brand_level_conversion_rate_7_days,
        td_sku.exposed_shoppers_3_days, 
        td_sku.converted_shoppers_3_days AS sku_level_converted_shoppers_3_days,
        td_brand.converted_shoppers_3_days AS brand_level_converted_shoppers_3_days,
        SAFE_DIVIDE(td_sku.converted_shoppers_3_days ,  td_sku.exposed_shoppers_3_days) AS sku_level_conversion_rate_3_days,
        SAFE_DIVIDE(td_brand.converted_shoppers_3_days ,  td_sku.exposed_shoppers_3_days) AS brand_level_conversion_rate_3_days,

        sd_sku.baskets_7_days AS sku_level_baskets_7_days,
        sd_brand.baskets_7_days AS brand_level_baskets_7_days,
        td_sku.baskets_3_days AS sku_level_baskets_3_days,
        td_brand.baskets_3_days AS brand_level_baskets_3_days,

        sd_sku.sales_amt_online_7_days AS sku_level_sales_amt_online_7_days,
        sd_brand.sales_amt_online_7_days AS brand_level_sales_amt_online_7_days,
        td_sku.sales_amt_online_3_days AS sku_level_sales_amt_online_3_days,
        td_brand.sales_amt_online_3_days AS brand_level_sales_amt_online_3_days,
        
        sd_sku.sales_amt_instore_7_days AS sku_level_sales_amt_instore_7_days,
        sd_brand.sales_amt_instore_7_days AS brand_level_sales_amt_instore_7_days,
        td_sku.sales_amt_instore_3_days AS sku_level_sales_amt_instore_3_days,
        td_brand.sales_amt_instore_3_days AS brand_level_sales_amt_instore_3_days

    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_sku_level` sd_sku 
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_brand_level` sd_brand
        ON sd_sku.booking_id = sd_brand.booking_id 
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_sku_level` td_sku
        ON sd_sku.booking_id = td_sku.booking_id
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_brand_level` td_brand
        ON sd_sku.booking_id = td_brand.booking_id
);

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [ ]:
check = client.query(
"""    
SELECT "Sku Level" AS sales_type, SUM(tot_net_incld_gst) AS total_sales
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` sl
WHERE sl.booking_id = "WOW20006759"
GROUP BY 1

UNION ALL 

SELECT "Brand Level", SUM(tot_net_incld_gst) AS total_sales
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` bl
WHERE booking_id = "WOW20006759"   
GROUP BY 1
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,sales_type,total_sales
0,Brand Level,5990.140000000
1,Sku Level,1273.740000000


In [189]:
check = client.query(
"""    
SELECT "Sku Level" AS sales_type, SUM(tot_net_incld_gst) AS total_sales
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2` sl
WHERE sl.booking_id = "WOW20006759"
GROUP BY 1

UNION ALL 

SELECT "Brand Level", SUM(tot_net_incld_gst) AS total_sales
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` bl
WHERE booking_id = "WOW20006759"   
GROUP BY 1
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,sales_type,total_sales
0,Brand Level,120876.440000000
1,Sku Level,77526.750000000


In [22]:
check = client.query(
"""    
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`  AS (
    with step_one AS (
        SELECT
            booking_number,
            campaign_start_date,
            campaign_end_date,
            line_name,
            CASE
                WHEN media_location LIKE ('%TV%') THEN 'Mass_Social_Media'
                WHEN media_type IN (
                    'Counter Card',
                    'POC',
                    'Spotify',
                    'YouTube',
                    'Social',
                    'POC',
                    'Social - Wrapper',
                    'Build Capacity'
                ) THEN 'Mass_Social_Media'
                WHEN media_location IN ('Category', 'Aisle', 'Store', 'Screens', 'Emma Reach') THEN 'in-store'
                WHEN media LIKE ('%FB Post%') THEN 'Mass_Social_Media'
                WHEN media_type = 'Always On eDMs' THEN 'eDM'
                WHEN media_type = 'Single Send eDM' THEN 'eDM'
                WHEN media_type = 'BRANDED SHOP' THEN 'Branded Shop'
                WHEN media_type = 'CONTENT CARD' THEN 'Content Card'
                WHEN media_type = 'Digital Standalone Catalogue' THEN 'Catalogue'
                WHEN media_type = 'Display Recipes - Wrapper' THEN 'Display Recipes'
                WHEN media_type = 'RECIPE PACKAGE' THEN 'Display Recipes'
                WHEN media_type = 'Recipe Package' THEN 'Display Recipes'
                WHEN media_type = 'Standalone Catalogue - Double Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue - Single Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue- Third Page Vertical' THEN 'Catalogue'
                WHEN media_type = 'Catalogue Card' THEN 'Catalogue'
                WHEN media_type = 'BF Meal' THEN 'Others'
                WHEN media_type = 'Sampling' THEN 'Others'
                WHEN media_type = 'Bus Stop' THEN 'in-store'
                WHEN media_type = 'Display Promo Tile' THEN 'in-store'
                WHEN media_type = 'Aisle Fin' THEN 'in-store'
                WHEN media_type = 'Additional Production Fees' THEN 'Others'
                WHEN media_type = 'Event' THEN 'Others'
                WHEN media_type = 'National Radio' THEN 'Mass_Social_Media'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'National TV' THEN 'Mass_Social_Media'
                WHEN media_type = 'Press' THEN 'Mass_Social_Media'
                WHEN media_type = 'Content Card' THEN 'Others'
                WHEN media_type = 'Competition Management Fee' THEN 'Others'
                WHEN media_type = 'Prize pool' THEN 'Others'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'Chilled Fin Package' THEN 'in-store'
                WHEN media_type = 'Editorial' THEN 'Mass_Social_Media'
                WHEN media_type = 'BUS STOP' THEN 'in-store'
                WHEN media_type = 'Door Take' THEN 'in-store'
                WHEN media_type = 'Freezer Package' THEN 'in-store'
                WHEN media_type = 'PL - Mag' THEN 'Others'
                WHEN media_type = 'PL - Fresh Mag' THEN 'Others'
                WHEN media_type = 'PL - Demos' THEN 'Others'
                WHEN media_type = 'Pelmet' THEN 'in-store'
                WHEN media_type = 'Overlay Card' THEN 'in-store'
                WHEN media_type = 'Mass/Social Media' THEN 'Others'
                WHEN lower(media_type) like "%catalogue%" THEN 'Catalogue'
                WHEN LOWER(media_type) LIKE "%homepage%" THEN "Home Page"
                ELSE
                media_type
            END
            AS media_type,
            individual_product_string,
            MIN(media_start_date) AS media_start_date,
            MAX(media_end_date) AS media_end_date,
            SUM(quantity) * SUM(media_spend) AS total_media_spend

        FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`,
        UNNEST(SPLIT(quoteline_sku, ',')) AS individual_product_string

        WHERE
            
            business_unit = 'Supermarket'
            AND booking_number IS NOT NULL
            AND quote_status NOT IN ('Not Approved','Draft','For Review','Sent')
            AND media_location NOT IN ('Category','Aisle','Store','Screens','Emma Reach')
            AND media_location NOT LIKE ('%TV%')
            AND media_type NOT IN ('Counter Card','POC','Spotify','YouTube','Social','POC','Social - Wrapper')
            AND (media_type <> 'Event')
            AND (media_location <> 'None' OR media LIKE ('%FB Post%') OR media LIKE ('%Branded%') OR media LIKE ('%Segments%'))
            AND media NOT IN ('Competition Management Fee','Additional Production Fees')
        AND media_start_date >= DATE("2024-01-01")
        AND media_end_date < CURRENT_DATE("Australia/Sydney")
        GROUP BY
            1,2,3,4,5,6
    ) 

    SELECT * 
    FROM step_one
    WHERE
        media_type NOT IN ('in-store','Others','eDM')
        AND media_type NOT LIKE ('%Woolworths Rewards%')
        AND media_type NOT LIKE ('%SAMPLING%') 
        AND media_type NOT LIKE "%Road Block%"
        AND media_type <> 'Radio'
        AND media_type <> 'Mass_Social_Media'
        AND LOWER(media_type) NOT LIKE "%branded shop%"

);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [93]:
check = client.query(
"""    
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v3 AS (

-- CIID Rows
    SELECT 
        "CI ID Rows" AS join_method,
        adobe.date_time,
        adobe.shopper_id,
        adobe.crn,
        adobe.hashed_crn,  

        adobe.booking_id, 
        adobe.ci_id,
        ciid_rows.campaign_start_date,
        ciid_rows.campaign_end_date,
        --COALESCE(ciid_rows.line_name, bshop_catalogue_homepage_rows.line_name) AS quoteline_id,
        ciid_rows.media_type AS media_type, 
        ciid_rows.media_start_date AS media_start_date,
        ciid_rows.media_end_date AS media_end_date,
        ciid_rows.campaign_skus AS campaign_skus
        
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe 

    -- Join Cartology Campaign Info for Adobe Rows that have CIID
    LEFT JOIN (
        SELECT 
            booking_number, 
            campaign_start_date,
            campaign_end_date,
            media_type,
            line_name, 
            media_start_date,
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        GROUP BY 1,2,3,4,5,6,7
    ) ciid_rows 
        ON ciid_rows.line_name = adobe.ci_id 
        AND adobe.booking_id = ciid_rows.booking_number
        AND adobe.date_time BETWEEN TIMESTAMP(ciid_rows.media_start_date) AND TIMESTAMP(ciid_rows.media_end_date + 1)
        AND adobe.ci_id IS NOT NULL 
        AND adobe.booking_id IS NOT NULL

    WHERE 
        -- Line Name exists
        (ciid_rows.line_name IS NOT NULL AND ciid_rows.line_name <> "") 
        AND adobe.date_time BETWEEN TIMESTAMP(ciid_rows.media_start_date) AND TIMESTAMP(ciid_rows.media_end_date + 1)
        AND adobe.ci_id IS NOT NULL 
        AND adobe.booking_id IS NOT NULL

    UNION ALL

-- Catalogue Rows 
    SELECT 
        "Catalogue" AS join_method,
        adobe2.date_time,
        adobe2.shopper_id,
        adobe2.crn,
        adobe2.hashed_crn,  
        
        catalogue_rows.booking_number AS booking_id,
        adobe2.ci_id,
        catalogue_rows.campaign_start_date,
        catalogue_rows.campaign_end_date,
        --COALESCE(ciid_rows.line_name, bshop_catalogue_homepage_rows.line_name) AS quoteline_id,
        catalogue_rows.media_type AS media_type, 
        catalogue_rows.media_start_date AS media_start_date,
        catalogue_rows.media_end_date AS media_end_date,
        catalogue_rows.campaign_skus AS campaign_skus

    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe2
    LEFT JOIN (
        SELECT 
            booking_number, 
            campaign_start_date,
            campaign_end_date,
            media_type, 
            line_name, 
            media_start_date, 
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        WHERE LOWER(media_type) = "catalogue"
        GROUP BY 1,2,3,4,5,6,7
    ) catalogue_rows
        ON adobe2.catalogue_flag = 1
        AND adobe2.booking_id = "Not Found"
    
    WHERE adobe2.catalogue_flag = 1
    AND adobe2.booking_id = "Not Found" 
    AND adobe2.date_time BETWEEN TIMESTAMP(catalogue_rows.media_start_date) AND TIMESTAMP(catalogue_rows.media_end_date + 1)

-- Home Page rows that have ci_id and booking_id in the adobe tracking

    UNION ALL  

    SELECT 
        "Homepage Rows" AS join_method,
        adobe3.date_time,
        adobe3.shopper_id,
        adobe3.crn,
        adobe3.hashed_crn,  
        
        homepage_rows.booking_number AS booking_id,
        adobe3.ci_id,
        homepage_rows.campaign_start_date,
        homepage_rows.campaign_end_date,
        homepage_rows.media_type AS media_type, 
        homepage_rows.media_start_date AS media_start_date,
        homepage_rows.media_end_date AS media_end_date,
        homepage_rows.campaign_skus AS campaign_skus

    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe3
    LEFT JOIN (
        SELECT 
            booking_number, 
            media_type, 
            campaign_start_date,
            campaign_end_date,
            "" AS line_name, 
            media_start_date, 
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        WHERE LOWER(media_type) = "home page"
        GROUP BY 1,2,3,4,5,6,7
    ) homepage_rows
        ON adobe3.homepage_flag = 1
        AND adobe3.booking_id = "Not Found"
        AND adobe3.date_time BETWEEN TIMESTAMP(homepage_rows.media_start_date) AND TIMESTAMP(homepage_rows.media_end_date + 1)
    WHERE adobe3.homepage_flag = 1
    AND adobe3.booking_id = "Not Found"
    AND adobe3.date_time BETWEEN TIMESTAMP(homepage_rows.media_start_date) AND TIMESTAMP(homepage_rows.media_end_date + 1)

);

CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 AS 
    SELECT DISTINCT
        DATE(date_time) date,
        booking_id, 
        campaign_start_date,
        campaign_end_date,
        ci_id,
        media_type, 
        shopper_id, 
        crn, 
        ARRAY_TO_STRING(campaign_skus, ",") skus
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v3
    WHERE crn IS NOT NULL
;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [257]:
check = client.query(
"""    
-- ######################################################################################################################################## 
-- Loop and Date Variables
DECLARE start_index INT64 DEFAULT 0;
DECLARE end_index INT64;

DECLARE campaigns_to_be_analysed_array_global_var ARRAY<STRING>;
DECLARE current_campaign_global_var STRING;

-- Query Execution Time Logging Variables
DECLARE campaign_run_start_time DATETIME;
DECLARE query_start_time DATETIME;
DECLARE query_end_time DATETIME;


SET campaigns_to_be_analysed_array_global_var = (
    SELECT 
        ARRAY_AGG(DISTINCT booking_id IGNORE NULLS) AS campaigns 
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 
    WHERE campaign_start_date >= DATE("2023-10-01") 
    AND booking_id NOT IN (SELECT DISTINCT booking_id FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control)
);

CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` AS
    SELECT  
        * 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs`
    WHERE query_step = "999"
;

SET end_index = ARRAY_LENGTH(campaigns_to_be_analysed_array_global_var);

-- ######################################################################################################################################## 

CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`  AS (
    with step_one AS (
        SELECT
            booking_number,
            campaign_start_date,
            line_name,
            CASE
                WHEN media_location LIKE ('%TV%') THEN 'Mass_Social_Media'
                WHEN media_type IN (
                    'Counter Card',
                    'POC',
                    'Spotify',
                    'YouTube',
                    'Social',
                    'POC',
                    'Social - Wrapper',
                    'Build Capacity'
                ) THEN 'Mass_Social_Media'
                WHEN media_location IN ('Category', 'Aisle', 'Store', 'Screens', 'Emma Reach') THEN 'in-store'
                WHEN media LIKE ('%FB Post%') THEN 'Mass_Social_Media'
                WHEN media_type = 'Always On eDMs' THEN 'eDM'
                WHEN media_type = 'Single Send eDM' THEN 'eDM'
                WHEN media_type = 'BRANDED SHOP' THEN 'Branded Shop'
                WHEN media_type = 'CONTENT CARD' THEN 'Content Card'
                WHEN media_type = 'Digital Standalone Catalogue' THEN 'Catalogue'
                WHEN media_type = 'Display Recipes - Wrapper' THEN 'Display Recipes'
                WHEN media_type = 'RECIPE PACKAGE' THEN 'Display Recipes'
                WHEN media_type = 'Recipe Package' THEN 'Display Recipes'
                WHEN media_type = 'Standalone Catalogue - Double Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue - Single Spot' THEN 'Catalogue'
                WHEN media_type = 'Standalone Catalogue- Third Page Vertical' THEN 'Catalogue'
                WHEN media_type = 'Catalogue Card' THEN 'Catalogue'
                WHEN media_type = 'BF Meal' THEN 'Others'
                WHEN media_type = 'Sampling' THEN 'Others'
                WHEN media_type = 'Bus Stop' THEN 'in-store'
                WHEN media_type = 'Display Promo Tile' THEN 'in-store'
                WHEN media_type = 'Aisle Fin' THEN 'in-store'
                WHEN media_type = 'Additional Production Fees' THEN 'Others'
                WHEN media_type = 'Event' THEN 'Others'
                WHEN media_type = 'National Radio' THEN 'Mass_Social_Media'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'National TV' THEN 'Mass_Social_Media'
                WHEN media_type = 'Press' THEN 'Mass_Social_Media'
                WHEN media_type = 'Content Card' THEN 'Others'
                WHEN media_type = 'Competition Management Fee' THEN 'Others'
                WHEN media_type = 'Prize pool' THEN 'Others'
                WHEN media_type = 'Chilled Bus Stop & Decal Package' THEN 'in-store'
                WHEN media_type = 'Chilled Fin Package' THEN 'in-store'
                WHEN media_type = 'Editorial' THEN 'Mass_Social_Media'
                WHEN media_type = 'BUS STOP' THEN 'in-store'
                WHEN media_type = 'Door Take' THEN 'in-store'
                WHEN media_type = 'Freezer Package' THEN 'in-store'
                WHEN media_type = 'PL - Mag' THEN 'Others'
                WHEN media_type = 'PL - Fresh Mag' THEN 'Others'
                WHEN media_type = 'PL - Demos' THEN 'Others'
                WHEN media_type = 'Pelmet' THEN 'in-store'
                WHEN media_type = 'Overlay Card' THEN 'in-store'
                WHEN media_type = 'Mass/Social Media' THEN 'Others'
                WHEN lower(media_type) like "%catalogue%" THEN 'Catalogue'
                WHEN LOWER(media_type) LIKE "%homepage%" THEN "Home Page"
                ELSE
                media_type
            END
            AS media_type,
            individual_product_string,
            MIN(media_start_date) AS media_start_date,
            MAX(media_end_date) AS media_end_date,
            SUM(quantity) * SUM(media_spend) AS total_media_spend

        FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`,
        UNNEST(SPLIT(quoteline_sku, ',')) AS individual_product_string

        WHERE
            
            business_unit = 'Supermarket'
            AND booking_number IS NOT NULL
            AND quote_status NOT IN ('Not Approved','Draft','For Review','Sent')
            AND media_location NOT IN ('Category','Aisle','Store','Screens','Emma Reach')
            AND media_location NOT LIKE ('%TV%')
            AND media_type NOT IN ('Counter Card','POC','Spotify','YouTube','Social','POC','Social - Wrapper')
            AND (media_type <> 'Event')
            AND (media_location <> 'None' OR media LIKE ('%FB Post%') OR media LIKE ('%Branded%') OR media LIKE ('%Segments%'))
            AND media NOT IN ('Competition Management Fee','Additional Production Fees')
        AND media_start_date >= DATE("2023-10-01")
        AND media_end_date < CURRENT_DATE("Australia/Sydney")
        GROUP BY
            1,2,3,4,5
    ) 

    SELECT * 
    FROM step_one
    WHERE
        media_type NOT IN ('in-store','Others','eDM')
        AND media_type NOT LIKE ('%Woolworths Rewards%')
        AND media_type NOT LIKE ('%SAMPLING%') 
        AND media_type NOT LIKE "%Road Block%"
        AND media_type <> 'Radio'
        AND media_type <> 'Mass_Social_Media'
        AND LOWER(media_type) NOT LIKE "%branded shop%"

);



-- 2. Adobe Events
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid AS (
    with 
    
    cartology_campaign_asset_impressions_in_adobe AS (
        SELECT 
            date_time,
            shopper_id_map.shopper_id, 
            crn_map.crn, 
            post_evar6 AS page_name,
            CASE WHEN SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue' THEN 1 ELSE 0 END AS catalogue_flag,    
            CASE WHEN post_evar6 = 'ww-sm:homepage' THEN 1 ELSE 0 END AS homepage_flag, 
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(0)] ELSE "Not Found" END AS booking_id,
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] ELSE NULL END AS ci_id
            
        FROM `gcp-wow-food-wlx-digaspt-dev.prod_adobe_data.grs_adobe` adobe

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON adobe.prop9 = shopper_id_map.visitor_id
            AND adobe.date_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON adobe.prop9 = crn_map.visitor_id
            AND adobe.date_time BETWEEN crn_map.effective_time and crn_map.expiry_time

        WHERE 
            site_name = "Supermarkets"
        AND 
            DATE(date_time) >= DATE("2023-10-01")
        AND    
            DATE(date_time) < CURRENT_DATE("Australia/Sydney")
        AND         
            (
                -- Catalogue
                (SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue')
                OR 
                -- Homepage
                (post_evar6 = 'ww-sm:homepage') 
                OR 
                -- ci_id 
                (SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] IS NOT NULL)
            )
        AND      
            post_channel NOT LIKE "Order Confirmation Section"
        AND 
            (shopper_id_map.shopper_id IS NOT NULL OR crn_map.crn IS NOT NULL)
    
    ) 

    SELECT * FROM cartology_campaign_asset_impressions_in_adobe 

);


CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_control AS (
    
    
        SELECT DISTINCT
            DATE(date_time) date,
            shopper_id_map.shopper_id, 
            crn_map.crn
            
        FROM `gcp-wow-food-wlx-digaspt-dev.prod_adobe_data.grs_adobe` adobe

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON adobe.prop9 = shopper_id_map.visitor_id
            AND adobe.date_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON adobe.prop9 = crn_map.visitor_id
            AND adobe.date_time BETWEEN crn_map.effective_time and crn_map.expiry_time

        WHERE 
            site_name = "Supermarkets"
        AND 
            DATE(date_time) >= DATE("2023-10-01")
        AND    
            DATE(date_time) < CURRENT_DATE("Australia/Sydney")
        AND         
            (
                -- Catalogue
                (SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue')
                OR 
                -- Homepage
                (post_evar6 = 'ww-sm:homepage') 
                OR 
                -- ci_id 
                (SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] IS NOT NULL)
            )
        AND      
            post_channel NOT LIKE "Order Confirmation Section"
        AND 
            (shopper_id_map.shopper_id IS NOT NULL OR crn_map.crn IS NOT NULL)

);

-- 3. Adobe Events + Cartology Campaign Info
-- Approx 33mins to run
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v3 AS (

-- CIID Rows
    SELECT 
        "CI ID Rows" AS join_method,
        adobe.date_time,
        adobe.shopper_id,
        adobe.crn,
        adobe.booking_id, 
        adobe.ci_id,
        ciid_rows.media_type AS media_type, 
        ciid_rows.media_start_date AS media_start_date,
        ciid_rows.media_end_date AS media_end_date,
        ciid_rows.campaign_skus AS campaign_skus

    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe 

    -- Join Cartology Campaign Info for Adobe Rows that have CIID
    LEFT JOIN (
        SELECT 
            booking_number, 
            media_type,
            line_name, 
            media_start_date,
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        GROUP BY 1,2,3,4,5
    ) ciid_rows 
        ON ciid_rows.line_name = adobe.ci_id 
        AND adobe.booking_id = ciid_rows.booking_number
        AND adobe.date_time BETWEEN TIMESTAMP(ciid_rows.media_start_date) AND TIMESTAMP(ciid_rows.media_end_date + 1)
        AND adobe.ci_id IS NOT NULL 
        AND adobe.booking_id IS NOT NULL

    WHERE 
        -- Line Name exists
        (ciid_rows.line_name IS NOT NULL AND ciid_rows.line_name <> "") 
        AND adobe.date_time BETWEEN TIMESTAMP(ciid_rows.media_start_date) AND TIMESTAMP(ciid_rows.media_end_date + 1)
        AND adobe.ci_id IS NOT NULL 
        AND adobe.booking_id IS NOT NULL

    UNION ALL

-- Catalogue Rows 
    SELECT 
        "Catalogue" AS join_method,
        adobe2.date_time,
        adobe2.shopper_id,
        adobe2.crn,
        catalogue_rows.booking_number AS booking_id,
        adobe2.ci_id,
        catalogue_rows.media_type AS media_type, 
        catalogue_rows.media_start_date AS media_start_date,
        catalogue_rows.media_end_date AS media_end_date,
        catalogue_rows.campaign_skus AS campaign_skus

    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe2
    LEFT JOIN (
        SELECT 
            booking_number, 
            media_type, 
            line_name, 
            media_start_date, 
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        WHERE LOWER(media_type) = "catalogue"
        GROUP BY 1,2,3,4,5
    ) catalogue_rows
        ON adobe2.catalogue_flag = 1
        AND adobe2.booking_id = "Not Found"
    
    WHERE adobe2.catalogue_flag = 1
    AND adobe2.booking_id = "Not Found" 
    AND adobe2.date_time BETWEEN TIMESTAMP(catalogue_rows.media_start_date) AND TIMESTAMP(catalogue_rows.media_end_date + 1)

-- Home Page rows that have ci_id and booking_id in the adobe tracking

    UNION ALL  

    SELECT 
        "Homepage Rows" AS join_method,
        adobe3.date_time,
        adobe3.shopper_id,
        adobe3.crn,
        homepage_rows.booking_number AS booking_id,
        adobe3.ci_id,
        homepage_rows.media_type AS media_type, 
        homepage_rows.media_start_date AS media_start_date,
        homepage_rows.media_end_date AS media_end_date,
        homepage_rows.campaign_skus AS campaign_skus

    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid adobe3
    LEFT JOIN (
        SELECT 
            booking_number, 
            media_type, 
            "" AS line_name, 
            media_start_date, 
            media_end_date, 
            ARRAY_AGG(DISTINCT individual_product_string) AS campaign_skus
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns`
        WHERE LOWER(media_type) = "home page"
        GROUP BY 1,2,3,4,5
    ) homepage_rows
        ON adobe3.homepage_flag = 1
        AND adobe3.booking_id = "Not Found"
        AND adobe3.date_time BETWEEN TIMESTAMP(homepage_rows.media_start_date) AND TIMESTAMP(homepage_rows.media_end_date + 1)
    WHERE adobe3.homepage_flag = 1
    AND adobe3.booking_id = "Not Found"
    AND adobe3.date_time BETWEEN TIMESTAMP(homepage_rows.media_start_date) AND TIMESTAMP(homepage_rows.media_end_date + 1)

);


-- 4. Reduce to only the distinct shoppers events + media types / bookings numbers
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 AS 
    SELECT DISTINCT
        DATE(date_time) date,
        booking_id, 
        ci_id,
        media_type, 
        shopper_id, 
        crn, 
        ARRAY_TO_STRING(campaign_skus, ",") skus
FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v3
WHERE crn IS NOT NULL;




LOOP
    IF start_index >= end_index THEN 
        LEAVE;
    END IF;

    -- Start time for this run
    SET campaign_run_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
    SET current_campaign_global_var = campaigns_to_be_analysed_array_global_var[OFFSET(start_index)];

    DELETE FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control WHERE booking_id = current_campaign_global_var;
    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control 
        WITH campaign_dates AS (
            SELECT DISTINCT
                booking_id,
                campaign_start_date,
                campaign_end_date,
                skus
            FROM
                gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 test
            WHERE booking_id = current_campaign_global_var
        ),
        exposed_shoppers AS (
            SELECT DISTINCT
                booking_id,
                campaign_start_date,
                shopper_id,
                crn
            FROM
                gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 test
            WHERE booking_id = current_campaign_global_var
            AND crn IS NOT NULL
        ),
        activity_during_campaign AS (
            SELECT DISTINCT
                c.booking_id,
                c.campaign_start_date,
                w.shopper_id,
                w.crn
            FROM
                gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_control w
            INNER JOIN
                campaign_dates c
            ON
                w.date BETWEEN c.campaign_start_date AND c.campaign_end_date
            AND w.crn IS NOT NULL
        )
        SELECT
            adc.booking_id,
            campaign_dates.campaign_start_date,
            campaign_dates.campaign_end_date,
            'Not Exposed' AS cohort,
            NULL AS date,  -- Since non-exposure is not tied to a specific date
            adc.shopper_id,
            adc.crn,
            campaign_dates.skus
        FROM activity_during_campaign adc
        
        LEFT JOIN campaign_dates 
            ON adc.booking_id = campaign_dates.booking_id
        
        -- remove exposed shopper_ids
        LEFT JOIN (
            SELECT DISTINCT 
                booking_id,
                campaign_start_date, 
                shopper_id
            FROM exposed_shoppers 
        ) exposed_shopper_id 
            ON adc.booking_id = exposed_shopper_id.booking_id 
            AND adc.campaign_start_date = exposed_shopper_id.campaign_start_date 
            AND adc.shopper_id = exposed_shopper_id.shopper_id

        -- remove exposed crns
        LEFT JOIN (
            SELECT DISTINCT 
                booking_id,
                campaign_start_date, 
                crn
            FROM exposed_shoppers 
        ) exposed_crn
            ON adc.booking_id = exposed_crn.booking_id 
            AND adc.campaign_start_date = exposed_crn.campaign_start_date 
            AND adc.crn = exposed_crn.crn

        WHERE exposed_shopper_id.shopper_id IS NULL 
        AND exposed_crn.crn IS NULL 
        AND adc.crn IS NOT NULL
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "All Control Shoppers" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

    DELETE FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control WHERE booking_id = current_campaign_global_var;
    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control  
        SELECT       
            date,
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            "Exposed" AS cohort,
            ci_id,
            media_type, 
            shopper_id, 
            crn, 
            skus
        FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4
        WHERE booking_id = current_campaign_global_var

        UNION ALL 

        SELECT 
            CAST(NULL AS DATE) AS date, 
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            cohort, 
            "" AS ci_id, 
            "" AS media_type, 
            shopper_id, 
            crn, 
            skus
        FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control 
        WHERE booking_id = current_campaign_global_var
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "Test Control Union" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2`
        with step_one AS (
            SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` 
            WHERE exposure.campaign_start_date >= DATE("2023-10-01") 
            AND booking_id = current_campaign_global_var
        ),
        step_two AS (
            SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
            WHERE DATE(start_txn_time) >= DATE("2023-10-01") 
            AND booking_id = current_campaign_global_var
        )
        -- Exposed Shopper Transactions
        SELECT 
            DATE(exposure.date) AS impression_date, 
            exposure.booking_id, 
            exposure.cohort AS exposure_context,
            exposure.crn AS exposure_crn,
            transactions.order_context,
            transactions.shopper_identification_number AS converted_shopper_identification_number,
            transactions.start_txn_time AS start_txn_time,
            transactions.basket_key AS basket_key,
            transactions.article AS article,
            transactions.tot_net_incld_gst AS tot_net_incld_gst,
            CASE WHEN DATE(transactions.start_txn_time) < DATETIME_ADD(exposure.date, INTERVAL 3 DAY) 
                THEN "3 Days" 
                ELSE "7 Days" 
            END AS exposure_to_purchase_window
        
        FROM step_one  exposure,
        UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

        -- WHere crn is not null
        LEFT JOIN step_two transactions 
            ON exposure.crn = transactions.shopper_identification_number
            AND individual_campaign_product_string = transactions.article 
            AND 
                (
                    (
                        DATE(transactions.start_txn_time) >= campaign_start_date
                        AND DATE(transactions.start_txn_time) <= campaign_end_date
                        AND cohort = "Exposed"
                    )
                    OR 
                    ( 
                        DATE(transactions.start_txn_time) <= campaign_end_date
                        AND DATE(transactions.start_txn_time) >= campaign_start_date
                        AND exposure.cohort = "Not Exposed"
                    )
                )
            AND transactions.shopper_identification_number IS NOT NULL
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "Sku Level Sales - Test & Control" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;
    
    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2`
        with step_one AS (
            SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure
            WHERE exposure.campaign_start_date >= DATE("2023-10-01") 
            AND exposure.campaign_end_date < DATE("2024-02-07")
            AND booking_id = current_campaign_global_var
        ),
        step_two AS (
            SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
            WHERE DATE(start_txn_time) >= DATE("2023-10-01") 
            AND DATE(start_txn_time) < DATE("2024-02-14") 
            AND booking_id = current_campaign_global_var
        )
        SELECT 
            DATE(exposure.date) AS impression_date, 
            exposure.booking_id, 
            exposure.cohort AS exposure_context,
            exposure.crn AS exposure_crn,
            transactions.order_context,
            transactions.shopper_identification_number AS converted_shopper_identification_number,
            transactions.start_txn_time AS start_txn_time,
            transactions.basket_key AS basket_key,
            brand_map.brand,
            transactions.article AS article,
            transactions.tot_net_incld_gst AS tot_net_incld_gst,
            CASE WHEN DATE(transactions.start_txn_time) < DATETIME_ADD(exposure.date, INTERVAL 3 DAY) 
                THEN "3 Days" 
                ELSE "7 Days" 
            END AS exposure_to_purchase_window
        
        FROM step_one exposure,
        UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

        LEFT JOIN (
            SELECT DISTINCT
                SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
                Brand AS brand
            FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
            WHERE SalesOrganisation = "1005"
        ) brand_map
            ON brand_map.Article = individual_campaign_product_string

        -- WHere crn is not null
        LEFT JOIN step_two transactions 
            ON exposure.crn = transactions.shopper_identification_number
            AND brand_map.brand = transactions.brand 
            AND 
            (
                (
                    --DATE(transactions.start_txn_time) >= DATE(exposure.date) 
                    --AND DATE(transactions.start_txn_time) <= DATE(DATE_ADD(exposure.date, INTERVAL 7 DAY))
                    DATE(transactions.start_txn_time) >= campaign_start_date
                    AND DATE(transactions.start_txn_time) <= campaign_end_date
                    AND cohort = "Exposed"
                )
                OR 
                ( 
                    exposure.cohort = "Not Exposed"
                    AND 
                    DATE(transactions.start_txn_time) <= campaign_end_date
                    AND 
                    DATE(transactions.start_txn_time) >= campaign_start_date
                )
            )
            AND (exposure.crn IS NOT NULL)
            AND transactions.shopper_identification_number IS NOT NULL  
        WHERE brand_map.brand IS NOT NULL
        AND brand_map.brand <> "" 
        AND brand_map.brand <> "UNBRANDED" 
        AND brand_map.brand <> "OTHER"
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "Brand Level Sales - Test & Control" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;


    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2` WHERE booking_id = current_campaign_global_var; 
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2`
        SELECT DISTINCT
            booking_id, 
            order_context,
            exposure_crn AS exposed_shopper_identification_number,
            converted_shopper_identification_number,
            start_txn_time,
            basket_key, 
            article,
            tot_net_incld_gst,
            "7 Days" AS exposure_to_purchase_window
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` converted_products
        WHERE booking_id = current_campaign_global_var
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "Distinct Sku Level Sales - Test & Control" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` WHERE booking_id = current_campaign_global_var; 
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2`
        SELECT DISTINCT
            booking_id, 
            brand,
            order_context, 
            exposure_crn AS exposed_shopper_identification_number,
            converted_shopper_identification_number,
            start_txn_time,
            basket_key, 
            article,
            tot_net_incld_gst,
            "7 Days" AS exposure_to_purchase_window
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` converted_products
    ;

    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs_control`
        SELECT  
            current_campaign_global_var AS campaign_id,
            "1" AS query_step,
            "Distinct Brand Level Sales - Test & Control" AS query_type,
            query_start_time, 
            query_end_time, 
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
            DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
    ;

    ######### SHOPPER SIMILARITY FEATURES - PURCHASE HISTORY
    DELETE FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency WHERE booking_id = current_campaign_global_var;
    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency 
        with spend_and_baskets AS (
            SELECT
                booking_id,
                campaign_start_date,
                campaign_end_date, 
                cohort,
                CASE WHEN shopper_identification_number IS NULL THEN "Never Buyer" ELSE "Buyer" END AS product_customer_relationship,
                shopper_identification_method, 
                exposure.crn AS exposure_identification_number,
                shopper_identification_number,      
                start_txn_date, 
                basket_key, 
                order_context,
                --individual_campaign_product_string,
                COUNT(DISTINCT article) AS n_articles, 
                SUM(tot_net_incld_gst) AS sku_level_spend
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure,
            UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

            LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` transactions
                ON exposure.crn = transactions.shopper_identification_number
                --AND TO_HEX(SHA256(transactions.shopper_identification_number)) IS NOT NULL
                AND individual_campaign_product_string = transactions.article 

            WHERE booking_id = current_campaign_global_var
            AND (start_txn_date IS NULL OR start_txn_date <= campaign_end_date)
            AND individual_campaign_product_string IS NOT NULL 
            AND individual_campaign_product_string <> ""
            
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11
            ORDER BY shopper_identification_number, start_txn_date
        ), 
        previous_order AS (
            SELECT 
                *, 
                LAG(start_txn_date) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) AS previous_txn_date
            FROM spend_and_baskets 
            ORDER BY shopper_identification_number, start_txn_date
        ), 
        n_days_since_last_order AS (
            SELECT 
                *, 
                DATE_DIFF(start_txn_date, previous_txn_date, DAY) AS days_since_last_order
            FROM previous_order 
        )
        SELECT 
            *, 
            AVG(days_since_last_order) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rolling_avg_days_between_orders,
            AVG(sku_level_spend) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rolling_avg_sku_level_spend_per_order
        FROM n_days_since_last_order
        ORDER BY shopper_identification_number, start_txn_date
    ;

    DELETE FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level WHERE booking_id = current_campaign_global_var;
    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level 
        with spend_and_baskets AS (
            SELECT DISTINCT
                booking_id,
                campaign_start_date,
                campaign_end_date, 
                cohort,
                CASE WHEN shopper_identification_number IS NULL THEN "Never Buyer" ELSE "Buyer" END AS product_customer_relationship,
                shopper_identification_method, 
                exposure.crn AS exposure_identification_number,
                shopper_identification_number,      
                start_txn_date, 
                basket_key, 
                order_context,
                --individual_campaign_product_string,
                ARRAY_TO_STRING(brands, ", ") AS brands,
                n_articles,
                brand_level_spend
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure,
            UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

            -- brand info
            LEFT JOIN (
                SELECT 
                    SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
                    MAX(SubCategoryShortDescription) AS SubcatDescription,
                    MAX(Brand) AS BrandDescription
                FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
                WHERE SalesOrganisation = "1005"
                AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
                AND Brand IS NOT NULL
                AND SubCategoryShortDescription IS NOT NULL
                GROUP BY 1
            ) am ON am.Article = individual_campaign_product_string

            -- brand level spend
            LEFT JOIN (
                SELECT 
                    shopper_identification_method,
                    shopper_identification_number, 
                    start_txn_date,
                    basket_key, 
                    order_context, 
                    ARRAY_AGG(DISTINCT brand IGNORE NULLS) AS brands,
                    COUNT(DISTINCT article) AS n_articles, 
                    SUM(tot_net_incld_gst) AS brand_level_spend
                FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
                WHERE brand IN (
                    SELECT DISTINCT 
                        BrandDescription 
                    FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure2,
                    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string2
                    LEFT JOIN (
                        SELECT 
                            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
                            MAX(SubCategoryShortDescription) AS SubcatDescription,
                            MAX(Brand) AS BrandDescription
                        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
                        WHERE SalesOrganisation = "1005"
                        AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
                        AND Brand IS NOT NULL
                        AND SubCategoryShortDescription IS NOT NULL
                        GROUP BY 1
                    ) am2 ON am2.Article = individual_campaign_product_string2
                    WHERE  exposure2.booking_id = current_campaign_global_var
                )
                GROUP BY 1,2,3,4,5
            ) transactions
                ON exposure.crn = transactions.shopper_identification_number

            WHERE booking_id = current_campaign_global_var
            AND (start_txn_date IS NULL OR start_txn_date < campaign_start_date)
            AND individual_campaign_product_string IS NOT NULL 
            AND individual_campaign_product_string <> ""
            ORDER BY shopper_identification_number, start_txn_date
            ), 
            previous_order AS (
            SELECT 
                *, 
                CASE WHEN shopper_identification_number IS NOT NULL THEN LAG(start_txn_date) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) ELSE NULL END AS previous_txn_date
            FROM spend_and_baskets 
            ORDER BY shopper_identification_number, start_txn_date
            ), 
            n_days_since_last_order AS (
                SELECT 
                    *, 
                    DATE_DIFF(start_txn_date, previous_txn_date, DAY) AS days_since_last_order
                FROM previous_order 
            )
            SELECT 
                *, 
                CASE WHEN shopper_identification_number IS NOT NULL THEN AVG(days_since_last_order) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ELSE NULL END AS rolling_avg_days_between_orders,
                CASE WHEN shopper_identification_number IS NOT NULL THEN AVG(brand_level_spend) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ELSE NULL END AS rolling_avg_brand_level_spend_per_order
            FROM n_days_since_last_order
            ORDER BY shopper_identification_number, start_txn_date
    ;

    ############ K-Means Model Training and Application
    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features` 
        SELECT 
            exposure_crn,
            IFNULL(total_orders_brand_level, 0) AS total_orders_brand_level,
            IFNULL(brand_level_spend, 0) AS brand_level_spend,
            IFNULL(rolling_avg_days_between_orders_brand_level, 0) AS rolling_avg_days_between_orders_brand_level,
            IFNULL(rolling_avg_brand_level_spend_per_order_brand_level, 0) AS rolling_avg_brand_level_spend_per_order_brand_level,
            IFNULL(total_orders_sku_level, 0) AS total_orders_sku_level,
            IFNULL(sku_level_spend, 0) AS sku_level_spend,
            IFNULL(rolling_avg_days_between_orders_sku_level, 0) AS rolling_avg_days_between_orders_sku_level,
            IFNULL(rolling_avg_sku_level_spend_per_order_sku_level, 0) AS rolling_avg_sku_level_spend_per_order_sku_level,
            cohort
        FROM 
            `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history`
        WHERE 
            booking_id = current_campaign_global_var
            AND (cohort = "Exposed" OR cohort = "Not Exposed");


    CREATE OR REPLACE MODEL `gcp-wow-food-wlx-digaspt-dev.playground.shopper_clusters_kmeans_10_clusters`
        OPTIONS(model_type='kmeans', num_clusters=10) AS
        SELECT 
            total_orders_brand_level,
            brand_level_spend,
            rolling_avg_days_between_orders_brand_level,
            rolling_avg_brand_level_spend_per_order_brand_level,
            total_orders_sku_level,
            sku_level_spend,
            rolling_avg_days_between_orders_sku_level,
            rolling_avg_sku_level_spend_per_order_sku_level
        FROM 
            `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features`
        WHERE booking_id = current_campaign_global_var;

    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features_with_clusters` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features_with_clusters`
        SELECT
            *
        FROM 
            ML.PREDICT(MODEL `gcp-wow-food-wlx-digaspt-dev.playground.shopper_clusters_kmeans_10_clusters`,
                    (SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` WHERE booking_id = current_campaign_global_var));

    -- DOWNSAMPLING 
    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.cluster_cohort_counts` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.cluster_cohort_counts`
        SELECT
            CENTROID_ID,
            cohort,
            COUNT(DISTINCT exposure_crn) AS customer_count
        FROM
            `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features_with_clusters`
        WHERE booking_id = current_campaign_global_var
        GROUP BY
            CENTROID_ID, cohort;
    
        DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_with_random_assignment` WHERE booking_id = current_campaign_global_var;
        INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_with_random_assignment`
            with step_one AS (
                SELECT DISTINCT
                    CENTROID_ID,
                    exposure_crn,
                    booking_id,
                    cohort
                FROM
                    `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features_with_clusters`
                WHERE booking_id = current_campaign_global_var
            ) SELECT *, RAND() AS random_value FROM step_one;

        
        DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_ranked` WHERE booking_id = current_campaign_global_var;
        INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_ranked`
            SELECT
                a.CENTROID_ID,
                a.exposure_crn,
                a.booking_id,
                a.cohort,
                a.random_value,
                ROW_NUMBER() OVER (PARTITION BY a.CENTROID_ID, a.cohort ORDER BY a.random_value) AS rank
            FROM
                `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_with_random_assignment` a
            WHERE booking_id = current_campaign_global_var;

        
        DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled` WHERE booking_id = current_campaign_global_var;
        INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled`
            -- Calculate the total customer count
            WITH total_customer_count AS (
                SELECT COUNT(DISTINCT exposure_crn) AS total_count
                FROM `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_ranked`
                WHERE booking_id = current_campaign_global_var
            ),
            -- Calculate the desired percentage of each CENTROID_ID-cohort pairing
            desired_percentages AS (
                SELECT 
                    CENTROID_ID,
                    cohort,
                    customer_count,
                    customer_count / (SELECT total_count FROM total_customer_count) AS percentage
                FROM 
                    `gcp-wow-food-wlx-digaspt-dev.playground.cluster_cohort_counts`
                WHERE booking_id = current_campaign_global_var
            ),
            -- Calculate the sample size for each CENTROID_ID-cohort pairing
            sample_sizes AS (
                SELECT 
                    CENTROID_ID,
                    cohort,
                    customer_count,
                    CASE 
                        WHEN customer_count > 6000 THEN 6000
                        ELSE customer_count
                    END AS max_customers,
                    FLOOR(CASE 
                        WHEN customer_count > 6000 THEN 6000 * percentage
                        ELSE customer_count
                    END) AS sample_size
                FROM 
                    desired_percentages
            ),
            -- Perform stratified sampling
            stratified_sample AS (
                SELECT
                    a.CENTROID_ID,
                    a.exposure_crn,
                    a.booking_id,
                    a.cohort,
                    a.rank,
                    b.sample_size
                FROM
                    `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_ranked` a
                JOIN
                    sample_sizes b
                ON
                    a.CENTROID_ID = b.CENTROID_ID
                    AND a.cohort = b.cohort
                WHERE
                    a.booking_id = current_campaign_global_var
                    AND a.rank <= b.sample_size
            )
            SELECT
                CENTROID_ID,
                exposure_crn,
                booking_id,
                cohort
            FROM
                stratified_sample
            WHERE
                booking_id = current_campaign_global_var;

        DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_one` WHERE booking_id = current_campaign_global_var;
        INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_one`
            WITH exposed_step AS (
                SELECT full_set.*, downsampled.CENTROID_ID
                FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` full_set 
                INNER JOIN `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled` downsampled 
                    ON full_set.booking_id = downsampled.booking_id 
                    AND full_set.exposure_crn = downsampled.exposure_crn 
                WHERE 
                full_set.booking_id = current_campaign_global_var
                AND downsampled.exposure_crn IS NOT NULL
                AND full_set.cohort = "Exposed"
            ),
            non_exposed_step AS (
                SELECT full_set.*, downsampled.CENTROID_ID
                FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` full_set
                INNER JOIN `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled` downsampled 
                    ON full_set.booking_id = downsampled.booking_id 
                    AND full_set.exposure_crn = downsampled.exposure_crn 
                WHERE full_set.cohort = "Not Exposed"
                AND downsampled.exposure_crn IS NOT NULL
                AND full_set.booking_id = current_campaign_global_var
            )
            SELECT 
                exposed_step.*, 
                non_exposed_step.exposure_crn AS candidate_comparison_shopper,
                non_exposed_step.total_orders_brand_level AS total_orders_brand_level_ccs,
                non_exposed_step.brand_level_spend AS brand_level_spend_ccs,
                non_exposed_step.rolling_avg_days_between_orders_brand_level AS rolling_avg_days_between_orders_brand_level_ccs,
                non_exposed_step.rolling_avg_brand_level_spend_per_order_brand_level AS rolling_avg_brand_level_spend_per_order_brand_level_ccs, 
                non_exposed_step.total_orders_sku_level AS total_orders_sku_level_ccs,
                non_exposed_step.sku_level_spend AS sku_level_spend_ccs, 
                non_exposed_step.rolling_avg_days_between_orders_sku_level AS rolling_avg_days_between_orders_sku_level_ccs,
                non_exposed_step.rolling_avg_sku_level_spend_per_order_sku_level AS rolling_avg_sku_level_spend_per_order_sku_level_ccs,
                ARRAY(SELECT x FROM UNNEST([exposed_step.exposure_crn, non_exposed_step.exposure_crn]) AS x ORDER BY x) AS sorted_crn_array
            FROM exposed_step
            CROSS JOIN non_exposed_step 
                
            WHERE exposed_step.exposure_crn IS NOT NULL 
            AND exposed_step.booking_id = non_exposed_step.booking_id 
            AND exposed_step.CENTROID_ID = non_exposed_step.CENTROID_ID
            AND exposed_step.exposure_crn <> non_exposed_step.exposure_crn
            AND non_exposed_step.exposure_crn IS NOT NULL;

        DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_two` WHERE booking_id = current_campaign_global_var;
        INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_two`
        WITH sorted_crn_string AS (
            SELECT * EXCEPT(sorted_crn_array), ARRAY_TO_STRING(sorted_crn_array, ", ") AS sorted_crn_string_array
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_one` 
            WHERE booking_id = current_campaign_global_var
        ),
        unique_sorted_crn_string AS (
            SELECT DISTINCT * FROM sorted_crn_string
        ),
        min_diff AS (
            SELECT 
                *,
                ABS(total_orders_brand_level_ccs - total_orders_brand_level) AS total_orders_brand_level_diff,
                ABS(brand_level_spend_ccs - brand_level_spend) AS brand_level_spend_diff, 
                ABS(rolling_avg_days_between_orders_brand_level_ccs - rolling_avg_days_between_orders_brand_level) AS rolling_avg_days_between_orders_brand_level_diff, 
                ABS(rolling_avg_brand_level_spend_per_order_brand_level_ccs - rolling_avg_brand_level_spend_per_order_brand_level) AS rolling_avg_brand_level_spend_per_order_brand_level_diff, 
                ABS(total_orders_sku_level_ccs - total_orders_sku_level) AS total_orders_sku_level_diff, 
                ABS(sku_level_spend_ccs - sku_level_spend) AS sku_level_spend_diff, 
                ABS(rolling_avg_days_between_orders_sku_level_ccs - rolling_avg_days_between_orders_sku_level) AS rolling_avg_days_between_orders_sku_level_diff, 
                ABS(rolling_avg_sku_level_spend_per_order_sku_level_ccs - rolling_avg_sku_level_spend_per_order_sku_level) AS rolling_avg_sku_level_spend_per_order_sku_level_diff 
            FROM unique_sorted_crn_string
        ),
        ranked_diff AS (
            SELECT 
                *,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_brand_level_diff) AS total_orders_brand_level_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY brand_level_spend_diff) AS brand_level_spend_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_brand_level_diff) AS rolling_avg_days_between_orders_brand_level_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_brand_level_spend_per_order_brand_level_diff) AS rolling_avg_brand_level_spend_per_order_brand_level_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_sku_level_diff) AS total_orders_sku_level_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY sku_level_spend_diff) AS sku_level_spend_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_sku_level_diff) AS rolling_avg_days_between_orders_sku_level_diff_rank,
                ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_sku_level_spend_per_order_sku_level_diff) AS rolling_avg_sku_level_spend_per_order_sku_level_diff_rank
            FROM min_diff
        ),
        rank_sum AS (
            SELECT 
                *,
                total_orders_brand_level_diff_rank + brand_level_spend_diff_rank + rolling_avg_days_between_orders_brand_level_diff_rank + rolling_avg_brand_level_spend_per_order_brand_level_diff_rank + 
                total_orders_sku_level_diff_rank + sku_level_spend_diff_rank + rolling_avg_days_between_orders_sku_level_diff_rank + rolling_avg_sku_level_spend_per_order_sku_level_diff_rank 
                AS sum_of_diff_ranks
            FROM ranked_diff 
        )
        SELECT * FROM rank_sum
    ;

    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_three` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_three`
        with min_diffs AS (
            SELECT 
                *,
                ROW_NUMBER() OVER (PARTITION BY booking_id, CENTROID_ID, exposure_crn ORDER BY sum_of_diff_ranks) AS row_num_test 
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_two`
            WHERE booking_id = current_campaign_global_var
            ORDER BY sum_of_diff_ranks
        )
        SELECT 
            *
        FROM min_diffs 
        WHERE row_num_test = 1
    ;


    DELETE FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level` WHERE booking_id = current_campaign_global_var;
    INSERT INTO `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level`
        SELECT 
            final_pairs.booking_id, 

            final_pairs.exposure_crn, 
            total_exposed_spend_sku_level_7_day,
            total_exposed_spend_brand_level_7_day,
            
            final_pairs.candidate_comparison_shopper, 
            total_not_exposed_spend_sku_level_7_day,
            total_not_exposed_spend_brand_level_7_day,

            total_exposed_spend_sku_level_7_day - total_not_exposed_spend_sku_level_7_day AS total_spend_sku_level_7_day_uplift,
            total_exposed_spend_brand_level_7_day - total_not_exposed_spend_brand_level_7_day AS total_spend_brand_level_7_day_uplift

        FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_three` final_pairs 
        
        -- Exposed/Test Shopper - Sku Level Sales, 7 Days
        LEFT JOIN (
            SELECT 
                booking_id,
                converted_shopper_identification_number, 
                SUM(tot_net_incld_gst) AS total_exposed_spend_sku_level_7_day
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2`
            GROUP BY 1,2
        )  exposed_campaign_conversions_sku_7_day
            ON final_pairs.booking_id = exposed_campaign_conversions_sku_7_day.booking_id 
            AND final_pairs.exposure_crn = exposed_campaign_conversions_sku_7_day.converted_shopper_identification_number 


        -- Not Exposed/Control Shopper - Sku Level Sales, 7 Days
        LEFT JOIN (
            SELECT 
                booking_id,
                converted_shopper_identification_number, 
                SUM(tot_net_incld_gst) AS total_not_exposed_spend_sku_level_7_day
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2`
            GROUP BY 1,2
        )  not_exposed_campaign_conversions_sku_7_day
            ON final_pairs.booking_id = not_exposed_campaign_conversions_sku_7_day.booking_id 
            AND final_pairs.candidate_comparison_shopper = not_exposed_campaign_conversions_sku_7_day.converted_shopper_identification_number 


        -- Exposed/Test Shopper - Brand Level Sales, 7 Days
        LEFT JOIN (
            SELECT 
                booking_id,
                converted_shopper_identification_number, 
                SUM(tot_net_incld_gst) AS total_exposed_spend_brand_level_7_day
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` 
            WHERE exposure_to_purchase_window = "7 Days" 
            GROUP BY 1,2
        )  exposed_campaign_conversions_brand_7_day
            ON final_pairs.booking_id = exposed_campaign_conversions_brand_7_day.booking_id 
            AND final_pairs.exposure_crn = exposed_campaign_conversions_brand_7_day.converted_shopper_identification_number 



        -- Not Exposed/Control Shopper - Brand Level Sales, 7 Days
        LEFT JOIN (
            SELECT 
                booking_id,
                converted_shopper_identification_number, 
                SUM(tot_net_incld_gst) AS total_not_exposed_spend_brand_level_7_day
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` 
            WHERE exposure_to_purchase_window = "7 Days" 
            GROUP BY 1,2
        )  not_exposed_campaign_conversions_brand_7_day
            ON final_pairs.booking_id = not_exposed_campaign_conversions_brand_7_day.booking_id 
            AND final_pairs.candidate_comparison_shopper = not_exposed_campaign_conversions_brand_7_day.converted_shopper_identification_number 

        WHERE final_pairs.booking_id = current_campaign_global_var
    ;



  SET start_index = start_index + 1;
END LOOP;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [282]:
check = client.query(
    """
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control AS
    SELECT * FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control WHERE crn IS NOT NULL;
    """
).result()
check_df = check.to_dataframe()
check_df

""


In [284]:
check = client.query(
"""  
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control AS 
    SELECT       
        date,
        booking_id, 
        campaign_start_date,
        campaign_end_date,
        "Exposed" AS cohort,
        ci_id,
        media_type, 
        shopper_id, 
        crn, 
        skus
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4

    UNION ALL 

    SELECT 
        CAST(NULL AS DATE) AS date, 
        booking_id, 
        campaign_start_date,
        campaign_end_date,
        cohort, 
        "" AS ci_id, 
        "" AS media_type, 
        shopper_id, 
        crn, 
        skus
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_control 
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


## During Campaign Performance (Sku Level Conversion, Subcat Level Conversion)

In [ ]:
check = client.query(
"""    
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` AS (
with step_one AS (
    SELECT * FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control exposure
    WHERE exposure.date > DATE("2024-01-01") 
    AND exposure.date < DATE("2024-02-07")
),
step_two AS (
    -- In-Store Transactions 
    
    
    UNION ALL
    -- Online Transactions


    SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
    WHERE DATE(start_txn_time) >= DATE("2024-01-01") 
    AND DATE(start_txn_time) < DATE("2024-02-14") 
)
    SELECT 
        DATE(exposure.date) AS impression_date, 
        exposure.booking_id, 
        COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) AS hashed_crn,
        transactions.order_context,
        transactions.shopper_identification_number AS converted_shopper_identification_number,
        transactions.start_txn_time AS start_txn_time,
        transactions.basket_key AS basket_key,
        transactions.article AS article,
        transactions.tot_net_incld_gst AS tot_net_incld_gst,
        CASE WHEN DATE(transactions.start_txn_time) < DATETIME_ADD(exposure.date, INTERVAL 3 DAY) 
            THEN "3 Days" 
            ELSE "7 Days" 
        END AS exposure_to_purchase_window
    
    FROM step_one  exposure,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

    -- Sku Level
    -- WHere crn is not null
    LEFT JOIN step_two transactions 
        ON COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) = TO_HEX(SHA256(transactions.shopper_identification_number))
        AND individual_campaign_product_string = transactions.article 
        AND DATE(transactions.start_txn_time) >= DATE(exposure.date) 
        AND DATE(transactions.start_txn_time) <= DATE(DATE_ADD(exposure.date, INTERVAL 7 DAY))
        --AND exposure.crn IS NOT NULL 
        AND transactions.shopper_identification_number IS NOT NULL

    -- Subcat Level
);

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

In [96]:
check = client.query(
"""    

CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_sku_level AS 
    with spend_and_baskets AS (
    SELECT
        booking_id,
        campaign_start_date,
        campaign_end_date, 
        cohort,
        CASE WHEN shopper_identification_number IS NULL THEN "Never Buyer" ELSE "Buyer" END AS product_customer_relationship,
        shopper_identification_method, 
        COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) AS exposure_identification_number,
        shopper_identification_number,      
        start_txn_date, 
        basket_key, 
        order_context,
        --individual_campaign_product_string,
        COUNT(DISTINCT article) AS n_articles, 
        SUM(tot_net_incld_gst) AS sku_level_spend
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` transactions
        ON COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) = TO_HEX(SHA256(transactions.shopper_identification_number))
        AND individual_campaign_product_string = transactions.article 

    WHERE booking_id = "WOW20011282"
    AND (start_txn_date IS NULL OR start_txn_date < campaign_start_date)
    AND COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) IS NOT NULL
    AND COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) <> ""
    AND individual_campaign_product_string IS NOT NULL 
    AND individual_campaign_product_string <> ""
    --AND start_txn_date IS NOT NULL
    GROUP BY 1,2,3,4,5,6,7,8,9,10,11
    ORDER BY shopper_identification_number, start_txn_date
    ), 
    previous_order AS (
    SELECT 
        *, 
        LAG(start_txn_date) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) AS previous_txn_date
    FROM spend_and_baskets 
    ORDER BY shopper_identification_number, start_txn_date
    ), 
    n_days_since_last_order AS (
        SELECT 
            *, 
            DATE_DIFF(start_txn_date, previous_txn_date, DAY) AS days_since_last_order
        FROM previous_order 
    )
    SELECT 
        *, 
        AVG(days_since_last_order) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rolling_avg_days_between_orders,
        AVG(sku_level_spend) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS rolling_avg_sku_level_spend_per_order
    FROM n_days_since_last_order
    ORDER BY shopper_identification_number, start_txn_date
;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [126]:
check = client.query(
"""    

CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level AS 
    with spend_and_baskets AS (
    SELECT DISTINCT
        booking_id,
        campaign_start_date,
        campaign_end_date, 
        cohort,
        CASE WHEN shopper_identification_number IS NULL THEN "Never Buyer" ELSE "Buyer" END AS product_customer_relationship,
        shopper_identification_method, 
        COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) AS exposure_identification_number,
        shopper_identification_number,      
        start_txn_date, 
        basket_key, 
        order_context,
        --individual_campaign_product_string,
        ARRAY_TO_STRING(brands, ", ") AS brands,
        n_articles,
        brand_level_spend
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

    LEFT JOIN (
        SELECT 
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            MAX(SubCategoryShortDescription) AS SubcatDescription,
            MAX(Brand) AS BrandDescription
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
        AND Brand IS NOT NULL
        AND SubCategoryShortDescription IS NOT NULL
        GROUP BY 1
    ) am ON am.Article = individual_campaign_product_string

    LEFT JOIN (
        SELECT 
            shopper_identification_method,
            shopper_identification_number, 
            start_txn_date,
            basket_key, 
            order_context, 
            ARRAY_AGG(DISTINCT brand IGNORE NULLS) AS brands,
            COUNT(DISTINCT article) AS n_articles, 
            SUM(tot_net_incld_gst) AS brand_level_spend
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
        WHERE brand IN (
            SELECT DISTINCT 
                BrandDescription 
            FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` exposure2,
            UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string2
            LEFT JOIN (
                SELECT 
                    SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
                    MAX(SubCategoryShortDescription) AS SubcatDescription,
                    MAX(Brand) AS BrandDescription
                FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
                WHERE SalesOrganisation = "1005"
                AND UPPER(Brand) NOT IN ("UNBRANDED", "OTHER", "")
                AND Brand IS NOT NULL
                AND SubCategoryShortDescription IS NOT NULL
                GROUP BY 1
            ) am2 ON am2.Article = individual_campaign_product_string2
            WHERE  exposure2.booking_id = "WOW20009954"
        )
        GROUP BY 1,2,3,4,5
    ) transactions
        ON COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) = TO_HEX(SHA256(transactions.shopper_identification_number))

    WHERE booking_id = "WOW20009954"
    AND (start_txn_date IS NULL OR start_txn_date < campaign_start_date)
    AND COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) IS NOT NULL
    AND COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) <> ""
    AND individual_campaign_product_string IS NOT NULL 
    AND individual_campaign_product_string <> ""
    --AND start_txn_date IS NOT NULL
    ORDER BY shopper_identification_number, start_txn_date
    ), 
    previous_order AS (
    SELECT 
        *, 
        CASE WHEN shopper_identification_number IS NOT NULL THEN LAG(start_txn_date) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) ELSE NULL END AS previous_txn_date
    FROM spend_and_baskets 
    ORDER BY shopper_identification_number, start_txn_date
    ), 
    n_days_since_last_order AS (
        SELECT 
            *, 
            DATE_DIFF(start_txn_date, previous_txn_date, DAY) AS days_since_last_order
        FROM previous_order 
    )
    SELECT 
        *, 
        CASE WHEN shopper_identification_number IS NOT NULL THEN AVG(days_since_last_order) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ELSE NULL END AS rolling_avg_days_between_orders,
        CASE WHEN shopper_identification_number IS NOT NULL THEN AVG(brand_level_spend) OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ELSE NULL END AS rolling_avg_brand_level_spend_per_order
    FROM n_days_since_last_order
    ORDER BY shopper_identification_number, start_txn_date
;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [345]:
check = client.query(
"""    
SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` LIMIT 20
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,date,booking_id,campaign_start_date,campaign_end_date,cohort,ci_id,media_type,shopper_id,crn,skus
0,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,6732800,1100000000172203486,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
1,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,3324813,1100000000002063068,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
2,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,5987454,1100000000006254151,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
3,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,7279643,1000000000002506533,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
4,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,1846808,1100000000089694250,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
5,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,6517619,1100000000056974659,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
6,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,10113900,1100000000076850151,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
7,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,1721782,1100000000018498648,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
8,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,6810377,1100000000005058513,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"
9,NaT,WOW20005286,2024-01-17,2024-01-30,Not Exposed,,,9788331,3300000000005698649,"173945,203552,229039,231302,240850,26506,53114,652009,801323,884260,886806,886808,886811,886812,886819,886838,886847,97471,98838"


In [304]:
check = client.query(
"""   
-- ######################################################################################################################################## 
-- Loop and Date Variables
DECLARE start_index INT64 DEFAULT 0;
DECLARE end_index INT64;

DECLARE campaigns_to_be_analysed_array_global_var ARRAY<STRING>;
DECLARE current_campaign_global_var STRING;

-- Query Execution Time Logging Variables
DECLARE campaign_run_start_time DATETIME;
DECLARE query_start_time DATETIME;
DECLARE query_end_time DATETIME;


SET campaigns_to_be_analysed_array_global_var = (
    SELECT 
        ARRAY_AGG(DISTINCT campaign_id IGNORE NULLS) AS campaigns 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs` 
);


SET end_index = 2; --ARRAY_LENGTH(campaigns_to_be_analysed_array_global_var);

-- ######################################################################################################################################## 

LOOP
    IF start_index >= end_index THEN 
        LEAVE;
    END IF;
    
    -- Start time for this run
    SET campaign_run_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
    SET current_campaign_global_var = "WOW20005051"; --campaigns_to_be_analysed_array_global_var[OFFSET(start_index)];

    DELETE FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs` WHERE campaign_id = current_campaign_global_var AND query_step = "4";

    SET query_start_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
    DELETE FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats WHERE booking_id = current_campaign_global_var;
    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats 
    --CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats  AS 
        with transaction_number_sku_level AS ( 
            SELECT 
                booking_id, 
                campaign_start_date, 
                campaign_end_date, 
                cohort,
                product_customer_relationship,
                exposure_identification_number,
                shopper_identification_number,
                start_txn_date, 
                basket_key,
                order_context, 
                n_articles,
                sku_level_spend,
                rolling_avg_days_between_orders,
                rolling_avg_sku_level_spend_per_order,
                ROW_NUMBER() OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) AS txn_number
            FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency a 
            WHERE shopper_identification_number IS NOT NULL 
            AND start_txn_date < campaign_start_date
            AND booking_id = current_campaign_global_var
            ), 
            latest_entry_sku_level AS (
                SELECT 
                    booking_id,
                    campaign_start_date,
                    shopper_identification_number, 
                    MAX(txn_number) AS latest_txn
                FROM transaction_number_sku_level 
                WHERE shopper_identification_number IS NOT NULL 
                AND booking_id = current_campaign_global_var
                GROUP BY 1,2,3
            ),
            aggregations_sku_level AS (
                SELECT 
                    booking_id,
                    campaign_start_date,
                    campaign_end_date, 
                    cohort, 
                    product_customer_relationship, 
                    exposure_identification_number,
                    shopper_identification_number, 
                    MIN(start_txn_date) AS earliest_order_sku_level, 
                    MAX(start_txn_date) AS latest_order_sku_level, 
                    COUNT(DISTINCT basket_key) AS total_orders_sku_level, 
                    SUM(sku_level_spend) AS sku_level_spend    
                FROM transaction_number_sku_level 
                WHERE shopper_identification_number IS NOT NULL 
                AND booking_id = current_campaign_global_var
                GROUP BY ALL
            ),
            final_results_sku_level AS (
                SELECT 
                    aggregations_sku_level.*,
                    latest_moving_averages_sku_level.rolling_avg_days_between_orders AS rolling_avg_days_between_orders_sku_level,
                    latest_moving_averages_sku_level.rolling_avg_sku_level_spend_per_order AS rolling_avg_sku_level_spend_per_order_sku_level
                FROM aggregations_sku_level
                LEFT JOIN (
                    SELECT 
                        latest_entry_sku_level.*,
                        transaction_number_sku_level.rolling_avg_days_between_orders,
                        transaction_number_sku_level.rolling_avg_sku_level_spend_per_order
                    FROM latest_entry_sku_level
                    INNER JOIN transaction_number_sku_level
                        ON transaction_number_sku_level.booking_id = latest_entry_sku_level.booking_id 
                        AND transaction_number_sku_level.campaign_start_date = latest_entry_sku_level.campaign_start_date 
                        AND transaction_number_sku_level.shopper_identification_number = latest_entry_sku_level.shopper_identification_number
                        AND transaction_number_sku_level.txn_number = latest_entry_sku_level.latest_txn
                        AND transaction_number_sku_level.shopper_identification_number IS NOT NULL
                    WHERE latest_entry_sku_level.shopper_identification_number IS NOT NULL 
                    AND latest_entry_sku_level.booking_id = current_campaign_global_var
                ) latest_moving_averages_sku_level
                    ON latest_moving_averages_sku_level.booking_id = aggregations_sku_level.booking_id 
                    AND latest_moving_averages_sku_level.campaign_start_date = aggregations_sku_level.campaign_start_date 
                    AND latest_moving_averages_sku_level.shopper_identification_number = aggregations_sku_level.shopper_identification_number
                    AND aggregations_sku_level.shopper_identification_number IS NOT NULL
                ORDER BY shopper_identification_number
            ),

            transaction_number_brand_level AS ( 
                SELECT 
                    booking_id, 
                    campaign_start_date, 
                    campaign_end_date, 
                    cohort,
                    product_customer_relationship,
                    exposure_identification_number,
                    shopper_identification_number,
                    start_txn_date, 
                    basket_key,
                    order_context, 
                    n_articles,
                    brand_level_spend,
                    rolling_avg_days_between_orders,
                    rolling_avg_brand_level_spend_per_order,
                    ROW_NUMBER() OVER(PARTITION BY shopper_identification_number ORDER BY start_txn_date) AS txn_number
                FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level
                WHERE shopper_identification_number IS NOT NULL 
                AND start_txn_date < campaign_start_date
                AND booking_id = current_campaign_global_var
            ), 
            latest_entry_brand_level AS (
                SELECT 
                    booking_id,
                    campaign_start_date,
                    shopper_identification_number, 
                    MAX(txn_number) AS latest_txn
                FROM transaction_number_brand_level 
                WHERE shopper_identification_number IS NOT NULL 
                AND booking_id = current_campaign_global_var
                GROUP BY 1,2,3
            ),
            aggregations_brand_level AS (
                SELECT 
                    booking_id,
                    campaign_start_date,
                    campaign_end_date, 
                    cohort, 
                    product_customer_relationship, 
                    exposure_identification_number,
                    shopper_identification_number, 
                    MIN(start_txn_date) AS earliest_order_brand_level, 
                    MAX(start_txn_date) AS latest_order_brand_level, 
                    COUNT(DISTINCT basket_key) AS total_orders_brand_level, 
                    SUM(brand_level_spend) AS brand_level_spend    
                FROM transaction_number_brand_level 
                WHERE shopper_identification_number IS NOT NULL 
                AND booking_id = current_campaign_global_var
                GROUP BY ALL
            ),
            final_results_brand_level AS (
                SELECT 
                    aggregations_brand_level.*,
                    latest_moving_averages_brand_level.rolling_avg_days_between_orders AS rolling_avg_days_between_orders_brand_level,
                    latest_moving_averages_brand_level.rolling_avg_brand_level_spend_per_order AS rolling_avg_brand_level_spend_per_order_brand_level
                FROM aggregations_brand_level
                LEFT JOIN (
                    SELECT 
                        latest_entry_brand_level.*,
                        transaction_number_brand_level.rolling_avg_days_between_orders,
                        transaction_number_brand_level.rolling_avg_brand_level_spend_per_order
                    FROM latest_entry_brand_level
                    INNER JOIN transaction_number_brand_level
                        ON transaction_number_brand_level.booking_id = latest_entry_brand_level.booking_id 
                        AND transaction_number_brand_level.campaign_start_date = latest_entry_brand_level.campaign_start_date 
                        AND transaction_number_brand_level.shopper_identification_number = latest_entry_brand_level.shopper_identification_number
                        AND transaction_number_brand_level.txn_number = latest_entry_brand_level.latest_txn
                        AND transaction_number_brand_level.shopper_identification_number IS NOT NULL
                    WHERE latest_entry_brand_level.shopper_identification_number IS NOT NULL 
                    AND latest_entry_brand_level.booking_id = current_campaign_global_var
                ) latest_moving_averages_brand_level
                    ON latest_moving_averages_brand_level.booking_id = aggregations_brand_level.booking_id 
                    AND latest_moving_averages_brand_level.campaign_start_date = aggregations_brand_level.campaign_start_date 
                    AND latest_moving_averages_brand_level.shopper_identification_number = aggregations_brand_level.shopper_identification_number
                    AND aggregations_brand_level.shopper_identification_number IS NOT NULL
                ORDER BY shopper_identification_number
            )
            SELECT 
                final_results_brand_level.*,
                final_results_sku_level.earliest_order_sku_level, 
                final_results_sku_level.latest_order_sku_level, 
                final_results_sku_level.total_orders_sku_level, 
                final_results_sku_level.sku_level_spend,
                final_results_sku_level.rolling_avg_days_between_orders_sku_level,
                final_results_sku_level.rolling_avg_sku_level_spend_per_order_sku_level
            FROM final_results_brand_level 
            LEFT JOIN final_results_sku_level 
                ON final_results_brand_level.booking_id = final_results_sku_level.booking_id 
                AND final_results_brand_level.exposure_identification_number = final_results_sku_level.exposure_identification_number
                AND final_results_brand_level.shopper_identification_number = final_results_sku_level.shopper_identification_number
                AND final_results_brand_level.campaign_start_date = final_results_sku_level.campaign_start_date
            
    ;

    INSERT INTO gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats 
        SELECT DISTINCT 
            brand.booking_id, 
            brand.campaign_start_date, 
            brand.campaign_end_date, 
            brand.cohort, 
            brand.product_customer_relationship, 
            brand.exposure_identification_number,
            brand.shopper_identification_number, 
            CAST(NULL AS DATE) AS earliest_order_brand_level,
            CAST(NULL AS DATE) AS latest_order_brand_level,
            CAST(NULL AS INT64) AS total_orders_brand_level,
            CAST(NULL AS NUMERIC) AS brand_level_spend,
            CAST(NULL AS NUMERIC) AS rolling_avg_days_between_orders_brand_level,
            CAST(NULL AS NUMERIC) AS rolling_avg_brand_level_spend_per_order_brand_level,
            CAST(NULL AS DATE) AS earliest_order_sku_level,
            CAST(NULL AS DATE) AS latest_order_sku_level,
            CAST(NULL AS INT64) AS total_orders_sku_level,
            CAST(NULL AS NUMERIC) AS sku_level_spend, 
            CAST(NULL AS NUMERIC) AS rolling_avg_days_between_orders_sku_level,
            CAST(NULL AS NUMERIC) AS rolling_avg_sku_level_spend_per_order_sku_level
        FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level brand
        LEFT JOIN gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency sku
            ON brand.booking_id = sku.booking_id 
            AND brand.campaign_start_date = sku.campaign_start_date 
            AND brand.exposure_identification_number = sku.exposure_identification_number 
        WHERE brand.product_customer_relationship = "Never Buyer"
        AND brand.booking_id = current_campaign_global_var
        ;

    SET query_end_time = DATETIME(CURRENT_TIMESTAMP(), 'Australia/Sydney');
    INSERT INTO `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_run_logs`
            SELECT  
                current_campaign_global_var AS campaign_id,
                "4" AS query_step,
                "Summary Stats" AS query_type,
                query_start_time, 
                query_end_time, 
                DATETIME_DIFF(query_end_time, query_start_time, SECOND) AS query_duration_in_seconds,
                DATETIME_DIFF(query_end_time, query_start_time, SECOND) / 60 AS query_duration_in_minutes
        ;

        SET start_index = start_index + 1;
    END LOOP;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [188]:
check = client.query(
""" 
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level` AS 
    SELECT 
        booking_id, 
        exposure_crn, 
        total_exposed_spend_sku_level_7_day,
        total_exposed_spend_brand_level_7_day,
        
        candidate_comparison_shopper, 
        total_not_exposed_spend_sku_level_7_day,
        total_not_exposed_spend_brand_level_7_day,

        total_spend_sku_level_7_day_uplift,
        total_spend_brand_level_7_day_uplift
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level`
;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [193]:
check = client.query(
""" 
SELECT * 
FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
ORDER BY campaign_id, CAST(query_step AS INT64)
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20002352,1,All Control Shoppers,2024-06-28 00:50:39.615760,2024-06-28 00:50:53.159170,14,0.233333
1,WOW20002352,2,Test Control Union,2024-06-28 00:50:58.135528,2024-06-28 00:51:09.708704,11,0.183333
2,WOW20002352,3,Sku Level Sales - Test & Control,2024-06-28 00:51:13.491053,2024-06-28 00:51:47.591467,34,0.566667
3,WOW20002352,4,Brand Level Sales - Test & Control,2024-06-28 00:51:51.085323,2024-06-28 00:52:25.781788,34,0.566667
4,WOW20002352,5,Distinct Sku Level Sales - Test & Control,2024-06-28 00:52:29.048143,2024-06-28 00:52:39.591640,10,0.166667
5,WOW20002352,6,Distinct Brand Level Sales - Test & Control,2024-06-28 00:52:43.600364,2024-06-28 00:53:46.764172,63,1.050000
6,WOW20002352,7,K-Means Feature Gen - Sku Level Historical Frequency,2024-06-28 00:53:50.807455,2024-06-28 00:55:29.390156,99,1.650000
7,WOW20002352,8,K-Means Feature Gen - Brand Level Historical Frequency,2024-06-28 00:55:34.172076,2024-06-28 00:56:04.136840,30,0.500000
8,WOW20002352,9,K-Means Feature Gen - All Features,2024-06-28 00:56:47.566476,2024-06-28 00:56:56.514314,9,0.150000
9,WOW20002352,10,K-Means Model Training,2024-06-28 00:56:59.509744,2024-06-28 00:58:56.420223,117,1.950000


In [ ]:
check = client.query(
""" 
SELECT * 
FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level`
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20002352,1,All Control Shoppers,2024-06-28 00:50:39.615760,2024-06-28 00:50:53.159170,14,0.233333
1,WOW20002352,2,Test Control Union,2024-06-28 00:50:58.135528,2024-06-28 00:51:09.708704,11,0.183333
2,WOW20002352,3,Sku Level Sales - Test & Control,2024-06-28 00:51:13.491053,2024-06-28 00:51:47.591467,34,0.566667
3,WOW20002352,4,Brand Level Sales - Test & Control,2024-06-28 00:51:51.085323,2024-06-28 00:52:25.781788,34,0.566667
4,WOW20002352,5,Distinct Sku Level Sales - Test & Control,2024-06-28 00:52:29.048143,2024-06-28 00:52:39.591640,10,0.166667
5,WOW20002352,6,Distinct Brand Level Sales - Test & Control,2024-06-28 00:52:43.600364,2024-06-28 00:53:46.764172,63,1.050000
6,WOW20002352,7,K-Means Feature Gen - Sku Level Historical Frequency,2024-06-28 00:53:50.807455,2024-06-28 00:55:29.390156,99,1.650000
7,WOW20002352,8,K-Means Feature Gen - Brand Level Historical Frequency,2024-06-28 00:55:34.172076,2024-06-28 00:56:04.136840,30,0.500000
8,WOW20002352,9,K-Means Feature Gen - All Features,2024-06-28 00:56:47.566476,2024-06-28 00:56:56.514314,9,0.150000
9,WOW20002352,10,K-Means Model Training,2024-06-28 00:56:59.509744,2024-06-28 00:58:56.420223,117,1.950000


In [308]:
check = client.query(
""" 
with step_one AS (
    SELECT 
        ss.booking_id, 
        exposure_identification_number, 
        COUNT(DISTINCT ss.cohort) AS n_cohorts
    FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats ss
    --WHERE ss.booking_id = "WOW20007741"
    GROUP BY 1,2
),
step_two AS (
    SELECT 
        booking_id, 
        COUNT(DISTINCT CASE WHEN n_cohorts = 1 THEN exposure_identification_number ELSE NULL END) AS n_shoppers_with_1_cohort,
        COUNT(DISTINCT CASE WHEN n_cohorts = 2 THEN exposure_identification_number ELSE NULL END) AS n_shoppers_with_2_cohorts,
        COUNT(DISTINCT CASE WHEN n_cohorts > 2 THEN exposure_identification_number ELSE NULL END) AS n_shoppers_with_more_than_2_cohorts
    FROM step_one 
    GROUP BY 1
    ORDER BY 1
), step_three AS (
    SELECT DISTINCT booking_id FROM step_two WHERE n_shoppers_with_2_cohorts > 0
) 
SELECT * 
FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats
WHERE booking_id NOT IN (SELECT DISTINCT booking_id FROM step_three)
ORDER BY booking_id
LIMIT 50
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,campaign_start_date,campaign_end_date,cohort,product_customer_relationship,exposure_identification_number,shopper_identification_number,earliest_order_brand_level,latest_order_brand_level,total_orders_brand_level,brand_level_spend,rolling_avg_days_between_orders_brand_level,rolling_avg_brand_level_spend_per_order_brand_level,earliest_order_sku_level,latest_order_sku_level,total_orders_sku_level,sku_level_spend,rolling_avg_days_between_orders_sku_level,rolling_avg_sku_level_spend_per_order_sku_level
0,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,1100000000092795911,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
1,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,3300000000004872991,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
2,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,1100000000005409889,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
3,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,1100000000003728593,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
4,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,3300000000000633519,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
5,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,3300000000005742128,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
6,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,3300000000006486963,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
7,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,3300000000003887998,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
8,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,1100000000163968564,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None
9,WOW20003368,2024-04-03,2024-04-30,Exposed,Never Buyer,1100000000128485354,None,NaT,NaT,<NA>,None,NaN,None,NaT,NaT,<NA>,None,NaN,None


In [169]:
check = client.query(
""" 
--SELECT DISTINCT booking_id FROM `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features` LIMIT 10;
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` ORDER BY query_end_time DESC LIMIT 10;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20003383,9,K-Means Feature Gen - All Features,2024-06-27 22:53:51.899994,2024-06-27 22:53:59.654385,8,0.133333
1,WOW20003383,8,K-Means Feature Gen - Brand Level Historical Frequency,2024-06-27 22:53:08.065746,2024-06-27 22:53:39.416398,31,0.516667
2,WOW20003383,7,K-Means Feature Gen - Sku Level Historical Frequency,2024-06-27 22:52:04.062714,2024-06-27 22:53:05.064977,61,1.016667
3,WOW20003383,6,Distinct Brand Level Sales - Test & Control,2024-06-27 22:50:59.266862,2024-06-27 22:52:00.060789,61,1.016667
4,WOW20003383,5,Distinct Sku Level Sales - Test & Control,2024-06-27 22:50:45.407039,2024-06-27 22:50:55.420538,10,0.166667
5,WOW20003383,4,Brand Level Sales - Test & Control,2024-06-27 22:50:14.223794,2024-06-27 22:50:41.897644,27,0.450000
6,WOW20003383,3,Sku Level Sales - Test & Control,2024-06-27 22:49:45.089902,2024-06-27 22:50:11.075447,26,0.433333
7,WOW20003383,2,Test Control Union,2024-06-27 22:49:29.520020,2024-06-27 22:49:41.693655,12,0.200000
8,WOW20003383,1,All Control Shoppers,2024-06-27 22:49:13.213194,2024-06-27 22:49:26.166625,13,0.216667


In [183]:
check = client.query(
""" 
--SELECT DISTINCT booking_id FROM `gcp-wow-food-wlx-digaspt-dev.playground.shopper_features` LIMIT 10;
SELECT * 
FROM 
--`gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` 
`gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history`
WHERE booking_id = "WOW20003383" 
LIMIT 10;
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,exposure_crn,converted,n_converted_baskets,total_converted_spend,booking_id,campaign_start_date,campaign_end_date,cohort,product_customer_relationship,exposure_identification_number,shopper_identification_number,earliest_order_brand_level,latest_order_brand_level,total_orders_brand_level,brand_level_spend,rolling_avg_days_between_orders_brand_level,rolling_avg_brand_level_spend_per_order_brand_level,earliest_order_sku_level,latest_order_sku_level,total_orders_sku_level,sku_level_spend,rolling_avg_days_between_orders_sku_level,rolling_avg_sku_level_spend_per_order_sku_level


In [ ]:
check = client.query(
""" 
SELECT 
    summary.booking_id, 
    COUNT(DISTINCT exposure_crn) AS exposed_shoppers,
    COUNT(DISTINCT converted_shopper_identification_number) AS converted_shoppers
FROM (SELECT DISTINCT booking_id, exposure_identification_number FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats) summary 
LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` results
    ON summary.booking_id = results.booking_id 
    AND summary.exposure_identification_number = results.converted_shopper_identification_number 
GROUP BY 1
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

In [394]:
check = client.query(
""" 
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` AS (
    SELECT 
        converts.exposure_crn,
        CASE WHEN converted_shopper_identification_number IS NULL THEN 0 ELSE 1 END AS converted, 
        COUNT(DISTINCT basket_key) AS n_converted_baskets, 
        SUM(tot_net_incld_gst) AS total_converted_spend,
        summary.* 
    FROM (SELECT DISTINCT * FROM  `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2`) converts 
    LEFT JOIN gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats summary
        ON summary.booking_id = converts.booking_id 
        AND summary.exposure_identification_number = converts.exposure_crn
    WHERE summary.booking_id IN (SELECT DISTINCT booking_id FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control`)
    GROUP BY ALL
);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [ ]:
check = client.query(
""" 
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control` AS ( 
with exposed_step AS (
    SELECT *
    FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` 
    WHERE booking_id = "WOW20005286"
    AND cohort = "Exposed"
),
non_exposed_step AS (
    SELECT *
    FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` 
    WHERE booking_id = "WOW20005286"
    AND cohort = "Not Exposed"
),
crossed AS (
    SELECT 
        exposed_step.*, 
        non_exposed_step.exposure_crn AS candidate_comparison_shopper,
        non_exposed_step.total_orders_brand_level AS total_orders_brand_level_ccs,
        non_exposed_step.brand_level_spend AS brand_level_spend_ccs,
        non_exposed_step.rolling_avg_days_between_orders_brand_level AS rolling_avg_days_between_orders_brand_level_ccs,
        non_exposed_step.rolling_avg_brand_level_spend_per_order_brand_level AS rolling_avg_brand_level_spend_per_order_brand_level_ccs, 
        non_exposed_step.total_orders_sku_level AS total_orders_sku_level_ccs,
        non_exposed_step.sku_level_spend AS sku_level_spend_ccs, 
        non_exposed_step.rolling_avg_days_between_orders_sku_level AS rolling_avg_days_between_orders_sku_level_ccs,
        non_exposed_step.rolling_avg_sku_level_spend_per_order_sku_level AS rolling_avg_sku_level_spend_per_order_sku_level_ccs,
        ARRAY(SELECT x FROM UNNEST([exposed_step.exposure_crn, non_exposed_step.exposure_crn]) AS x ORDER BY x) AS sorted_crn_array
    FROM exposed_step
    LEFT JOIN non_exposed_step 
        ON exposed_step.booking_id = non_exposed_step.booking_id 
        AND exposed_step.exposure_crn <> non_exposed_step.exposure_crn
),
sorted_crn_string AS (
    SELECT * EXCEPT(sorted_crn_array), ARRAY_TO_STRING(sorted_crn_array, ", ") AS sorted_crn_string_array
    FROM crossed 
),
unique_sorted_crn_string AS (
    SELECT DISTINCT * FROM sorted_crn_string
),
min_diff AS (
    SELECT 
        *,
        ABS(total_orders_brand_level_ccs - total_orders_brand_level) AS total_orders_brand_level_diff,
        ABS(brand_level_spend_ccs - brand_level_spend) AS brand_level_spend_diff, 
        ABS(rolling_avg_days_between_orders_brand_level_ccs - rolling_avg_days_between_orders_brand_level) AS rolling_avg_days_between_orders_brand_level_diff, 
        ABS(rolling_avg_brand_level_spend_per_order_brand_level_ccs - rolling_avg_brand_level_spend_per_order_brand_level) AS rolling_avg_brand_level_spend_per_order_brand_level_diff, 
        ABS(total_orders_sku_level_ccs - total_orders_sku_level) AS total_orders_sku_level_diff, 
        ABS(sku_level_spend_ccs - sku_level_spend) AS sku_level_spend_diff, 
        ABS(rolling_avg_days_between_orders_sku_level_ccs - rolling_avg_days_between_orders_sku_level) AS rolling_avg_days_between_orders_sku_level_diff, 
        ABS(rolling_avg_sku_level_spend_per_order_sku_level_ccs - rolling_avg_sku_level_spend_per_order_sku_level) AS rolling_avg_sku_level_spend_per_order_sku_level_diff, 
    FROM unique_sorted_crn_string
), 
min_diff_rank AS (
    SELECT 
        *, 
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_brand_level_diff) AS total_orders_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY brand_level_spend_diff) AS brand_level_spend_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_brand_level_diff) AS rolling_avg_days_between_orders_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_brand_level_spend_per_order_brand_level_diff) AS rolling_avg_brand_level_spend_per_order_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_sku_level_diff) AS total_orders_sku_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY sku_level_spend_diff) AS sku_level_spend_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_sku_level_diff) AS rolling_avg_days_between_orders_sku_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_sku_level_spend_per_order_sku_level_diff) AS rolling_avg_sku_level_spend_per_order_sku_level_diff_rank
    FROM min_diff
),
min_diff_rank_sum AS (
    SELECT *, 
        total_orders_brand_level_diff_rank + brand_level_spend_diff_rank + rolling_avg_days_between_orders_brand_level_diff_rank + rolling_avg_brand_level_spend_per_order_brand_level_diff_rank + 
        total_orders_sku_level_diff_rank + sku_level_spend_diff_rank + rolling_avg_days_between_orders_sku_level_diff_rank + rolling_avg_sku_level_spend_per_order_sku_level_diff_rank 
        AS sum_of_diff_ranks
    FROM min_diff_rank 
), 
target_candidate AS (
    SELECT 
        booking_id, 
        exposure_crn, 
        MIN(sum_of_diff_ranks) AS lowest_sum_of_diff_ranks 
    FROM min_diff_rank_sum
    GROUP BY 1,2
)
SELECT 
    min_diff_rank_sum.* 
FROM min_diff_rank_sum
INNER JOIN target_candidate
    ON min_diff_rank_sum.booking_id = target_candidate.booking_id 
    AND min_diff_rank_sum.exposure_crn = target_candidate.exposure_crn 
    AND min_diff_rank_sum.sum_of_diff_ranks = target_candidate.lowest_sum_of_diff_ranks

);



"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

In [396]:
check = client.query(
""" 
SELECT 
    booking_id, 
    cohort,
    product_customer_relationship, 
    COUNT(DISTINCT exposure_crn) AS crns_in_cohort, 
    COUNT(DISTINCT CASE WHEN total_converted_spend IS NOT NULL THEN shopper_identification_number ELSE NULL END) AS converted_shoppers,
    COUNT(DISTINCT CASE WHEN total_converted_spend IS NOT NULL THEN shopper_identification_number ELSE NULL END) / COUNT(DISTINCT exposure_crn) AS ctr
FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` 
--WHERE booking_id = "WOW20005286"
WHERE product_customer_relationship = "Never Buyer"
GROUP BY 1,2,3
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,cohort,product_customer_relationship,crns_in_cohort,converted_shoppers,ctr
0,WOW20009633,Not Exposed,Never Buyer,1271138,0,0.0
1,WOW20009532,Not Exposed,Never Buyer,1222184,0,0.0
2,WOW20009323,Not Exposed,Never Buyer,912165,0,0.0
3,WOW20008903,Not Exposed,Never Buyer,1163379,0,0.0
4,WOW20008283,Not Exposed,Never Buyer,1167618,0,0.0
5,WOW20008563,Not Exposed,Never Buyer,2379501,0,0.0
6,WOW20008599,Not Exposed,Never Buyer,797127,0,0.0
7,WOW20008320,Exposed,Never Buyer,13156,0,0.0
8,WOW20007461,Not Exposed,Never Buyer,1629588,0,0.0
9,WOW20007465,Exposed,Never Buyer,200674,0,0.0


In [390]:
check = client.query(
""" 
SELECT 
    COUNT(DISTINCT CASE WHEN cohort = "Exposed" THEN exposure_crn ELSE NULL END) AS exposed_crns,
    COUNT(DISTINCT CASE WHEN cohort <> "Exposed" THEN exposure_crn ELSE NULL END) AS non_exposed_crns
FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` 
WHERE booking_id = "WOW20005286"
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,exposed_crns,non_exposed_crns
0,19824,1903008


In [391]:
19824 * 1903008

37725230592

In [ ]:
check = client.query(
""" 
with step_one AS (
    SELECT 
        ss.booking_id, 
        exposure_identification_number, 
        COUNT(DISTINCT ss.cohort) AS n_cohorts
    FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats ss
    WHERE ss.booking_id = "WOW20007741"
    GROUP BY 1,2
)
SELECT 
    ss.* 
FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats ss
LEFT JOIN step_one 
    ON step_one.booking_id = ss.booking_id 
    AND step_one.exposure_identification_number = ss.exposure_identification_number 
WHERE step_one.n_cohorts > 1
ORDER BY booking_id, exposure_identification_number
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,campaign_start_date,campaign_end_date,cohort,product_customer_relationship,exposure_identification_number,shopper_identification_number,earliest_order_brand_level,latest_order_brand_level,total_orders_brand_level,brand_level_spend,rolling_avg_days_between_orders_brand_level,rolling_avg_brand_level_spend_per_order_brand_level,earliest_order_sku_level,latest_order_sku_level,total_orders_sku_level,sku_level_spend,rolling_avg_days_between_orders_sku_level,rolling_avg_sku_level_spend_per_order_sku_level


In [ ]:
check = client.query(
""" 
SELECT * FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats
WHERE booking_id = "WOW20007741"
LIMIT 20
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

In [301]:
check = client.query(
"""
SELECT *, TO_HEX(SHA256(crn)) AS my_hash
FROM  `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control`
WHERE TO_HEX(SHA256(crn)) = "00003e627ebee394e0e4b224b7791382f4d16c686cf28c1a60339c578d70df2b"
AND booking_id = "WOW20007741"
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,date,booking_id,campaign_start_date,campaign_end_date,cohort,ci_id,media_type,shopper_id,crn,skus,my_hash
0,2024-02-20,WOW20005051,2024-02-14,2024-03-12,Exposed,CI-10115252,Results Page Display,4281025,1100000000004830303,"103774,289456,331591,331602,341277,51639,690125,690672",00003e627ebee394e0e4b224b7791382f4d16c686cf28c1a60339c578d70df2b


In [279]:
check = client.query(
"""
SELECT COUNT(DISTINCT hashed_crn) AS n_hashed_crns
FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid
WHERE 
    hashed_crn IS NOT NULL 
    AND crn IS NOT NULL 
    AND TO_HEX(SHA256(crn)) <> hashed_crn
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,n_hashed_crns
0,121201


In [273]:
check = client.query(
"""
CREATE OR REPLACE TABLE gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_check AS (
    with 
    
    cartology_campaign_asset_impressions_in_adobe AS (
        SELECT 
            date_time,
            shopper_id_map.shopper_id, 
            crn_map.crn, 
            post_evar37 AS hashed_crn,
            prop9,
            post_prop9,
            --CONCAT( CAST( post_visid_high AS STRING), CAST( post_visid_low AS STRING) , "_", CAST( visit_num AS STRING)) AS session_id,
            --site_platform, 
            --post_channel,
            post_evar6 AS page_name,
            --post_evar7 AS site_section, 
            --post_evar8 AS page_type,
            CASE WHEN SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue' THEN 1 ELSE 0 END AS catalogue_flag,    
            CASE WHEN post_evar6 = 'ww-sm:homepage' THEN 1 ELSE 0 END AS homepage_flag, 
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(0)] ELSE "Not Found" END AS booking_id,
            CASE WHEN post_mvvar2 IS NOT NULL THEN SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] ELSE NULL END AS ci_id
            --post_mvvar1, 
            --post_mvvar2,
            --post_mvvar3
            
        FROM `gcp-wow-food-wlx-digaspt-dev.prod_adobe_data.grs_adobe` adobe

        -- shopper id
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_shopperid_map_window` shopper_id_map
            ON adobe.prop9 = shopper_id_map.visitor_id
            AND adobe.date_time BETWEEN shopper_id_map.effective_time AND shopper_id_map.expiry_time
        
        -- crn
        LEFT JOIN `gcp-wow-rwds-ai-mlt-evs-prod.event_store.tealium_visitorid_crn_map_window` crn_map
            ON adobe.prop9 = crn_map.visitor_id
            AND adobe.date_time BETWEEN crn_map.effective_time and crn_map.expiry_time

        WHERE 
            site_name = "Supermarkets"
        AND 
            DATE(date_time) >= DATE("2024-01-01")
        AND    
            DATE(date_time) < DATE("2024-02-01")
        AND         
            (
                -- Catalogue
                (SPLIT(post_evar6,":")[SAFE_OFFSET(2)] = 'catalogue' OR LOWER(post_evar7) = 'catalogue')
                OR 
                -- Homepage
                (post_evar6 = 'ww-sm:homepage') 
                OR 
                -- ci_id 
                (SPLIT(post_mvvar2,":")[SAFE_OFFSET(3)] IS NOT NULL)
            )
        AND      
            post_channel NOT LIKE "Order Confirmation Section"
        AND 
            (shopper_id_map.shopper_id IS NOT NULL OR post_evar37 IS NOT NULL)
    
    ) 

    SELECT * FROM cartology_campaign_asset_impressions_in_adobe 

);
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [280]:
check = client.query(
""" 
with step_one AS (

    SELECT DISTINCT  
        booking_id,
        prop9,
        post_prop9, 
        shopper_id,
        crn,
        hashed_crn 
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_check
    --WHERE booking_id = "WOW20007741"
    --AND crn = "3300000000008413170"
    --AND hashed_crn = "02169dca063420b79a84c14988c711e1aa5f8df6d3a90625eee8c46093f81ccb"
    WHERE crn IS NOT NULL 
    AND hashed_crn IS NOT NULL 
    AND TO_HEX(SHA256(crn)) <> hashed_crn
    ORDER BY 1,2
    
    --gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_control
)
SELECT *, TO_HEX(SHA256(crn)) AS my_hash FROM step_one

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,prop9,post_prop9,shopper_id,crn,hashed_crn,my_hash
0,,01621e97530a0075254c9c479af004073001e06b0086e,01621e97530a0075254c9c479af004073001e06b0086e,3898772,1100000000063903358,aa36d734addbf457742bd23e8d0a9cbeeb77242e73cbc185bf4d989f76cd787c,634a77c3c29cb7c2139ed5b1e646d81d1812e6f268a549de6ea25b1a948e3276
1,,01622d47edc8001336f3886681e9010810101079009e7,01622d47edc8001336f3886681e9010810101079009e7,4107317,1100000000005590731,a44f765e213533b729a0495533311aa2d1390928a2fdffd2a5f5dbef5a8b84e9,a4f2637a6f5ad7a656495bb967e3cb1784bfd17c5235457e52b88e3447192c87
2,,0164ab61c9e6001305e894e7e8420108100e307900bd0,0164ab61c9e6001305e894e7e8420108100e307900bd0,7044715,1000000000001031836,aae8952e1f9d9a34c7b631b28d9611477894f783c32b8d9a944d997c87b77bca,452508cb5fde12a13efc84fa8f6dcaa67c39478096d5e3100f88133bc2869a37
3,,0164fe85129c001e40f866d1977001081001e07900bd0,0164fe85129c001e40f866d1977001081001e07900bd0,4913741,1100000000012998817,fd614323d81a62134d16392e116059ff09d003a8a0a5e45af646ba52f5221852,56dab38cd1e7e3a64400a077a76af22b9417ced187d4830e164bf0541af47c61
4,,0166b28a69eb0015707fa0fca16202075001e06d0093c,0166b28a69eb0015707fa0fca16202075001e06d0093c,2837556,1100000000090951671,a6c779b74aacd4369cd62d2afa511784fe21c0c0d3ed7b77a51213c37505570d,f9c06b7ea98e56cb371b339267b34ca8019af6185da5792e28be16990020d2e7
...,...,...,...,...,...,...,...
182134,WOW20009856,018c45bf4db30013d3892cf0c0d70508b001e08300536,018c45bf4db30013d3892cf0c0d70508b001e08300536,4100490,1100000000004633002,1128d0046360ce49a35e10470c19dbf157bdc073894dfdd1014c4dbc32967bef,c0423d99aaca80297093134abebf2cfd217e301dbf3dbddd1ab936a2d12dcfcb
182135,WOW20009856,018c83db7825000c20f4aa0887f70508b001e083004ea,018c83db7825000c20f4aa0887f70508b001e083004ea,10791973,1100000000130247327,7f97599f47f56e2d1270846962134b29e38d58b534c24721b21a75ba62ab5a69,230e16465a473829759751f2bad4a11fb07fbb61ed49b86c28a666e8540b4f0f
182136,WOW20009856,018c945a8707004361c9b8e14ce805087005907f0042a,018c945a8707004361c9b8e14ce805087005907f0042a,5722063,1100000000105904830,713551e326b1e7ff3bce359069bd70a67adf34a3d9f42d2d1404f9a99e01eae9,41a79b919e76140c437175397a7f013feb1c315f5576d928c98a2054db26217c
182137,WOW20009856,018ccda13972001419c17a8c187b0506f004b06700bd0,018ccda13972001419c17a8c187b0506f004b06700bd0,12134614,3300000000008515000,00dba5fe93d7fcfa648ae75ffa84fa336c00aae1569a31fead477228964f2206,c12324eaeb791798bbd5e17eb05edb1dda5397232d6b26c94d2d8adc52d2a103


In [281]:
check = client.query(
""" 


    SELECT 
        COUNT(DISTINCT CASE WHEN hashed_crn IS NULL THEN prop9 ELSE NULL END) AS n_visitors_no_hashed_crn,
        COUNT(DISTINCT CASE WHEN prop9 IS NULL THEN hashed_crn ELSE NULL END) AS n_hashed_crn_no_visitor_id
    FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_check
    WHERE (hashed_crn IS NULL AND prop9 IS NOT NULL) 
        OR (prop9 IS NULL AND hashed_crn IS NOT NULL)
    --AND crn = "3300000000008413170"
    --AND hashed_crn = "02169dca063420b79a84c14988c711e1aa5f8df6d3a90625eee8c46093f81ccb"
    --gcp-wow-food-wlx-digaspt-dev.playground.adobe_carto_catalogue_homepage_ciid_control


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,n_visitors_no_hashed_crn,n_hashed_crn_no_visitor_id
0,958873,2028


In [238]:
check = client.query(
""" 
SELECT 
    booking_id, 
    cohort,
    product_customer_relationship, 
    COUNT(DISTINCT exposure_identification_number) AS n_shoppers
FROM gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_shopper_frequency_brand_level  --gcp-wow-food-wlx-digaspt-dev.playground.cartology_dx_summary_stats
WHERE booking_id = "WOW20007741"
GROUP BY 1,2,3
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,cohort,product_customer_relationship,n_shoppers
0,WOW20007741,Not Exposed,Never Buyer,2754295
1,WOW20007741,Exposed,Never Buyer,10182
2,WOW20007741,Exposed,Buyer,196
3,WOW20007741,Not Exposed,Buyer,23778


In [367]:
check = client.query(
""" 
with step_one AS (
SELECT 
    booking_id,
    crn,
    COUNT(DISTINCT cohort) AS n_cohorts
FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control`
GROUP BY 1,2
),
step_two AS (
SELECT 
    booking_id, 
    COUNT(DISTINCT CASE WHEN n_cohorts = 1 THEN crn ELSE NULL END) AS n_crns_one_cohort,
    COUNT(DISTINCT CASE WHEN n_cohorts = 2 THEN crn ELSE NULL END) AS n_crns_two_cohorts,
    COUNT(DISTINCT CASE WHEN n_cohorts > 2 THEN crn ELSE NULL END) AS n_crns_more_than_two_cohorts
FROM step_one 
GROUP BY 1
) SELECT * FROM step_two WHERE n_crns_two_cohorts > 0
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,n_crns_one_cohort,n_crns_two_cohorts,n_crns_more_than_two_cohorts


In [372]:
check = client.query(
""" 
with step_one AS (
SELECT 
    booking_id,
    exposure_crn,
    COUNT(DISTINCT exposure_context) AS n_cohorts
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` 
GROUP BY 1,2
),
step_two AS (
SELECT 
    booking_id, 
    COUNT(DISTINCT CASE WHEN n_cohorts = 1 THEN exposure_crn ELSE NULL END) AS n_crns_one_cohort,
    COUNT(DISTINCT CASE WHEN n_cohorts = 2 THEN exposure_crn ELSE NULL END) AS n_crns_two_cohorts,
    COUNT(DISTINCT CASE WHEN n_cohorts > 2 THEN exposure_crn ELSE NULL END) AS n_crns_more_than_two_cohorts
FROM step_one 
GROUP BY 1
) SELECT * FROM step_two WHERE n_crns_two_cohorts > 0
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,n_crns_one_cohort,n_crns_two_cohorts,n_crns_more_than_two_cohorts


In [379]:
check = client.query(
""" 

SELECT 
    booking_id,
    COUNT(DISTINCT CASE WHEN exposure_context = "Exposed" THEN converted_shopper_identification_number ELSE NULL END) AS n_crns_exposed,
    COUNT(DISTINCT CASE WHEN exposure_context <> "Exposed" THEN converted_shopper_identification_number ELSE NULL END) AS n_crns_not_exposed
FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` 
GROUP BY 1
LIMIT 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,n_crns_exposed,n_crns_not_exposed
0,WOW20006796,3735,52658
1,WOW20008554,7,9202
2,WOW20008904,1534,416224
3,WOW20006683,5432,72096
4,WOW20007085,14145,166681


In [378]:
check = client.query(
""" 

SELECT 
    *
FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control` 
WHERE booking_id = "WOW20006912"
AND cohort = "Not Exposed"
LIMIT 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,date,booking_id,campaign_start_date,campaign_end_date,cohort,ci_id,media_type,shopper_id,crn,skus
0,NaT,WOW20006912,2024-01-10,2024-02-06,Not Exposed,,,2715961,1100000000083273885,"104400,104409,110534,110543,174718,176313,193729,222491,222777,223033,225624,307614,63584,63927,65887,96938,97275"
1,NaT,WOW20006912,2024-01-10,2024-02-06,Not Exposed,,,None,1100000000103725338,"104400,104409,110534,110543,174718,176313,193729,222491,222777,223033,225624,307614,63584,63927,65887,96938,97275"
2,NaT,WOW20006912,2024-01-10,2024-02-06,Not Exposed,,,4824080,1100000000135432565,"104400,104409,110534,110543,174718,176313,193729,222491,222777,223033,225624,307614,63584,63927,65887,96938,97275"
3,NaT,WOW20006912,2024-01-10,2024-02-06,Not Exposed,,,1600775,1100000000002160219,"104400,104409,110534,110543,174718,176313,193729,222491,222777,223033,225624,307614,63584,63927,65887,96938,97275"
4,NaT,WOW20006912,2024-01-10,2024-02-06,Not Exposed,,,2469427,1100000000000956483,"104400,104409,110534,110543,174718,176313,193729,222491,222777,223033,225624,307614,63584,63927,65887,96938,97275"


In [233]:
check = client.query(
"""    
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` AS (
with step_one AS (
    SELECT * FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4 exposure
    WHERE exposure.date > DATE("2024-01-01") 
    AND exposure.date < DATE("2024-02-07")
),
step_two AS (
    SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.test_transactions` 
    WHERE DATE(start_txn_time) >= DATE("2024-01-01") 
    AND DATE(start_txn_time) < DATE("2024-02-14") 
)
    SELECT 
        DATE(exposure.date) AS impression_date, 
        exposure.booking_id, 
        COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) AS hashed_crn,
        transactions.order_context,
        transactions.shopper_identification_number AS converted_shopper_identification_number,
        transactions.start_txn_time AS start_txn_time,
        transactions.basket_key AS basket_key,
        brand_map.brand,
        transactions.article AS article,
        transactions.tot_net_incld_gst AS tot_net_incld_gst,
        CASE WHEN DATE(transactions.start_txn_time) < DATETIME_ADD(exposure.date, INTERVAL 3 DAY) 
            THEN "3 Days" 
            ELSE "7 Days" 
        END AS exposure_to_purchase_window
    
    FROM step_one exposure,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string

    LEFT JOIN (
        SELECT DISTINCT
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            Brand AS brand
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
    ) brand_map
        ON brand_map.Article = individual_campaign_product_string

    -- WHere crn is not null
    LEFT JOIN step_two transactions 
        ON COALESCE(TO_HEX(SHA256(exposure.crn)), exposure.hashed_crn) = TO_HEX(SHA256(transactions.shopper_identification_number))
        AND brand_map.brand = transactions.brand 
        AND DATE(transactions.start_txn_time) >= DATE(exposure.date) 
        AND DATE(transactions.start_txn_time) <= DATE(DATE_ADD(exposure.date, INTERVAL 7 DAY))
        AND (exposure.crn IS NOT NULL OR exposure.hashed_crn IS NOT NULL)
        AND transactions.shopper_identification_number IS NOT NULL  
    WHERE brand_map.brand IS NOT NULL
    AND brand_map.brand <> "" 
    AND brand_map.brand <> "UNBRANDED" 
);

"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [214]:
check = client.query(
"""    
-- 6. Campaign Exposed Sales 7 Days
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2` AS
    SELECT DISTINCT
        booking_id, 
        order_context,
        --quoteline_id, 
        --asset_type, 
        hashed_crn AS exposed_shopper_identification_number,
        converted_shopper_identification_number,
        start_txn_time,
        basket_key, 
        article,
        tot_net_incld_gst,
        "7 Days" AS exposure_to_purchase_window
        --ARRAY_AGG(converted_products.session_id IGNORE NULLS) AS session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "App" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS app_session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "Web" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS web_session_ids,
        /*
        ARRAY_AGG(TO_JSON_STRING(STRUCT(
            converted_products.impression_time AS event_time,
            IFNULL(converted_products.session_id, "unknown") AS event_session_id,
            --IFNULL(converted_products.event_type, "unknown") AS event_type,
            --IFNULL(converted_products.product_source, "unknown") AS event_product_source,
            --IFNULL(converted_products.platform, "unknown") AS event_platform, 
            IFNULL(converted_products.site_section, "unknown") AS event_site_section,
            IFNULL(converted_products.page_name, "unknown") AS event_page_name,
            IFNULL(converted_products.post_channel, "unknown") AS event_post_channel
            --IFNULL(converted_products.page_type, "unknown") AS event_page_type,
            --ARRAY_TO_STRING(impression_products, ",") AS impression_products,
            --ARRAY_TO_STRING(impression_ids, ",") as impression_ids
        ))
        ORDER BY converted_products.impression_time
        ) AS events
        */
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` converted_products
    --WHERE exposure_to_purchase_window = "7 Days"
    --GROUP BY 1,2,3,4,5,6
;


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [228]:
check = client.query(
"""    
-- Campaign Exposed Sales 3 Days (Sku Level)
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_sku_level_v2` AS
    SELECT DISTINCT
        booking_id, 
        order_context,
        --quoteline_id, 
        --asset_type, 
        hashed_crn AS exposed_shopper_identification_number,
        converted_shopper_identification_number,
        start_txn_time,
        basket_key, 
        article,
        tot_net_incld_gst,
        "7 Days" AS exposure_to_purchase_window
        --ARRAY_AGG(converted_products.session_id IGNORE NULLS) AS session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "App" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS app_session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "Web" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS web_session_ids,
        /*
        ARRAY_AGG(TO_JSON_STRING(STRUCT(
            converted_products.impression_time AS event_time,
            IFNULL(converted_products.session_id, "unknown") AS event_session_id,
            --IFNULL(converted_products.event_type, "unknown") AS event_type,
            --IFNULL(converted_products.product_source, "unknown") AS event_product_source,
            --IFNULL(converted_products.platform, "unknown") AS event_platform, 
            IFNULL(converted_products.site_section, "unknown") AS event_site_section,
            IFNULL(converted_products.page_name, "unknown") AS event_page_name,
            IFNULL(converted_products.post_channel, "unknown") AS event_post_channel
            --IFNULL(converted_products.page_type, "unknown") AS event_page_type,
            --ARRAY_TO_STRING(impression_products, ",") AS impression_products,
            --ARRAY_TO_STRING(impression_ids, ",") as impression_ids
        ))
        ORDER BY converted_products.impression_time
        ) AS events
        */
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_sku_level_v2` converted_products
    WHERE (exposure_to_purchase_window = "3 Days" OR converted_shopper_identification_number IS NULL)
    --GROUP BY 1,2,3,4,5,6
;


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [234]:
check = client.query(
"""    
-- 6. Campaign Exposed Sales 7 Days - Brand Level
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` AS
    SELECT DISTINCT
        booking_id, 
        brand,
        order_context,
        --quoteline_id, 
        --asset_type, 
        hashed_crn AS exposed_shopper_identification_number,
        converted_shopper_identification_number,
        start_txn_time,
        basket_key, 
        article,
        tot_net_incld_gst,
        "7 Days" AS exposure_to_purchase_window
        --ARRAY_AGG(converted_products.session_id IGNORE NULLS) AS session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "App" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS app_session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "Web" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS web_session_ids,
        /*
        ARRAY_AGG(TO_JSON_STRING(STRUCT(
            converted_products.impression_time AS event_time,
            IFNULL(converted_products.session_id, "unknown") AS event_session_id,
            --IFNULL(converted_products.event_type, "unknown") AS event_type,
            --IFNULL(converted_products.product_source, "unknown") AS event_product_source,
            --IFNULL(converted_products.platform, "unknown") AS event_platform, 
            IFNULL(converted_products.site_section, "unknown") AS event_site_section,
            IFNULL(converted_products.page_name, "unknown") AS event_page_name,
            IFNULL(converted_products.post_channel, "unknown") AS event_post_channel
            --IFNULL(converted_products.page_type, "unknown") AS event_page_type,
            --ARRAY_TO_STRING(impression_products, ",") AS impression_products,
            --ARRAY_TO_STRING(impression_ids, ",") as impression_ids
        ))
        ORDER BY converted_products.impression_time
        ) AS events
        */
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` converted_products
    --WHERE exposure_to_purchase_window = "7 Days"
    --GROUP BY 1,2,3,4,5,6
;


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [235]:
check = client.query(
"""    
-- 6. Campaign Exposed Sales 7 Days - Brand Level
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_brand_level_v2` AS
    SELECT DISTINCT
        booking_id, 
        brand,
        order_context,
        --quoteline_id, 
        --asset_type, 
        hashed_crn AS exposed_shopper_identification_number,
        converted_shopper_identification_number,
        start_txn_time,
        basket_key, 
        article,
        tot_net_incld_gst,
        "7 Days" AS exposure_to_purchase_window
        --ARRAY_AGG(converted_products.session_id IGNORE NULLS) AS session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "App" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS app_session_ids,
        --ARRAY_AGG(DISTINCT CASE WHEN converted_products.platform = "Web" THEN converted_products.session_id ELSE NULL END IGNORE NULLS) AS web_session_ids,
        /*
        ARRAY_AGG(TO_JSON_STRING(STRUCT(
            converted_products.impression_time AS event_time,
            IFNULL(converted_products.session_id, "unknown") AS event_session_id,
            --IFNULL(converted_products.event_type, "unknown") AS event_type,
            --IFNULL(converted_products.product_source, "unknown") AS event_product_source,
            --IFNULL(converted_products.platform, "unknown") AS event_platform, 
            IFNULL(converted_products.site_section, "unknown") AS event_site_section,
            IFNULL(converted_products.page_name, "unknown") AS event_page_name,
            IFNULL(converted_products.post_channel, "unknown") AS event_post_channel
            --IFNULL(converted_products.page_type, "unknown") AS event_page_type,
            --ARRAY_TO_STRING(impression_products, ",") AS impression_products,
            --ARRAY_TO_STRING(impression_ids, ",") as impression_ids
        ))
        ORDER BY converted_products.impression_time
        ) AS events
        */
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2` converted_products
    WHERE (exposure_to_purchase_window = "3 Days" OR converted_shopper_identification_number IS NULL)
    --GROUP BY 1,2,3,4,5,6
;


"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


After LOWER(TRIM()) on `brand` join

In [236]:

check = client.query(
    """ 
-- 8. 7 Day Purchase Window Aggregated by booking_id
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_sku_level_v2` AS (
    SELECT 
        booking_id,
        --asset_type,
        COUNT(DISTINCT sd.exposed_shopper_identification_number) AS exposed_shoppers_7_days,
        COUNT(DISTINCT sd.converted_shopper_identification_number) AS converted_shoppers_7_days,
        COUNT(DISTINCT sd.basket_key) AS baskets_7_days,
        SUM(CASE WHEN order_context = "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_online_7_days,
        SUM(CASE WHEN order_context <> "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_instore_7_days
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_sku_level_v2` sd
    GROUP BY 1
);

CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_brand_level_v2` AS (
    SELECT 
        booking_id,
        ARRAY_AGG(DISTINCT brand IGNORE NULLS ORDER BY brand) AS brands,
        --asset_type,
        COUNT(DISTINCT sd.exposed_shopper_identification_number) AS exposed_shoppers_7_days,
        COUNT(DISTINCT sd.converted_shopper_identification_number) AS converted_shoppers_7_days,
        COUNT(DISTINCT sd.basket_key) AS baskets_7_days,
        SUM(CASE WHEN order_context = "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_online_7_days,
        SUM(CASE WHEN order_context <> "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_instore_7_days
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_brand_level_v2` sd
    GROUP BY 1
);

-- 9. 3 Day Purchase Window Aggregated by booking_id
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_sku_level_v2` AS (
    SELECT 
        booking_id, 
        --asset_type,
        COUNT(DISTINCT sd.exposed_shopper_identification_number) AS exposed_shoppers_3_days,
        COUNT(DISTINCT sd.converted_shopper_identification_number) AS converted_shoppers_3_days,
        COUNT(DISTINCT sd.basket_key) AS baskets_3_days,
        SUM(CASE WHEN order_context = "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_online_3_days,
        SUM(CASE WHEN order_context <> "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_instore_3_days
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_sku_level_v2` sd 
    GROUP BY 1
);


CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_brand_level_v2` AS (
    SELECT 
        booking_id, 
        ARRAY_AGG(DISTINCT brand IGNORE NULLS ORDER BY brand) AS brands,
        --asset_type,
        COUNT(DISTINCT sd.exposed_shopper_identification_number) AS exposed_shoppers_3_days,
        COUNT(DISTINCT sd.converted_shopper_identification_number) AS converted_shoppers_3_days,
        COUNT(DISTINCT sd.basket_key) AS baskets_3_days,
        SUM(CASE WHEN order_context = "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_online_3_days,
        SUM(CASE WHEN order_context <> "Online Order" THEN sd.tot_net_incld_gst ELSE 0 END) AS sales_amt_instore_3_days
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_brand_level_v2` sd 
    GROUP BY 1
);

    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [237]:

check = client.query(
    """ 
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_vs_7_days` AS (
    SELECT 
        sd_sku.booking_id,
        sd_brand.brands, 
        sd_sku.exposed_shoppers_7_days exposed_shoppers,
        sd_sku.converted_shoppers_7_days AS sku_level_converted_shoppers_7_days,
        sd_brand.converted_shoppers_7_days AS brand_level_converted_shoppers_7_days,
        SAFE_DIVIDE(sd_sku.converted_shoppers_7_days  , sd_sku.exposed_shoppers_7_days) AS sku_level_conversion_rate_7_days,
        SAFE_DIVIDE(sd_brand.converted_shoppers_7_days  , sd_sku.exposed_shoppers_7_days) AS brand_level_conversion_rate_7_days,
        td_sku.exposed_shoppers_3_days, 
        td_sku.converted_shoppers_3_days AS sku_level_converted_shoppers_3_days,
        td_brand.converted_shoppers_3_days AS brand_level_converted_shoppers_3_days,
        SAFE_DIVIDE(td_sku.converted_shoppers_3_days ,  td_sku.exposed_shoppers_3_days) AS sku_level_conversion_rate_3_days,
        SAFE_DIVIDE(td_brand.converted_shoppers_3_days ,  td_sku.exposed_shoppers_3_days) AS brand_level_conversion_rate_3_days,

        sd_sku.baskets_7_days AS sku_level_baskets_7_days,
        sd_brand.baskets_7_days AS brand_level_baskets_7_days,
        td_sku.baskets_3_days AS sku_level_baskets_3_days,
        td_brand.baskets_3_days AS brand_level_baskets_3_days,

        sd_sku.sales_amt_online_7_days AS sku_level_sales_amt_online_7_days,
        sd_brand.sales_amt_online_7_days AS brand_level_sales_amt_online_7_days,
        td_sku.sales_amt_online_3_days AS sku_level_sales_amt_online_3_days,
        td_brand.sales_amt_online_3_days AS brand_level_sales_amt_online_3_days,
        
        sd_sku.sales_amt_instore_7_days AS sku_level_sales_amt_instore_7_days,
        sd_brand.sales_amt_instore_7_days AS brand_level_sales_amt_instore_7_days,
        td_sku.sales_amt_instore_3_days AS sku_level_sales_amt_instore_3_days,
        td_brand.sales_amt_instore_3_days AS brand_level_sales_amt_instore_3_days

    FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_sku_level_v2` sd_sku 
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_7_days_aggregated_brand_level_v2` sd_brand
        ON sd_sku.booking_id = sd_brand.booking_id 
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_sku_level_v2` td_sku
        ON sd_sku.booking_id = td_sku.booking_id
    LEFT JOIN `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_aggregated_brand_level_v2` td_brand
        ON sd_sku.booking_id = td_brand.booking_id
);
    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

""


In [243]:

check = client.query(
    """ 
    --CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_3_vs_7_days` AS 
        SELECT * EXCEPT(exposed_shoppers_3_days) 
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_vs_7_days` 
        WHERE booking_id <> "WOW20007609_WOW20007607_WOW20007606_WOW20008226_WOW20007648_WOW20007655_WOW20007608_WOW20005378_WOW20007605_WOW20007611_WOW20007610_WOW20007647"
        ORDER BY booking_id 
    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,brands,exposed_shoppers,sku_level_converted_shoppers_7_days,brand_level_converted_shoppers_7_days,sku_level_conversion_rate_7_days,brand_level_conversion_rate_7_days,sku_level_converted_shoppers_3_days,brand_level_converted_shoppers_3_days,sku_level_conversion_rate_3_days,brand_level_conversion_rate_3_days,sku_level_baskets_7_days,brand_level_baskets_7_days,sku_level_baskets_3_days,brand_level_baskets_3_days,sku_level_sales_amt_online_7_days,brand_level_sales_amt_online_7_days,sku_level_sales_amt_online_3_days,brand_level_sales_amt_online_3_days,sku_level_sales_amt_instore_7_days,brand_level_sales_amt_instore_7_days,sku_level_sales_amt_instore_3_days,brand_level_sales_amt_instore_3_days
0,WOW20004498,"[AGRUM, BUBLY, COOL RIDGE, COTTEE'S, DIET PEPSI, FRANTELLE, GATORADE, LIPTON, MOUNTAIN DEW, PASSIONA, PEPSI, PEPSI MAX, POP TOPS, SCHWEPPES, SOLO, SUNKIST, T2, VIBE]",99713,25285,28322,0.253578,0.284035,18797,20945,0.188511,0.210053,37031,41535,23226,25909,542281.690000000,570875.640000000,384942.050000000,404364.490000000,72670.560000000,77927.990000000,29634.890000000,31486.160000000
1,WOW20004681,[DAIRY FARMERS],26463,87,2523,0.003288,0.095341,61,1609,0.002305,0.060802,96,3297,64,1800,351.250000000,17511.050000000,261.050000000,10722.600000000,169.920000000,3084.160000000,66.270000000,953.150000000
2,WOW20005286,[BUNDABERG],24639,859,892,0.034863,0.036203,580,607,0.023540,0.024636,1049,1093,658,694,12369.150000000,13121.050000000,8084.900000000,8591.350000000,1946.980000000,2012.200000000,1003.180000000,1032.880000000
3,WOW20005378,"[CADBURY DAIRY MILK, CARAMILK, CONNOISSEUR, CRUNCHIE, DRUMSTICK (PETERS), FROSTY FRUITS, KIT KAT, MAXIBON, MILO, OREO, PETERS, PETERS ORIGINAL, PROUD & PUNCH, SKINNY COW, TWIN POLE]",522282,73519,151818,0.140765,0.290682,55931,109059,0.107090,0.208812,100739,250183,69986,148332,648195.610000000,1531413.180000000,490735.480000000,1043448.030000000,266270.420000000,712758.000000000,142811.100000000,337241.730000000
4,WOW20005378_Cadbury,[CADBURY DAIRY MILK],511072,1113,59892,0.002178,0.117189,283,37015,0.000554,0.072426,1392,87895,334,46675,7296.480000000,391544.510000000,2711.740000000,228641.400000000,3954.390000000,234349.800000000,621.340000000,97855.570000000
5,WOW20005378_Connoisseur,[CONNOISSEUR],511072,7,11452,0.000014,0.022408,3,8821,0.000006,0.017260,7,14298,3,10379,0E-9,109973.320000000,0E-9,83986.050000000,31.540000000,51429.940000000,15.920000000,29923.810000000
6,WOW20005378_Drumstick,[DRUMSTICK (PETERS)],511072,0,35527,0.000000,0.069515,0,31128,0.000000,0.060907,0,44547,0,37754,0E-9,250729.850000000,0E-9,215687.350000000,0E-9,94916.020000000,0E-9,65990.780000000
7,WOW20005378_FrostyFruits,[FROSTY FRUITS],511072,2875,7452,0.005625,0.014581,2319,5951,0.004538,0.011644,3499,9256,2701,7045,17555.200000000,51703.090000000,14056.400000000,40627.100000000,7933.560000000,21223.910000000,4627.930000000,11953.600000000
8,WOW20005378_Maxibon,[MAXIBON],511072,1879,4958,0.003677,0.009701,922,2473,0.001804,0.004839,2140,5834,1030,2780,10674.900000000,29942.540000000,6149.300000000,17465.940000000,5204.130000000,18062.660000000,1839.210000000,6605.940000000
9,WOW20005398,"[MIAPPLE, WOOLWORTHS, WOOLWORTHS SELECT]",33242,9574,26729,0.288009,0.804073,6661,22249,0.200379,0.669304,12948,55395,7386,29364,52290.250000000,1561328.030000000,32188.980000000,992724.710000000,7513.970000000,289799.910000000,2438.720000000,105681.780000000


In [227]:

check = client.query(
    """ 
    with sku_level AS (
        SELECT 
            booking_id,
            COUNT(DISTINCT sku.exposed_shopper_identification_number) AS n_exposed_sku_level,
            COUNT(DISTINCT sku.converted_shopper_identification_number) AS n_converted_sku_level
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_sku_level_v2` sku
        WHERE booking_id <> "WOW20007609_WOW20007607_WOW20007606_WOW20008226_WOW20007648_WOW20007655_WOW20007608_WOW20005378_WOW20007605_WOW20007611_WOW20007610_WOW20007647"
        GROUP BY 1
    ), 
    brand_level AS (
        SELECT 
            booking_id,
            COUNT(DISTINCT brand.exposed_shopper_identification_number) AS n_exposed_brand_level,
            COUNT(DISTINCT brand.converted_shopper_identification_number) AS n_converted_brand_level
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_sales_3_days_brand_level_v2` brand
        WHERE booking_id <> "WOW20007609_WOW20007607_WOW20007606_WOW20008226_WOW20007648_WOW20007655_WOW20007608_WOW20005378_WOW20007605_WOW20007611_WOW20007610_WOW20007647"
        GROUP BY 1
    )
    SELECT 
        sku_level.booking_id, 
        n_exposed_sku_level,
        n_exposed_brand_level, 
        n_exposed_brand_level - n_exposed_sku_level AS exposed_dif, 
        n_converted_sku_level,
        n_converted_brand_level,
        n_converted_brand_level - n_converted_sku_level AS converted_diff 
    FROM sku_level 
    LEFT JOIN brand_level 
        ON sku_level.booking_id = brand_level.booking_id
    """
).result()
check_df = check.to_dataframe()
chime.success()
check_df

,booking_id,n_exposed_sku_level,n_exposed_brand_level,exposed_dif,n_converted_sku_level,n_converted_brand_level,converted_diff
0,WOW20006683,4215,6558,2343,4215,6558,2343
1,WOW20004498,18797,20945,2148,18797,20945,2148
2,WOW20007654,1517,1546,29,1517,1546,29
3,WOW20008184,3761,7010,3249,3761,7010,3249
4,WOW20007647_GoldenGaytime,5777,6680,903,5777,6680,903
5,WOW20007647,37155,37164,9,37155,37164,9
6,WOW20006680,4572,77683,73111,4572,77683,73111
7,WOW20005378,55931,109059,53128,55931,109059,53128
8,WOW20006501,6828,15343,8515,6828,15343,8515
9,WOW20007737,8095,41621,33526,8095,41621,33526


In [33]:
check = client.query(
    """  
    with 
    online_order_table AS (
        SELECT
            oh.CustomerNumber AS shopper_id,
            cust.CustomerRegistrationNumber AS crn,
            CAST(oh.DisColDateTime AS DATE) AS ecf_order_date,
            od.CreateDateTime as ecf_order_time,
            OriginalOrderNumber as order_id,        
            ARRAY_AGG(CAST(od.StockCode as STRING)) as Article,
            ARRAY_AGG(od.OrderQuantity) as n_units_purchased,
            ARRAY_AGG(od.OrderLineAmount) as price_paid,
            ARRAY_AGG(od.OrderLineDiscountAmount) as discount_amount_from_rrp
        FROM `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_online_view_smkt.ecf_smkt_order_header_v` as oh
        INNER JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_online_view_smkt.ecf_smkt_order_detail_v` as od
            ON oh.OrderNumber = od.OrderNumber
        LEFT JOIN `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_customer_view.customer_online_shopper_v` cust
            ON cust.ShopperID = CAST(oh.CustomerNumber AS STRING)
        WHERE 
            CAST(oh.DisColDateTime as DATE) >= DATE("2024-01-01")
        AND 
            CAST(oh.DisColDateTime as DATE) <= (DATE("2024-01-01") + 6)
        GROUP BY 1,2,3,4,5
    ),
    instore_order_table AS (
    
        
    )


    """
).result()


In [496]:
check = client.query( 
    """
    CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level` AS  
    SELECT 
        final_pairs.booking_id, 

        final_pairs.exposure_crn, 
        total_exposed_spend_sku_level_7_day,
        total_exposed_spend_brand_level_7_day,

        n_exposed_baskets_sku_level,
        n_not_exposed_baskets_sku_level,
        n_exposed_baskets_brand_level,
        n_not_exposed_baskets_brand_level,
        
        final_pairs.candidate_comparison_shopper, 
        total_not_exposed_spend_sku_level_7_day,
        total_not_exposed_spend_brand_level_7_day,

        total_exposed_spend_sku_level_7_day - total_not_exposed_spend_sku_level_7_day AS total_spend_sku_level_7_day_uplift,
        total_exposed_spend_brand_level_7_day - total_not_exposed_spend_brand_level_7_day AS total_spend_brand_level_7_day_uplift

    FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history_test_vs_control_step_three` final_pairs 
    
    -- Exposed/Test Shopper - Sku Level Sales, 7 Days
    LEFT JOIN (
        SELECT 
            booking_id,
            converted_shopper_identification_number, 
            COUNT(DISTINCT basket_key) AS n_exposed_baskets_sku_level,
            SUM(tot_net_incld_gst) AS total_exposed_spend_sku_level_7_day
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2`
        GROUP BY 1,2
    )  exposed_campaign_conversions_sku_7_day
        ON final_pairs.booking_id = exposed_campaign_conversions_sku_7_day.booking_id 
        AND final_pairs.exposure_crn = exposed_campaign_conversions_sku_7_day.converted_shopper_identification_number 

    -- Not Exposed/Control Shopper - Sku Level Sales, 7 Days
    LEFT JOIN (
        SELECT 
            booking_id,
            converted_shopper_identification_number, 
            COUNT(DISTINCT basket_key) AS n_not_exposed_baskets_sku_level,
            SUM(tot_net_incld_gst) AS total_not_exposed_spend_sku_level_7_day
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2`
        GROUP BY 1,2
    )  not_exposed_campaign_conversions_sku_7_day
        ON final_pairs.booking_id = not_exposed_campaign_conversions_sku_7_day.booking_id 
        AND final_pairs.candidate_comparison_shopper = not_exposed_campaign_conversions_sku_7_day.converted_shopper_identification_number 

    -- Exposed/Test Shopper - Brand Level Sales, 7 Days
    LEFT JOIN (
        SELECT 
            booking_id,
            converted_shopper_identification_number, 
            COUNT(DISTINCT basket_key) AS n_exposed_baskets_brand_level,
            SUM(tot_net_incld_gst) AS total_exposed_spend_brand_level_7_day
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_brand_level_v2` 
        WHERE exposure_to_purchase_window = "7 Days" 
        GROUP BY 1,2
    )  exposed_campaign_conversions_brand_7_day
        ON final_pairs.booking_id = exposed_campaign_conversions_brand_7_day.booking_id 
        AND final_pairs.exposure_crn = exposed_campaign_conversions_brand_7_day.converted_shopper_identification_number 

    -- Not Exposed/Control Shopper - Brand Level Sales, 7 Days
    LEFT JOIN (
        SELECT 
            booking_id,
            converted_shopper_identification_number, 
            COUNT(DISTINCT basket_key) AS n_not_exposed_baskets_brand_level,
            SUM(tot_net_incld_gst) AS total_not_exposed_spend_brand_level_7_day
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_brand_level_v2` 
        WHERE exposure_to_purchase_window = "7 Days" 
        GROUP BY 1,2
    )  not_exposed_campaign_conversions_brand_7_day
        ON final_pairs.booking_id = not_exposed_campaign_conversions_brand_7_day.booking_id 
        AND final_pairs.candidate_comparison_shopper = not_exposed_campaign_conversions_brand_7_day.converted_shopper_identification_number  

    WHERE final_pairs.booking_id IN (
        SELECT DISTINCT campaign_id 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
        WHERE query_end_time >= "2024-06-30 00:26:11.539971"
        AND query_step = "14"
    );
    
    /*
    CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_campaign_average_level` AS  
        SELECT 
            booking_id, 
            AVG(total_spend_sku_level_3_day_uplift) avg_sku_level_uplift_3_day_interval,
            AVG(total_spend_sku_level_7_day_uplift) avg_sku_level_uplift_7_day_interval,
            AVG(total_spend_brand_level_3_day_uplift) avg_brand_level_uplift_3_day_interval,
            AVG(total_spend_brand_level_7_day_uplift) avg_brand_level_uplift_7_day_interval
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_uplift_test_vs_control_shopper_level`
        GROUP BY 1 
    ;
    */
    """
).result()
check_df = check.to_dataframe()
check_df

""


In [16]:
check = client.query( 
    """
    CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level_2` AS 
    with step_one AS (
    SELECT 
        *, 
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY exposure_crn) AS n_exposed_shopper_occurrence, 
        ROW_NUMBER() OVER(PARTITION BY booking_id, candidate_comparison_shopper ORDER BY candidate_comparison_shopper) AS n_not_exposed_shopper_occurrence
    FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level`  
    ORDER BY 5 DESC
    ) 
      
    SELECT * 
    FROM step_one 
    WHERE n_exposed_shopper_occurrence = 1 
    AND n_not_exposed_shopper_occurrence = 1
    """
).result()
check_df = check.to_dataframe()
check_df

""


In [502]:
check = client.query( 
    """
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results` AS 
    with step_one AS (
        SELECT 
            res.booking_id, 
            total_exposed.exposed_crns AS total_exposed_shoppers,
            downsampled_exposed_shoppers, 
            downsampled_converted_shoppers,
            SUM(n_exposed_baskets_sku_level) n_exposed_baskets_sku_level,
            SUM(n_not_exposed_baskets_sku_level) n_not_exposed_baskets_sku_level,
            SUM(n_exposed_baskets_brand_level) n_exposed_baskets_brand_level,
            SUM(n_not_exposed_baskets_brand_level) n_not_exposed_baskets_brand_level,
            COUNT(candidate_comparison_shopper) AS downsampled_control_shoppers,
            SUM(total_exposed_spend_sku_level_7_day) AS downsampled_exposed_sku_level_spend,
            SUM(total_not_exposed_spend_sku_level_7_day) AS downsampled_not_exposed_sku_level_spend,
            SUM(total_exposed_spend_brand_level_7_day) AS downsampled_exposed_brand_level_spend,
            SUM(total_not_exposed_spend_brand_level_7_day) AS downsampled_not_exposed_brand_level_spend,
            SUM(total_exposed_spend_sku_level_7_day) - SUM(total_not_exposed_spend_sku_level_7_day) AS total_downsampled_spend_sku_level_uplift,
            SUM(total_exposed_spend_brand_level_7_day) - SUM(total_not_exposed_spend_brand_level_7_day) AS total_downsampled_spend_brand_level_uplift,
            SAFE_DIVIDE(SUM(total_exposed_spend_sku_level_7_day) , SUM(total_not_exposed_spend_sku_level_7_day) ) - 1 AS percentage_uplift_sku_spend,
            ( SAFE_DIVIDE(SUM(total_exposed_spend_brand_level_7_day) , SUM(total_not_exposed_spend_brand_level_7_day)) ) - 1 AS percentage_uplift_brand_spend

        FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level` res 
        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT crn) AS exposed_crns 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control`
            WHERE cohort = "Exposed"
            GROUP BY 1 
        ) AS total_exposed ON total_exposed.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS exposed_crns_from_exposure_plus_history_dataset
            FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history`
            WHERE cohort = "Exposed"
            GROUP BY 1
        ) history ON history.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS downsampled_exposed_shoppers,
                COUNT(DISTINCT CASE WHEN converted = 1 THEN exposure_crn ELSE NULL END) AS downsampled_converted_shoppers
            FROM (
                SELECT full_set.*, downsampled.CENTROID_ID
                FROM  `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history` full_set 
                INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.clustered_data_sampled` downsampled 
                    ON full_set.booking_id = downsampled.booking_id 
                    AND full_set.exposure_crn = downsampled.exposure_crn 
                WHERE 
                downsampled.exposure_crn IS NOT NULL
                AND full_set.cohort = "Exposed"
            )
            GROUP BY 1
        ) AS downsampled_crns ON downsampled_crns.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS exposed_shoppers_from_features_with_clusters
            FROM `gcp-wow-cart-data-dev-d4d7.davide.shopper_features_with_clusters`
            WHERE cohort = "Exposed" 
            GROUP BY 1 
        ) AS clustered ON clustered.booking_id = res.booking_id

        WHERE res.booking_id IN (
            SELECT DISTINCT campaign_id 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
            WHERE query_end_time >= "2024-06-30 00:26:11.539971"
            AND query_step = "14"
        )
        GROUP BY ALL
    ), 
    step_two AS (
        SELECT 
            *, 
            total_downsampled_spend_sku_level_uplift / downsampled_exposed_shoppers AS exposed_shopper_avg_sku_level_uplift,
            total_downsampled_spend_brand_level_uplift / downsampled_exposed_shoppers AS exposed_shopper_avg_brand_level_uplift
        FROM step_one
    ) 
    
    SELECT 
        step_two.*,
        sku_level_converted_exposed_shoppers,
        brand_level_converted_exposed_shoppers,
        exposed_shopper_avg_sku_level_uplift * total_exposed_shoppers AS estimated_campaign_uplift_sku_level, 
        exposed_shopper_avg_brand_level_uplift * total_exposed_shoppers AS estimated_campaign_uplift_brand_level
    FROM step_two 
    LEFT JOIN (
        SELECT 
            booking_id, 
            COUNT(DISTINCT converted_shopper_identification_number) AS sku_level_converted_exposed_shoppers 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2`
        WHERE exposure_context = "Exposed"
        GROUP BY 1
    ) sku_convert_exposed ON sku_convert_exposed.booking_id = step_two.booking_id 


    LEFT JOIN (
        SELECT 
            booking_id, 
            COUNT(DISTINCT converted_shopper_identification_number) AS brand_level_converted_exposed_shoppers 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_brand_level_v2`
        WHERE exposure_context = "Exposed"
        GROUP BY 1
    ) brand_convert_exposed ON brand_convert_exposed.booking_id = step_two.booking_id 

    """
).result()
check_df = check.to_dataframe()
check_df

""


In [17]:
check = client.query( 
    """
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results_2` AS 
    with step_one AS (
        SELECT 
            res.booking_id, 
            total_exposed.exposed_crns AS total_exposed_shoppers,
            downsampled_exposed_shoppers, 
            downsampled_converted_shoppers,
            SUM(n_exposed_baskets_sku_level) n_exposed_baskets_sku_level,
            SUM(n_not_exposed_baskets_sku_level) n_not_exposed_baskets_sku_level,
            SUM(n_exposed_baskets_brand_level) n_exposed_baskets_brand_level,
            SUM(n_not_exposed_baskets_brand_level) n_not_exposed_baskets_brand_level,
            COUNT(candidate_comparison_shopper) AS downsampled_control_shoppers,
            SUM(total_exposed_spend_sku_level_7_day) AS downsampled_exposed_sku_level_spend,
            SUM(total_not_exposed_spend_sku_level_7_day) AS downsampled_not_exposed_sku_level_spend,
            SUM(total_exposed_spend_brand_level_7_day) AS downsampled_exposed_brand_level_spend,
            SUM(total_not_exposed_spend_brand_level_7_day) AS downsampled_not_exposed_brand_level_spend,
            SUM(total_exposed_spend_sku_level_7_day) - SUM(total_not_exposed_spend_sku_level_7_day) AS total_downsampled_spend_sku_level_uplift,
            SUM(total_exposed_spend_brand_level_7_day) - SUM(total_not_exposed_spend_brand_level_7_day) AS total_downsampled_spend_brand_level_uplift,
            SAFE_DIVIDE(SUM(total_exposed_spend_sku_level_7_day) , SUM(total_not_exposed_spend_sku_level_7_day) ) - 1 AS percentage_uplift_sku_spend,
            ( SAFE_DIVIDE(SUM(total_exposed_spend_brand_level_7_day) , SUM(total_not_exposed_spend_brand_level_7_day)) ) - 1 AS percentage_uplift_brand_spend

        FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level_2` res 
        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT crn) AS exposed_crns 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control`
            WHERE cohort = "Exposed"
            GROUP BY 1 
        ) AS total_exposed ON total_exposed.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS exposed_crns_from_exposure_plus_history_dataset
            FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history`
            WHERE cohort = "Exposed"
            GROUP BY 1
        ) history ON history.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS downsampled_exposed_shoppers,
                COUNT(DISTINCT CASE WHEN converted = 1 THEN exposure_crn ELSE NULL END) AS downsampled_converted_shoppers
            FROM (
                SELECT full_set.*, downsampled.CENTROID_ID
                FROM  `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_plus_history` full_set 
                INNER JOIN `gcp-wow-cart-data-dev-d4d7.davide.clustered_data_sampled` downsampled 
                    ON full_set.booking_id = downsampled.booking_id 
                    AND full_set.exposure_crn = downsampled.exposure_crn 
                WHERE 
                downsampled.exposure_crn IS NOT NULL
                AND full_set.cohort = "Exposed"
            )
            GROUP BY 1
        ) AS downsampled_crns ON downsampled_crns.booking_id = res.booking_id

        LEFT JOIN (
            SELECT 
                booking_id, 
                COUNT(DISTINCT exposure_crn) AS exposed_shoppers_from_features_with_clusters
            FROM `gcp-wow-cart-data-dev-d4d7.davide.shopper_features_with_clusters`
            WHERE cohort = "Exposed" 
            GROUP BY 1 
        ) AS clustered ON clustered.booking_id = res.booking_id

        WHERE res.booking_id IN (
            SELECT DISTINCT campaign_id 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
            WHERE query_end_time >= "2024-06-30 00:26:11.539971"
            AND query_step = "14"
        )
        GROUP BY ALL
    ), 
    step_two AS (
        SELECT 
            *, 
            total_downsampled_spend_sku_level_uplift / downsampled_exposed_shoppers AS exposed_shopper_avg_sku_level_uplift,
            total_downsampled_spend_brand_level_uplift / downsampled_exposed_shoppers AS exposed_shopper_avg_brand_level_uplift
        FROM step_one
    ) 
    
    SELECT 
        step_two.*,
        sku_level_converted_exposed_shoppers,
        brand_level_converted_exposed_shoppers,
        exposed_shopper_avg_sku_level_uplift * total_exposed_shoppers AS estimated_campaign_uplift_sku_level, 
        exposed_shopper_avg_brand_level_uplift * total_exposed_shoppers AS estimated_campaign_uplift_brand_level
    FROM step_two 
    LEFT JOIN (
        SELECT 
            booking_id, 
            COUNT(DISTINCT converted_shopper_identification_number) AS sku_level_converted_exposed_shoppers 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2`
        WHERE exposure_context = "Exposed"
        GROUP BY 1
    ) sku_convert_exposed ON sku_convert_exposed.booking_id = step_two.booking_id 


    LEFT JOIN (
        SELECT 
            booking_id, 
            COUNT(DISTINCT converted_shopper_identification_number) AS brand_level_converted_exposed_shoppers 
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_brand_level_v2`
        WHERE exposure_context = "Exposed"
        GROUP BY 1
    ) brand_convert_exposed ON brand_convert_exposed.booking_id = step_two.booking_id 

    """
).result()
check_df = check.to_dataframe()
check_df

""


In [15]:
check = client.query( 
    """
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results_2` LIMIT 10
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,total_exposed_shoppers,downsampled_exposed_shoppers,downsampled_converted_shoppers,n_exposed_baskets_sku_level,n_not_exposed_baskets_sku_level,n_exposed_baskets_brand_level,n_not_exposed_baskets_brand_level,downsampled_control_shoppers,downsampled_exposed_sku_level_spend,downsampled_not_exposed_sku_level_spend,downsampled_exposed_brand_level_spend,downsampled_not_exposed_brand_level_spend,total_downsampled_spend_sku_level_uplift,total_downsampled_spend_brand_level_uplift,percentage_uplift_sku_spend,percentage_uplift_brand_spend,exposed_shopper_avg_sku_level_uplift,exposed_shopper_avg_brand_level_uplift,sku_level_converted_exposed_shoppers,brand_level_converted_exposed_shoppers,estimated_campaign_uplift_sku_level,estimated_campaign_uplift_brand_level
0,WOW20000952,74130,3465,629,713,555,718,561,1628,22967.660000000,16151.540000000,23172.790000000,16322.860000000,6816.120000000,6849.930000000,0.422010533,0.419652561,1.967134199,1.976891775,872,880,145823.658171870,146546.987280750


In [ ]:
check = client.query( 
    """
    CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types` AS 
    SELECT 
        results.booking_id,
        ARRAY_AGG(DISTINCT brand IGNORE NULLS ORDER BY brand) AS brands,
        ARRAY_AGG(DISTINCT CategoryDescription IGNORE NULLS) AS categories,
        ARRAY_TO_STRING(media_types, ", ") AS media_types,
        COUNT(DISTINCT individual_campaign_product_string) AS n_skus, 
        DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS n_campaign_weeks,
        total_exposed_shoppers,  
        downsampled_exposed_shoppers,
        downsampled_converted_shoppers,
        n_exposed_baskets_sku_level,
        n_not_exposed_baskets_sku_level,
        n_exposed_baskets_brand_level,
        n_not_exposed_baskets_brand_level,
        ROUND(downsampled_exposed_sku_level_spend, 2) AS downsampled_exposed_sku_level_spend,
        ROUND(downsampled_not_exposed_sku_level_spend, 2) AS downsampled_not_exposed_sku_level_spend,
        ROUND(downsampled_not_exposed_sku_level_spend / downsampled_converted_shoppers, 2) AS sku_spend_per_convert,
        ROUND(downsampled_exposed_brand_level_spend, 2) AS downsampled_exposed_brand_level_spend,
        ROUND(downsampled_not_exposed_brand_level_spend, 2) AS downsampled_not_exposed_brand_level_spend,
        ROUND(downsampled_exposed_brand_level_spend / downsampled_converted_shoppers, 2) AS brand_spend_per_convert,
        ROUND(total_downsampled_spend_sku_level_uplift, 2) AS total_downsampled_spend_sku_level_uplift,
        ROUND(total_downsampled_spend_brand_level_uplift, 2) AS total_downsampled_spend_brand_level_uplift
    FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results` results
    LEFT JOIN (
        SELECT DISTINCT 
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            skus
        FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control 
    ) campaign_skus ON results.booking_id = campaign_skus.booking_id,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string
    LEFT JOIN (
        SELECT DISTINCT
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            CategoryDescription,
            Brand AS brand
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND Brand NOT IN ("OTHER", "UNBRANDED", "")
    ) brand_map
        ON brand_map.Article = individual_campaign_product_string
    LEFT JOIN (
        SELECT  
            booking_id, 
            ARRAY_AGG(DISTINCT media_type IGNORE NULLS) AS media_types
        FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v3`
        GROUP BY 1
    ) cc 
        ON cc.booking_id = results.booking_id 
    WHERE results.booking_id IN (
            SELECT DISTINCT campaign_id 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
            WHERE query_end_time >= "2024-06-30 00:26:11.539971"
            AND query_step = "14"
        )
    AND n_exposed_baskets_brand_level > 2000
    AND n_not_exposed_baskets_brand_level > 2000
    GROUP BY ALL
    ORDER BY total_downsampled_spend_brand_level_uplift DESC;
    """
).result()
check_df = check.to_dataframe()
check_df

In [24]:
check = client.query( 
    """
    CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types_2` AS 
    SELECT 
        results.booking_id,
        ARRAY_AGG(DISTINCT brand IGNORE NULLS ORDER BY brand) AS brands,
        ARRAY_AGG(DISTINCT CategoryDescription IGNORE NULLS) AS categories,
        ARRAY_TO_STRING(media_types, ", ") AS media_types,
        COUNT(DISTINCT individual_campaign_product_string) AS n_skus, 
        DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS n_campaign_weeks,
        total_exposed_shoppers,  
        downsampled_exposed_shoppers,
        downsampled_converted_shoppers,
        n_exposed_baskets_sku_level,
        n_not_exposed_baskets_sku_level,
        n_exposed_baskets_brand_level,
        n_not_exposed_baskets_brand_level,
        ROUND(downsampled_exposed_sku_level_spend, 2) AS downsampled_exposed_sku_level_spend,
        ROUND(downsampled_not_exposed_sku_level_spend, 2) AS downsampled_not_exposed_sku_level_spend,
        ROUND(downsampled_not_exposed_sku_level_spend / downsampled_converted_shoppers, 2) AS sku_spend_per_convert,
        ROUND(downsampled_exposed_brand_level_spend, 2) AS downsampled_exposed_brand_level_spend,
        ROUND(downsampled_not_exposed_brand_level_spend, 2) AS downsampled_not_exposed_brand_level_spend,
        ROUND(downsampled_exposed_brand_level_spend / downsampled_converted_shoppers, 2) AS brand_spend_per_convert,
        ROUND(total_downsampled_spend_sku_level_uplift, 2) AS total_downsampled_spend_sku_level_uplift,
        ROUND(total_downsampled_spend_brand_level_uplift, 2) AS total_downsampled_spend_brand_level_uplift
    FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results_2` results
    LEFT JOIN (
        SELECT DISTINCT 
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            skus
        FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control 
    ) campaign_skus ON results.booking_id = campaign_skus.booking_id,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string
    LEFT JOIN (
        SELECT DISTINCT
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            CategoryDescription,
            Brand AS brand
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND Brand NOT IN ("OTHER", "UNBRANDED", "")
    ) brand_map
        ON brand_map.Article = individual_campaign_product_string
    LEFT JOIN (
        SELECT  
            booking_id, 
            ARRAY_AGG(DISTINCT media_type IGNORE NULLS) AS media_types
        FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v3`
        GROUP BY 1
    ) cc 
        ON cc.booking_id = results.booking_id 
    WHERE results.booking_id IN (
            SELECT DISTINCT campaign_id 
            FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
            WHERE query_end_time >= "2024-06-30 00:26:11.539971"
            AND query_step = "14"
        )
    AND n_exposed_baskets_brand_level >= 200
    AND n_not_exposed_baskets_brand_level >= 200
    GROUP BY ALL
    ORDER BY total_downsampled_spend_brand_level_uplift DESC;
    """
).result()
check_df = check.to_dataframe()
check_df

""


In [25]:
check = client.query( 
    """
 SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types_2` 
    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Media Type Uplift.csv")
check_df

,booking_id,brands,categories,media_types,n_skus,n_campaign_weeks,total_exposed_shoppers,downsampled_exposed_shoppers,downsampled_converted_shoppers,n_exposed_baskets_sku_level,n_not_exposed_baskets_sku_level,n_exposed_baskets_brand_level,n_not_exposed_baskets_brand_level,downsampled_exposed_sku_level_spend,downsampled_not_exposed_sku_level_spend,sku_spend_per_convert,downsampled_exposed_brand_level_spend,downsampled_not_exposed_brand_level_spend,brand_spend_per_convert,total_downsampled_spend_sku_level_uplift,total_downsampled_spend_brand_level_uplift
0,WOW20004382,"[COCA COLA, COCA COLA LIFE, COCA COLA NO SUGAR, COCA COLA VANILLA, DIET COCA COLA, FANTA, KIRKS, MOUNT FRANKLIN, POWERADE, SPRITE]","[ETHNIC / GOURMET FOOD, LIFESTYLE/WATER NON CARBONATED, DIRECT MARKETING, CARBONATED SOFT DRINKS]","Catalogue, Display Search",122,8,517884,4509,3155,9852,8077,9865,8084,889366.990000000,750695.270000000,237.940000000,890450.350000000,751237.930000000,282.230000000,138671.720000000,139212.420000000
1,WOW20004985,"[MR CHEN'S, WOOLWORTHS]","[MEAT, FROZEN MEALS]","Home Page, Display Search, Display Aisle and Category",28,4,292384,5546,5184,1562,1246,38873,36859,21433.150000000,15792.420000000,3.050000000,844791.930000000,727493.970000000,162.960000000,5640.730000000,117297.960000000
2,WOW20004498,"[AGRUM, BUBLY, COOL RIDGE, COTTEE'S, DIET PEPSI, FRANTELLE, GATORADE, LIPTON, MOUNTAIN DEW, PASSIONA, PEPSI, PEPSI MAX, POP TOPS, SCHWEPPES, SOLO, SUNKIST, T2, VIBE]","[LIFESTYLE/WATER NON CARBONATED, CARBONATED SOFT DRINKS, LONGLIFE JUICE / DRINKS, CORDIAL / DRINK BASES]","Display Search, Results Page Display",144,7,82119,13356,7620,3005,2415,3216,2564,171256.620000000,114876.110000000,15.080000000,173654.230000000,116535.240000000,22.790000000,56380.510000000,57118.990000000
3,WOW20004139,"[BUONISSIMO, FREE FROM GLUTEN, OTWAY, THE NATURAL FOOD GROUP, WOOLWORTHS, WOOLWORTHS SELECT]","[PROPRIETARY BAKERY, INSTORE BAKERY, MEAT]","Display Customer Segment, Display Search, Home Page, Display Aisle and Category",110,4,313176,3273,2903,442,321,4601,3296,5045.390000000,2212.970000000,0.760000000,129819.650000000,75024.850000000,44.720000000,2832.420000000,54794.800000000
4,WOW20004004,[WHISKAS],[PET FOOD],"Display Search, Display Aisle and Category, Display Customer Segment, Home Page",8,13,301107,17090,11024,4373,4170,11031,10464,90878.270000000,80383.170000000,7.290000000,234312.650000000,201334.950000000,21.250000000,10495.100000000,32977.700000000
5,WOW20004608,"[CADBURY, CADBURY DAIRY MILK, CADBURY ROSES, CARAMILK, CHERRY RIPE, CHOMP, CRUNCHIE, CURLY WURLY, FAVOURITES, FLAKE, MORO, PICNIC, SOUR PATCH, TNCC, TOBLERONE, TURKISH DELIGHT, TWIRL]",[CONFECTIONERY],"Display Aisle and Category, Display Reach, Home Page, Display Search, Catalogue",78,4,655346,9897,4714,1866,1302,2578,1820,47582.430000000,25133.730000000,5.330000000,63106.110000000,34327.370000000,13.390000000,22448.700000000,28778.740000000
6,WOW20004733,[OMO],[CLEANSING],"Display Search, Display Customer Segment, Catalogue, Display Aisle and Category",26,1,266709,17206,3921,2097,1439,2097,1439,49304.830000000,31700.770000000,8.080000000,49304.830000000,31700.770000000,12.570000000,17604.060000000,17604.060000000
7,WOW20004147,[RED ROCK DELI],[BISCUITS],"Home Page, Display Search",13,5,293172,18627,7821,516,391,6181,5782,5121.050000000,2302.180000000,0.290000000,83436.090000000,66615.160000000,10.670000000,2818.870000000,16820.930000000
8,WOW20004972,[WESTERN STAR],[DAIRY - BUTTER & MARGARINE],"Display Recipes, Display Search",9,8,31437,14858,8676,1903,1535,2079,1599,26681.300000000,14459.950000000,1.670000000,28108.330000000,15015.430000000,3.240000000,12221.350000000,13092.900000000
9,WOW20004819,[APTAMIL],[BABY NEEDS],"Display Aisle and Category, Display Customer Segment",14,5,17027,1401,445,812,517,910,566,29736.330000000,19820.090000000,44.540000000,35363.410000000,23096.040000000,79.470000000,9916.240000000,12267.370000000


In [36]:
check = client.query( 
    """
    CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.dx_media_level_uplift` AS
    with uplift_1 AS (
        SELECT 
            booking_id, 
            media_types, 
            downsampled_exposed_sku_level_spend, 
            downsampled_not_exposed_sku_level_spend,
            downsampled_exposed_sku_level_spend - downsampled_not_exposed_sku_level_spend AS downsampled_sku_level_uplift,
            (downsampled_exposed_sku_level_spend / downsampled_not_exposed_sku_level_spend) - 1 AS downsampled_sku_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    ),
    uplift_2 AS (
        SELECT 
            booking_id, 
            media_types, 
            downsampled_exposed_sku_level_spend, 
            downsampled_not_exposed_sku_level_spend,
            downsampled_exposed_sku_level_spend - downsampled_not_exposed_sku_level_spend AS downsampled_sku_level_uplift,
            (downsampled_exposed_sku_level_spend / downsampled_not_exposed_sku_level_spend) - 1 AS downsampled_sku_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types_2`
    ),
    step_one AS (
        SELECT 
            media_type, 
            COUNT(DISTINCT uplift_1.booking_id) AS n_campaigns_1,
            AVG(uplift_1.downsampled_sku_level_uplift_perc) AS avg_downsampled_sku_level_uplift_perc_1
        FROM uplift_1, 
        UNNEST(SPLIT(media_types, ", ")) AS media_type
        WHERE uplift_1.downsampled_sku_level_uplift_perc IS NOT NULL
        GROUP BY 1
    ),
    step_two AS (
        SELECT
            media_type,
            COUNT(DISTINCT uplift_2.booking_id) AS n_campaigns_2,
            AVG(uplift_2.downsampled_sku_level_uplift_perc) AS avg_downsampled_sku_level_uplift_perc_2
        FROM uplift_2,
        UNNEST(SPLIT(media_types, ", ")) AS media_type
        WHERE uplift_2.downsampled_sku_level_uplift_perc IS NOT NULL
        GROUP BY 1
    )
    SELECT 
        step_one.media_type, 
        n_campaigns_1,
        n_campaigns_2,
        avg_downsampled_sku_level_uplift_perc_1,
        avg_downsampled_sku_level_uplift_perc_2
    FROM step_one 
    LEFT JOIN step_two ON step_one.media_type = step_two.media_type

    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Media Type Uplift.csv")
check_df

""


In [511]:
check = client.query( 
    """
    with uplift AS (
        SELECT 
            booking_id, 
            media_types, 
            downsampled_exposed_brand_level_spend, 
            downsampled_not_exposed_brand_level_spend,
            downsampled_exposed_brand_level_spend - downsampled_not_exposed_brand_level_spend AS downsampled_brand_level_uplift,
            (downsampled_exposed_brand_level_spend / downsampled_not_exposed_brand_level_spend) - 1 AS downsampled_brand_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    )
    
    SELECT 
        media_type, 
        COUNT(DISTINCT booking_id) AS n_campaigns,
        AVG(downsampled_brand_level_uplift_perc) AS avg_downsampled_brand_level_uplift_perc
    FROM uplift, 
    UNNEST(SPLIT(media_types, ", ")) AS media_type
    WHERE downsampled_brand_level_uplift_perc IS NOT NULL
    GROUP BY 1
    
    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Media Type Uplift.csv")
check_df

,media_type,n_campaigns,avg_downsampled_brand_level_uplift_perc
0,Display Search,29,0.385866554
1,Results Page Display,4,0.406035586
2,Display Customer Segment,8,0.401220447
3,Home Page,15,0.353501466
4,Display Aisle and Category,8,0.410986143
5,Catalogue,10,0.321717277
6,Display Reach,6,0.456253185
7,Display Recipes,2,0.492671259
8,Display Specials,5,0.437437070


In [512]:
check = client.query( 
    """
    with uplift AS (
        SELECT 
            booking_id, 
            categories, 
            downsampled_exposed_sku_level_spend, 
            downsampled_not_exposed_sku_level_spend,
            downsampled_exposed_sku_level_spend - downsampled_not_exposed_sku_level_spend AS downsampled_sku_level_uplift,
            (downsampled_exposed_sku_level_spend / downsampled_not_exposed_sku_level_spend) - 1 AS downsampled_sku_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    )
    SELECT 
        category, 
        COUNT(DISTINCT CASE WHEN downsampled_sku_level_uplift_perc IS NOT NULL THEN booking_id ELSE NULL END) AS n_campaigns,
        AVG(downsampled_sku_level_uplift_perc) AS avg_downsampled_sku_level_uplift_perc
    FROM uplift, 
    UNNEST(categories) AS category
    WHERE downsampled_sku_level_uplift_perc IS NOT NULL
    GROUP BY 1

    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Category Uplift.csv")
check_df

,category,n_campaigns,avg_downsampled_sku_level_uplift_perc
0,CARBONATED SOFT DRINKS,3,0.356300800
1,CORDIAL / DRINK BASES,2,0.448083621
2,LONGLIFE JUICE / DRINKS,1,0.782408246
3,LIFESTYLE/WATER NON CARBONATED,3,0.356300800
4,MEAT,2,0.944765418
5,PROPRIETARY BAKERY,1,1.534078461
6,INSTORE BAKERY,1,1.534078461
7,DIRECT MARKETING,1,0.172735158
8,ETHNIC / GOURMET FOOD,3,0.227771993
9,FROZEN MEALS,1,0.355452374


In [513]:
check = client.query( 
    """
    with uplift AS (
        SELECT 
            booking_id, 
            categories, 
            downsampled_exposed_brand_level_spend, 
            downsampled_not_exposed_brand_level_spend,
            downsampled_exposed_brand_level_spend - downsampled_not_exposed_brand_level_spend AS downsampled_brand_level_uplift,
            (downsampled_exposed_brand_level_spend / downsampled_not_exposed_brand_level_spend) - 1 AS downsampled_brand_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    )
    SELECT 
        category, 
        COUNT(DISTINCT CASE WHEN downsampled_brand_level_uplift_perc IS NOT NULL THEN booking_id ELSE NULL END) AS n_campaigns,
        AVG(downsampled_brand_level_uplift_perc) AS avg_downsampled_brand_level_uplift_perc
    FROM uplift, 
    UNNEST(categories) AS category
    WHERE downsampled_brand_level_uplift_perc IS NOT NULL
    GROUP BY 1

    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Category Uplift.csv")
check_df

,category,n_campaigns,avg_downsampled_brand_level_uplift_perc
0,CARBONATED SOFT DRINKS,3,0.334802036
1,CORDIAL / DRINK BASES,2,0.415457844
2,LONGLIFE JUICE / DRINKS,1,0.708665035
3,LIFESTYLE/WATER NON CARBONATED,3,0.334802036
4,MEAT,2,0.494373217
5,PROPRIETARY BAKERY,1,0.819814837
6,INSTORE BAKERY,1,0.819814837
7,DIRECT MARKETING,1,0.173490421
8,ETHNIC / GOURMET FOOD,3,0.187275311
9,FROZEN MEALS,1,0.168931596


In [514]:
check = client.query( 
    """
    with uplift AS (
        SELECT 
            booking_id, 
            categories, 
            downsampled_exposed_brand_level_spend, 
            downsampled_not_exposed_brand_level_spend,
            downsampled_exposed_brand_level_spend - downsampled_not_exposed_brand_level_spend AS downsampled_brand_level_uplift,
            (downsampled_exposed_brand_level_spend / downsampled_not_exposed_brand_level_spend) - 1 AS downsampled_brand_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    ) 
    
    SELECT  
        COUNT(DISTINCT booking_id) AS n_campaigns,
        COUNT(DISTINCT category) AS n_categories,
        AVG(downsampled_brand_level_uplift_perc) AS avg_downsampled_brand_level_uplift_perc
    FROM uplift,
    UNNEST(categories) AS category
    WHERE downsampled_brand_level_uplift_perc IS NOT NULL
    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Total Uplift.csv")
check_df

,n_campaigns,n_categories,avg_downsampled_brand_level_uplift_perc
0,45,29,0.386600886


In [ ]:
check = client.query( 
    """
    with uplift AS (
        SELECT 
            booking_id, 
            categories, 
            downsampled_exposed_sku_level_spend, 
            downsampled_not_exposed_sku_level_spend,
            downsampled_exposed_sku_level_spend - downsampled_not_exposed_sku_level_spend AS downsampled_sku_level_uplift,
            (downsampled_exposed_sku_level_spend / downsampled_not_exposed_sku_level_spend) - 1 AS downsampled_sku_level_uplift_perc
        FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_results_plus_brands_and_media_types`
    ) 
    
    SELECT  
        COUNT(DISTINCT booking_id) AS n_campaigns,
        AVG(downsampled_sku_level_uplift_perc) AS avg_downsampled_sku_level_uplift_perc
    FROM uplift
    WHERE downsampled_sku_level_uplift_perc IS NOT NULL
    """
).result()
check_df = check.to_dataframe()
check_df.to_csv("~/Downloads/DX Total Uplift.csv")
check_df

,n_campaigns,avg_downsampled_sku_level_uplift_perc
0,105,0.740740898


In [515]:
check = client.query( 
    """
SELECT * 
FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` 
WHERE query_end_time >= "2024-06-30 00:26:11.539971"
AND query_step = "14"
--AND campaign_id = "WOW20000952"
ORDER BY query_end_time DESC 
LIMIT 200;
    """
).result()
check_df = check.to_dataframe()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20004985,14,Total Campaign Analysis Runtime,2024-07-01 11:40:22.220776,2024-07-01 11:40:36.690483,898,14.966667
1,WOW20004978,14,Total Campaign Analysis Runtime,2024-07-01 11:25:18.454724,2024-07-01 11:25:33.766346,918,15.300000
2,WOW20004972,14,Total Campaign Analysis Runtime,2024-07-01 11:09:57.432876,2024-07-01 11:10:11.444087,819,13.650000
3,WOW20004969,14,Total Campaign Analysis Runtime,2024-07-01 10:56:17.437019,2024-07-01 10:56:29.421130,697,11.616667
4,WOW20004968,14,Total Campaign Analysis Runtime,2024-07-01 10:44:34.950907,2024-07-01 10:44:48.631978,771,12.850000
5,WOW20004967,14,Total Campaign Analysis Runtime,2024-07-01 10:31:41.473840,2024-07-01 10:31:53.708807,634,10.566667
6,WOW20004966,14,Total Campaign Analysis Runtime,2024-07-01 10:21:00.134817,2024-07-01 10:21:15.127567,788,13.133333
7,WOW20004958,14,Total Campaign Analysis Runtime,2024-07-01 10:07:50.645985,2024-07-01 10:08:04.090718,855,14.250000
8,WOW20004952,14,Total Campaign Analysis Runtime,2024-07-01 09:48:31.596955,2024-07-01 09:48:43.772735,754,12.566667
9,WOW20004948,14,Total Campaign Analysis Runtime,2024-07-01 09:35:48.652503,2024-07-01 09:36:06.334076,807,13.450000


In [353]:
check = client.query( 
    """
SELECT * 
--FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level` 
FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2`
WHERE booking_id = "WOW20004382" 
AND converted_shopper_identification_number IS NOT NULL
ORDER BY converted_shopper_identification_number, basket_key, article
LIMIT 50;
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,order_context,exposed_shopper_identification_number,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window
0,WOW20004382,Instore Order,1000000000000000052,1000000000000000052,2023-11-22 11:05:47+00:00,20231122110547062036911313,42605,20.900000000,7 Days
1,WOW20004382,Instore Order,1000000000000000076,1000000000000000076,2023-12-07 14:20:07+00:00,20231207142007001039321313,140090,2.550000000,7 Days
2,WOW20004382,Instore Order,1000000000000000076,1000000000000000076,2023-12-07 14:20:07+00:00,20231207142007001039321313,159797,5.220000000,7 Days
3,WOW20004382,Instore Order,1000000000000000076,1000000000000000076,2023-12-07 14:20:07+00:00,20231207142007001039321313,159798,5.220000000,7 Days
4,WOW20004382,Instore Order,1000000000000000076,1000000000000000076,2023-12-07 14:20:07+00:00,20231207142007001039321313,313019,7.630000000,7 Days
5,WOW20004382,Instore Order,1000000000000000076,1000000000000000076,2023-12-15 16:39:25+00:00,20231215163925005060111313,140090,1.330000000,7 Days
6,WOW20004382,Online Order,1000000000000000084,1000000000000000084,2023-12-22 03:19:49.417000+00:00,185223958,32731,7.080000000,7 Days
7,WOW20004382,Online Order,1000000000000000084,1000000000000000084,2023-12-22 03:19:49.417000+00:00,185223958,52558,3.000000000,7 Days
8,WOW20004382,Online Order,1000000000000000084,1000000000000000084,2023-12-22 03:19:49.417000+00:00,185223958,84552,26.900000000,7 Days
9,WOW20004382,Instore Order,1000000000000000109,1000000000000000109,2023-12-12 15:22:33+00:00,20231212152233001058351120,178416,6.470000000,7 Days


In [347]:
1103761.200000000 / 2224

496.29550359712226

In [231]:
check = client.query( 
    """
    with avg_query_time AS (
        SELECT AVG(query_duration_in_minutes) AS avg_duration  
        FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs` results
        WHERE query_step = "14" 
    ),
    campaigns_left_to_run AS (
        SELECT COUNT(DISTINCT booking_number) remaining_campaigns 
        FROM gcp-wow-food-wlx-digaspt-dev.playground.carto_campaigns
        WHERE campaign_start_date >= DATE("2023-10-01") 
        AND booking_number NOT IN (SELECT DISTINCT booking_id FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control)
        AND booking_number NOT IN (SELECT DISTINCT campaign_id FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs`) 
    )
    SELECT remaining_campaigns, avg_duration, (avg_duration * remaining_campaigns) / 60 AS estimated_time_left 
    FROM avg_query_time CROSS JOIN campaigns_left_to_run
    """
).result()
check_df = check.to_dataframe()
check_df

,remaining_campaigns,avg_duration,estimated_time_left
0,835,13.932895,193.899452


In [257]:
6000 * 6000

36000000

In [384]:
check = client.query( 
    """
    with step_one AS (
    SELECT 
        results.booking_id, 
        ARRAY_AGG(DISTINCT brand IGNORE NULLS) AS brands,
        COUNT(DISTINCT individual_campaign_product_string) AS n_skus, 
        campaign_start_date,
        campaign_end_date,
        results.* EXCEPT(booking_id)
    FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results` results
    LEFT JOIN (
        SELECT DISTINCT 
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            skus
        FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control 
    ) campaign_skus ON results.booking_id = campaign_skus.booking_id,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string
    LEFT JOIN (
        SELECT DISTINCT
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            Brand AS brand
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND Brand NOT IN ("OTHER", "UNBRANDED", "")
    ) brand_map
        ON brand_map.Article = individual_campaign_product_string

    WHERE results.booking_id IN (SELECT DISTINCT campaign_id FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs`)
    GROUP BY ALL
    ORDER BY estimated_campaign_uplift_brand_level DESC
)
SELECT 
    booking_id, 
    brands, 
    n_skus, 
    campaign_start_date, 
    campaign_end_date, 
    DATE_DIFF(campaign_end_date, campaign_start_date, DAY) AS campaign_days, 
    DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS campaign_weeks, 
    total_exposed_shoppers, 
    sku_level_converted_exposed_shoppers,
    CONCAT(ROUND((sku_level_converted_exposed_shoppers / total_exposed_shoppers) * 100, 2), "%") AS sku_level_ctr,
    brand_level_converted_exposed_shoppers,
    CONCAT(ROUND((brand_level_converted_exposed_shoppers / total_exposed_shoppers) * 100, 2), "%")  AS brand_level_ctr,
    downsampled_exposed_shoppers, 
    downsampled_converted_shoppers,
    CAST(downsampled_exposed_sku_level_spend AS INT64) downsampled_exposed_sku_level_spend, 
    CAST(downsampled_exposed_brand_level_spend AS INT64) downsampled_exposed_brand_level_spend, 
    downsampled_exposed_sku_level_spend / downsampled_converted_shoppers AS sku_spend_per_converted_shopper,
    downsampled_exposed_brand_level_spend / downsampled_converted_shoppers AS brand_spend_per_converted_shopper,
    
    -- Per converted shopper per week
    (downsampled_exposed_sku_level_spend / downsampled_converted_shoppers) / DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS sku_spend_per_converted_shopper_per_week,
    (downsampled_exposed_brand_level_spend / downsampled_converted_shoppers) / DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS brand_spend_per_converted_shopper_per_week,
    CAST(total_downsampled_spend_sku_level_uplift AS INT64) total_downsampled_spend_sku_level_uplift, 
    CAST(total_downsampled_spend_brand_level_uplift AS INT64) total_downsampled_spend_brand_level_uplift, 
    CAST(estimated_campaign_uplift_sku_level AS INT64) estimated_campaign_uplift_sku_level, 
    CAST(estimated_campaign_uplift_brand_level AS INT64) estimated_campaign_uplift_brand_level
FROM step_one 
ORDER BY estimated_campaign_uplift_brand_level DESC

    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,brands,n_skus,campaign_start_date,campaign_end_date,campaign_days,campaign_weeks,total_exposed_shoppers,sku_level_converted_exposed_shoppers,sku_level_ctr,brand_level_converted_exposed_shoppers,brand_level_ctr,downsampled_exposed_shoppers,downsampled_converted_shoppers,downsampled_exposed_sku_level_spend,downsampled_exposed_brand_level_spend,sku_spend_per_converted_shopper,brand_spend_per_converted_shopper,sku_spend_per_converted_shopper_per_week,brand_spend_per_converted_shopper_per_week,total_downsampled_spend_sku_level_uplift,total_downsampled_spend_brand_level_uplift,estimated_campaign_uplift_sku_level,estimated_campaign_uplift_brand_level
0,WOW20004139,"[WOOLWORTHS, BUONISSIMO, FREE FROM GLUTEN, WOOLWORTHS SELECT, OTWAY, THE NATURAL FOOD GROUP]",110,2023-10-04,2023-10-31,27,4,313176,72821,23.25%,256156,81.79%,3273,2903,20566,16884576,7.084447124,5816.250885291,1.771111781,1454.062721323,12450,7606393,1191305,727815344
1,WOW20004382,"[MOUNT FRANKLIN, POWERADE, COCA COLA, DIET COCA COLA, COCA COLA VANILLA, COCA COLA LIFE, COCA COLA NO SUGAR, KIRKS, FANTA, SPRITE]",122,2023-11-01,2023-12-26,55,8,517884,281970,54.45%,282509,54.55%,4509,3155,1427976,20019245,452.607324881,6345.243961965,56.575915610,793.155495246,210330,2959672,24157612,339935012
2,WOW20004004,[WHISKAS],8,2023-11-08,2024-02-06,90,13,301107,7873,2.61%,18167,6.03%,17090,11024,300479,32839465,27.256845972,2978.906483128,2.096680459,229.146652548,70305,6066031,1238694,106876794
3,WOW20003791,[MCCAIN],6,2023-10-04,2023-10-31,27,4,294334,10707,3.64%,73156,24.85%,2510,1131,12510,1625268,11.061131742,1437.018143236,2.765282936,359.254535809,5224,486909,612613,57097175
4,WOW20004147,[RED ROCK DELI],13,2023-10-04,2023-11-07,34,5,293172,6061,2.07%,52027,17.75%,18627,7821,22071,5950095,2.822017645,760.784472574,0.564403529,152.156894515,12987,1078428,204401,16973479
5,WOW20000952,[CAMPOS],17,2023-10-04,2023-10-31,27,4,74130,872,1.18%,880,1.19%,3465,630,30568,2011189,48.521063492,3192.363571429,12.130265873,798.090892857,11180,738453,239181,15798411
6,WOW20003728,"[NUTRI-GRAIN, JUST RIGHT, SULTANA BRAN, SPECIAL K]",9,2023-10-04,2023-10-17,13,2,276741,8707,3.15%,25090,9.07%,11324,3827,26694,2820929,6.975181604,737.112296838,3.487590802,368.556148419,1757,417325,42930,10198788
7,WOW20004033,[LYNX],15,2023-11-01,2023-11-28,27,4,293659,7503,2.56%,10659,3.63%,10880,3130,25540,1647854,8.159853035,526.470939297,2.039963259,131.617734824,2557,275299,69024,7430508
8,WOW20004498,"[T2, SOLO, LIPTON, MOUNTAIN DEW, FRANTELLE, BUBLY, PEPSI, SCHWEPPES, GATORADE, VIBE, PEPSI MAX, SUNKIST, COOL RIDGE, AGRUM, COTTEE'S, POP TOPS, PASSIONA, DIET PEPSI]",144,2024-01-03,2024-02-20,48,7,82119,38534,46.92%,43080,52.46%,13356,7620,735283,2267064,96.493839895,297.514948819,13.784834271,42.502135546,322761,940260,1984486,5781161
9,WOW20004392,[SENSODYNE],10,2023-11-15,2023-11-21,6,1,297629,5680,1.91%,6492,2.18%,16817,3273,33381,481190,10.198799267,147.017925451,10.198799267,147.017925451,14578,195973,258004,3468347


In [343]:
check = client.query( 
    """
    with step_one AS (
    SELECT 
        results.booking_id, 
        ARRAY_AGG(DISTINCT brand IGNORE NULLS) AS brands,
        ARRAY_AGG(DISTINCT individual_campaign_product_string) AS campaign_skus,
        COUNT(DISTINCT individual_campaign_product_string) AS n_skus, 
        campaign_start_date,
        campaign_end_date,
        results.* EXCEPT(booking_id)
    FROM `gcp-wow-cart-data-dev-d4d7.davide.dx_exposed_vs_not_exposed_results` results
    LEFT JOIN (
        SELECT DISTINCT 
            booking_id, 
            campaign_start_date,
            campaign_end_date,
            skus
        FROM gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control 
    ) campaign_skus ON results.booking_id = campaign_skus.booking_id,
    UNNEST(SPLIT(skus, ",")) AS individual_campaign_product_string
    LEFT JOIN (
        SELECT DISTINCT
            SUBSTR(ProductNumber, 1, STRPOS(ProductNumber, '-') - 1) AS Article, 
            Brand AS brand
        FROM `gcp-wow-ent-im-tbl-prod.adp_quantium_wow_commercials_view.qtm_product_attributes_v`
        WHERE SalesOrganisation = "1005"
        AND Brand NOT IN ("OTHER", "UNBRANDED", "")
    ) brand_map
        ON brand_map.Article = individual_campaign_product_string

    WHERE results.booking_id IN (SELECT DISTINCT campaign_id FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs`)
    GROUP BY ALL
    ORDER BY estimated_campaign_uplift_brand_level DESC
)
SELECT 
    booking_id, 
    brands, 
    n_skus, 
    campaign_start_date, 
    campaign_end_date, 
    DATE_DIFF(campaign_end_date, campaign_start_date, DAY) AS campaign_days, 
    DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS campaign_weeks, 
    total_exposed_shoppers, 
    sku_level_converted_exposed_shoppers,
    CONCAT(ROUND((sku_level_converted_exposed_shoppers / total_exposed_shoppers) * 100, 2), "%") AS sku_level_ctr,
    brand_level_converted_exposed_shoppers,
    CONCAT(ROUND((brand_level_converted_exposed_shoppers / total_exposed_shoppers) * 100, 2), "%")  AS brand_level_ctr,
    downsampled_exposed_shoppers, 
    downsampled_converted_shoppers,
    CAST(downsampled_exposed_sku_level_spend AS INT64) downsampled_exposed_sku_level_spend, 
    CAST(downsampled_exposed_brand_level_spend AS INT64) downsampled_exposed_brand_level_spend, 
    downsampled_exposed_sku_level_spend / downsampled_converted_shoppers AS sku_spend_per_converted_shopper,
    downsampled_exposed_brand_level_spend / downsampled_converted_shoppers AS brand_spend_per_converted_shopper,
    
    -- Per converted shopper per week
    (downsampled_exposed_sku_level_spend / downsampled_converted_shoppers) / DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS sku_spend_per_converted_shopper_per_week,
    (downsampled_exposed_brand_level_spend / downsampled_converted_shoppers) / DATE_DIFF(campaign_end_date, campaign_start_date, WEEK) AS brand_spend_per_converted_shopper_per_week,
    CAST(total_downsampled_spend_sku_level_uplift AS INT64) total_downsampled_spend_sku_level_uplift, 
    CAST(total_downsampled_spend_brand_level_uplift AS INT64) total_downsampled_spend_brand_level_uplift, 
    CAST(estimated_campaign_uplift_sku_level AS INT64) estimated_campaign_uplift_sku_level, 
    CAST(estimated_campaign_uplift_brand_level AS INT64) estimated_campaign_uplift_brand_level
FROM step_one
WHERE booking_id = "WOW20004382"
ORDER BY estimated_campaign_uplift_brand_level DESC

    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,brands,n_skus,campaign_start_date,campaign_end_date,campaign_days,campaign_weeks,total_exposed_shoppers,sku_level_converted_exposed_shoppers,sku_level_ctr,brand_level_converted_exposed_shoppers,brand_level_ctr,downsampled_exposed_shoppers,downsampled_converted_shoppers,downsampled_exposed_sku_level_spend,downsampled_exposed_brand_level_spend,sku_spend_per_converted_shopper,brand_spend_per_converted_shopper,sku_spend_per_converted_shopper_per_week,brand_spend_per_converted_shopper_per_week,total_downsampled_spend_sku_level_uplift,total_downsampled_spend_brand_level_uplift,estimated_campaign_uplift_sku_level,estimated_campaign_uplift_brand_level
0,WOW20004382,"[COCA COLA NO SUGAR, SPRITE, DIET COCA COLA, COCA COLA VANILLA, POWERADE, COCA COLA, KIRKS, FANTA, COCA COLA LIFE, MOUNT FRANKLIN]",122,2023-11-01,2023-12-26,55,8,517884,281970,54.45%,282509,54.55%,3361,2224,1103761,1105456,496.295503597,497.057738309,62.036937950,62.132217289,317517,318815,48924988,49124971


In [432]:
check = client.query( 
    """
SELECT 
    *
FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs`
WHERE query_step = "14"
AND query_end_time >= "2024-06-30 18:07:04.303860"
ORDER BY query_end_time DESC 
    """
).result()
check_df = check.to_dataframe()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20002176,14,Total Campaign Analysis Runtime,2024-06-30 19:20:24.061366,2024-06-30 19:20:46.712288,1176,19.600000
1,WOW20000952,14,Total Campaign Analysis Runtime,2024-06-30 19:00:42.764849,2024-06-30 19:01:05.955677,826,13.766667
2,WOW20004576,14,Total Campaign Analysis Runtime,2024-06-30 18:43:29.430412,2024-06-30 18:43:44.089209,996,16.600000
3,WOW20004575,14,Total Campaign Analysis Runtime,2024-06-30 18:26:48.618237,2024-06-30 18:27:05.723852,996,16.600000
4,WOW20004552,14,Total Campaign Analysis Runtime,2024-06-30 18:06:49.328717,2024-06-30 18:07:04.303860,1163,19.383333


In [433]:
check = client.query( 
    """
SELECT 
    booking_id, 
    brand,
    order_context, 
    exposed_shopper_identification_number,
    converted_shopper_identification_number,
    start_txn_time,
    basket_key, 
    article,
    tot_net_incld_gst,
    "7 Days" AS exposure_to_purchase_window
FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_brand_level_v2` converted_products
WHERE booking_id = "WOW20002176"
AND converted_shopper_identification_number IS NOT NULL
ORDER BY booking_id, converted_shopper_identification_number, basket_key, article
LIMIT 20
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,brand,order_context,exposed_shopper_identification_number,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window
0,WOW20002176,CAMPOS,Instore Order,1000000000000006057,1000000000000006057,2023-12-19 18:22:49+00:00,20231219182249002060311310,129065,21.970000000,7 Days
1,WOW20002176,CAMPOS,Instore Order,1000000000000006057,1000000000000006057,2023-12-19 18:22:49+00:00,20231219182249002060311310,129065,21.970000000,7 Days
2,WOW20002176,CAMPOS,Instore Order,1000000000000006491,1000000000000006491,2023-12-07 14:18:08+00:00,20231207141808003037461167,94126,28.860000000,7 Days
3,WOW20002176,CAMPOS,Instore Order,1000000000000006491,1000000000000006491,2023-12-07 14:18:08+00:00,20231207141808003037461167,94126,28.860000000,7 Days
4,WOW20002176,CAMPOS,Instore Order,1000000000000008206,1000000000000008206,2023-11-25 18:16:43+00:00,20231125181643061033771167,128377,15.920000000,7 Days
5,WOW20002176,CAMPOS,Instore Order,1000000000000008206,1000000000000008206,2023-11-25 18:16:43+00:00,20231125181643061033771167,128377,15.920000000,7 Days
6,WOW20002176,CAMPOS,Instore Order,1000000000000008206,1000000000000008206,2023-12-05 15:33:47+00:00,20231205153347001040301167,128377,15.920000000,7 Days
7,WOW20002176,CAMPOS,Instore Order,1000000000000008206,1000000000000008206,2023-12-05 15:33:47+00:00,20231205153347001040301167,128377,15.920000000,7 Days
8,WOW20002176,CAMPOS,Online Order,1000000000000035800,1000000000000035800,2023-11-22 10:05:44.967000+00:00,181447613,94126,29.000000000,7 Days
9,WOW20002176,CAMPOS,Online Order,1000000000000035800,1000000000000035800,2023-11-22 10:05:44.967000+00:00,181447613,94126,29.000000000,7 Days


In [438]:
check = client.query( 
    """
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_brand_level_v2` AS 
SELECT DISTINCT
    booking_id, 
    brand,
    order_context, 
    exposure_crn AS exposed_shopper_identification_number,
    converted_shopper_identification_number,
    start_txn_time,
    basket_key, 
    article,
    tot_net_incld_gst,
    "7 Days" AS exposure_to_purchase_window
FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_brand_level_v2` converted_products
--WHERE booking_id = "WOW20002176"
--AND converted_shopper_identification_number IS NOT NULL
--ORDER BY booking_id, converted_shopper_identification_number, basket_key, article
--LIMIT 100
    """
).result()
check_df = check.to_dataframe()
check_df

""


In [441]:
check = client.query( 
    """

SELECT *
FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_sku_level_v2` converted_products
WHERE booking_id = "WOW20002176"
AND converted_shopper_identification_number IS NOT NULL
ORDER BY booking_id, converted_shopper_identification_number, basket_key, article
LIMIT 100
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,order_context,exposed_shopper_identification_number,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window
0,WOW20002176,Instore Order,1000000000000006491,1000000000000006491,2023-12-07 14:18:08+00:00,20231207141808003037461167,94126,28.860000000,7 Days
1,WOW20002176,Online Order,1000000000000035800,1000000000000035800,2023-11-22 10:05:44.967000+00:00,181447613,94126,29.000000000,7 Days
2,WOW20002176,Online Order,1000000000000035800,1000000000000035800,2023-12-02 07:35:25.680000+00:00,182672995,94126,29.000000000,7 Days
3,WOW20002176,Instore Order,1000000000000141223,1000000000000141223,2023-12-14 15:10:01+00:00,20231214151001061015572842,94126,144.280000000,7 Days
4,WOW20002176,Instore Order,1000000000000162893,1000000000000162893,2023-11-25 16:29:44+00:00,20231125162944002059765606,94126,27.170000000,7 Days
5,WOW20002176,Instore Order,1000000000000162893,1000000000000162893,2023-12-12 19:41:42+00:00,20231212194142008003485646,94126,28.860000000,7 Days
6,WOW20002176,Online Order,1000000000000253885,1000000000000253885,2023-12-07 20:22:17.830000+00:00,183346159,94126,29.000000000,7 Days
7,WOW20002176,Instore Order,1000000000000289285,1000000000000289285,2023-11-23 10:44:37+00:00,20231123104437061075632653,94126,28.860000000,7 Days
8,WOW20002176,Online Order,1000000000000529675,1000000000000529675,2023-11-30 23:15:01.800000+00:00,181522890,94126,29.000000000,7 Days
9,WOW20002176,Online Order,1000000000000529675,1000000000000529675,2023-12-07 21:34:45.107000+00:00,183355023,94126,29.000000000,7 Days


In [394]:
check = client.query( 
    """
SELECT 
    *
FROM  `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` 
WHERE shopper_identification_number = "1000000000000001499" 
AND basket_key IN ("20231011144152001024162867")
LIMIT 5
    """
).result()
check_df = check.to_dataframe()
check_df

,order_context,start_txn_date,start_txn_time,shopper_identification_method,shopper_identification_number,article,brand,SubcatDescription,prod_nbr,basket_key,tot_net_incld_gst
0,Instore Order,2023-10-11,2023-10-11 14:41:52+00:00,CRN,1000000000000001499,256632,MINT,KITCHEN TOOLS & PREPARATION,256632-EA,20231011144152001024162867,1.190000000
1,Instore Order,2023-10-11,2023-10-11 14:41:52+00:00,CRN,1000000000000001499,785977,NONG SHIM,ASIAN FOODS,785977-EA,20231011144152001024162867,6.970000000
2,Instore Order,2023-10-11,2023-10-11 14:41:52+00:00,CRN,1000000000000001499,33373,FOSTER CLARKS,JELLY & PUDDINGS,33373-EA,20231011144152001024162867,6.960000000
3,Instore Order,2023-10-11,2023-10-11 14:41:52+00:00,CRN,1000000000000001499,296813,None,None,296813-EA,20231011144152001024162867,6.970000000
4,Instore Order,2023-10-11,2023-10-11 14:41:52+00:00,CRN,1000000000000001499,307057,CAMPOS,COFFEE,307057-EA,20231011144152001024162867,7.560000000


In [443]:
check = client.query( 
    """
        SELECT 
            booking_id, 
            brand,
            order_context, 
            exposed_shopper_identification_number,
            converted_shopper_identification_number,
            start_txn_time,
            basket_key, 
            article,
            tot_net_incld_gst,
            "7 Days" AS exposure_to_purchase_window
        FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_sales_7_days_brand_level_v2` converted_products
        WHERE booking_id = "WOW20000952" 
        AND converted_shopper_identification_number = "1000000000000001499"
    ;
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,brand,order_context,exposed_shopper_identification_number,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window
0,WOW20000952,CAMPOS,Instore Order,1000000000000001499,1000000000000001499,2023-10-11 14:41:52+00:00,20231011144152001024162867,307057,7.560000000,7 Days


In [314]:
check = client.query( 
    """
    with step_one AS (
        SELECT DISTINCT
            booking_id, 
            exposure_crn, 
            basket_key, 
            article,
            tot_net_incld_gst
        FROM `gcp-wow-food-wlx-digaspt-dev.playground.campaign_exposed_conversions_brand_level_v2`
        WHERE booking_id = "WOW20004382" 
    ),
    step_two AS (
    SELECT 
        booking_id, 
        exposure_crn, 
        SUM(tot_net_incld_gst) AS total_spend_calculated_by_script,
        SUM(tot_net_incld_gst) / 8 AS spend_per_week,
        (SUM(tot_net_incld_gst) / 8) / 122 AS spend_per_week_per_sku
    FROM step_one 
    GROUP BY 1,2
    ORDER BY 3 DESC
    ), 
    step_three AS (
    
        SELECT 
            step_two.*,
            transaction_table_sales.spend_calculated_by_transaction_table
        FROM step_two
        LEFT JOIN (
            SELECT 
                booking_id, 
                shopper_identification_number crn,
                SUM(tot_net_incld_gst) AS spend_calculated_by_transaction_table
            FROM `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` transactions
            INNER JOIN (
                SELECT DISTINCT 
                    booking_number booking_id, 
                    campaign_start_date,
                    campaign_end_date,
                    individual_product_string
                FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns`
                WHERE booking_number = "WOW20004382"
                AND campaign_start_date = "2023-11-01"
            ) AS campaign_skus 
                ON campaign_skus.individual_product_string = transactions.Article
                AND transactions.start_txn_date >= campaign_start_date 
                AND transactions.start_txn_date <= campaign_end_date
            GROUP BY 1,2
        ) AS transaction_table_sales ON transaction_table_sales.crn = step_two.exposure_crn
    )
    

    SELECT 
        *
    FROM step_three
    ORDER BY 3 DESC
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,exposure_crn,total_spend_calculated_by_script,spend_per_week,spend_per_week_per_sku,spend_calculated_by_transaction_table
0,WOW20004382,1100000000161458134,24769.240000000,3096.155000000,25.378319672,24769.240000000
1,WOW20004382,3300000000005035294,20372.360000000,2546.545000000,20.873319672,20372.360000000
2,WOW20004382,3300000000000113466,14881.000000000,1860.125000000,15.246926230,14881.000000000
3,WOW20004382,3300000000002865255,12281.600000000,1535.200000000,12.583606557,12281.600000000
4,WOW20004382,3300000000006969376,10836.240000000,1354.530000000,11.102704918,10807.740000000
...,...,...,...,...,...,...
1513378,WOW20004382,3300000000006609841,None,None,None,None
1513379,WOW20004382,1100000000004831305,None,None,None,None
1513380,WOW20004382,3300000000002264969,None,None,None,None
1513381,WOW20004382,3300000000007241949,None,None,None,None


In [340]:
check = client.query( 
    """
SELECT booking_id, campaign_start_date, campaign_end_date, transactions.* 
FROM `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` transactions 
INNER JOIN (
    SELECT DISTINCT 
        booking_id,
        campaign_start_date,
        campaign_end_date,
        sku 
    FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4, 
    UNNEST(SPLIT(skus, ",")) AS sku 
    WHERE booking_id = "WOW20004382"
) ON TRIM(sku) = article
WHERE shopper_identification_number = "1100000000085084482" 
AND start_txn_date >= DATE("2023-11-01") 
AND start_txn_date <= DATE("2023-12-26")
ORDER BY basket_key, article
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,campaign_start_date,campaign_end_date,order_context,start_txn_date,start_txn_time,shopper_identification_method,shopper_identification_number,article,brand,SubcatDescription,prod_nbr,basket_key,tot_net_incld_gst
0,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,32731,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,14.700000000
1,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,32812,FANTA,SOFT DRINKS - BOTTLES & CANS,,178942721,16.800000000
2,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,38121,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,18.250000000
3,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,623034,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,178942721,10.500000000
4,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,178942721,26.900000000
5,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,53.800000000
6,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-03,2023-11-03 03:42:19.650000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,179127050,26.900000000
7,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-03,2023-11-03 03:42:19.650000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,179127050,53.800000000
8,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-03,2023-11-03 07:29:52.143000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,179149864,53.800000000
9,WOW20004382,2023-11-01,2023-12-26,Online Order,2023-11-03,2023-11-03 07:29:52.143000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,179149864,26.900000000


In [327]:
check = client.query( 
    """
SELECT * 
FROM `gcp-wow-cart-data-dev-d4d7.davide.ces_exposure_uplift_test_vs_control_shopper_level`
WHERE booking_id = "WOW20004382"
AND total_exposed_spend_sku_level_7_day IS NOT NULL
AND exposure_crn  = "1100000000085084482"
ORDER BY total_exposed_spend_sku_level_7_day 
LIMIT 400
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,exposure_crn,total_exposed_spend_sku_level_7_day,total_exposed_spend_brand_level_7_day,candidate_comparison_shopper,total_not_exposed_spend_sku_level_7_day,total_not_exposed_spend_brand_level_7_day,total_spend_sku_level_7_day_uplift,total_spend_brand_level_7_day_uplift
0,WOW20004382,1100000000085084482,6043.270000000,6043.270000000,3300000000007168070,3234.600000000,3234.600000000,2808.670000000,2808.670000000


In [337]:
check = client.query( 
    """
SELECT 
    transactions.* 
FROM `gcp-wow-cart-data-dev-d4d7.davide.test_transactions` transactions
CROSS JOIN (SELECT DISTINCT campaign_start_date, campaign_end_date FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4_test_and_control` WHERE booking_id = "WOW20004382" LIMIT 1) dates
INNER JOIN (
    SELECT DISTINCT 
        booking_id,
        campaign_start_date,
        campaign_end_date,
        sku 
    FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4, 
    UNNEST(SPLIT(skus, ",")) AS sku 
    WHERE booking_id = "WOW20004382"
) ON sku = article
WHERE DATE(start_txn_time) >= dates.campaign_start_date 
AND DATE(start_txn_time) <= dates.campaign_end_date
AND shopper_identification_number = "1100000000085084482"
ORDER BY basket_key, article
    """
).result()
check_df = check.to_dataframe()
check_df

,order_context,start_txn_date,start_txn_time,shopper_identification_method,shopper_identification_number,article,brand,SubcatDescription,prod_nbr,basket_key,tot_net_incld_gst
0,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,32731,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,14.700000000
1,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,32812,FANTA,SOFT DRINKS - BOTTLES & CANS,,178942721,16.800000000
2,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,38121,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,18.250000000
3,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,623034,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,178942721,10.500000000
4,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,178942721,26.900000000
5,Online Order,2023-11-02,2023-11-01 12:47:53.183000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,178942721,53.800000000
6,Online Order,2023-11-03,2023-11-03 03:42:19.650000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,179127050,26.900000000
7,Online Order,2023-11-03,2023-11-03 03:42:19.650000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,179127050,53.800000000
8,Online Order,2023-11-03,2023-11-03 07:29:52.143000+00:00,CRN,1100000000085084482,679121,COCA COLA NO SUGAR,SOFT DRINKS - BOTTLES & CANS,,179149864,53.800000000
9,Online Order,2023-11-03,2023-11-03 07:29:52.143000+00:00,CRN,1100000000085084482,84552,COCA COLA,SOFT DRINKS - BOTTLES & CANS,,179149864,26.900000000


In [333]:
check = client.query( 
    """
SELECT DISTINCT res.booking_id, res.exposure_crn, res.basket_key, res.article, res.tot_net_incld_gst 
FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_sku_level_v2` res
INNER JOIN (
    SELECT DISTINCT 
        booking_id,
        campaign_start_date,
        campaign_end_date,
        sku 
    FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4, 
    UNNEST(SPLIT(skus, ",")) AS sku 
    WHERE booking_id = "WOW20004382"
) ON sku = article
WHERE res.booking_id = "WOW20004382"
AND res.exposure_crn  = "1100000000085084482"
ORDER BY res.basket_key, res.article
LIMIT 400
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,exposure_crn,basket_key,article,tot_net_incld_gst
0,WOW20004382,1100000000085084482,178942721,32731,14.700000000
1,WOW20004382,1100000000085084482,178942721,32812,16.800000000
2,WOW20004382,1100000000085084482,178942721,38121,18.250000000
3,WOW20004382,1100000000085084482,178942721,623034,10.500000000
4,WOW20004382,1100000000085084482,178942721,679121,26.900000000
5,WOW20004382,1100000000085084482,178942721,84552,53.800000000
6,WOW20004382,1100000000085084482,179127050,679121,26.900000000
7,WOW20004382,1100000000085084482,179127050,84552,53.800000000
8,WOW20004382,1100000000085084482,179149864,679121,53.800000000
9,WOW20004382,1100000000085084482,179149864,84552,26.900000000


In [312]:
check = client.query( 
    """
    SELECT 
        *
    FROM `gcp-wow-cart-data-dev-d4d7.davide.campaign_exposed_conversions_brand_level_v2` 
    LIMIT 1
    """
).result()
check_df = check.to_dataframe()
check_df

,impression_date,booking_id,exposure_context,exposure_crn,order_context,converted_shopper_identification_number,start_txn_time,basket_key,brand,article,tot_net_incld_gst,exposure_to_purchase_window
0,2023-11-20,WOW20004382,Exposed,1000000000000421303,Online Order,1000000000000421303,2023-11-19 21:54:19.150000+00:00,181132050,COCA COLA,793315,11.400000000,3 Days


In [307]:
check = client.query( 
    """
with step_one AS (
    SELECT * 
    -- FROM `gcp-wow-food-wlx-digaspt-dev.playground.adobe_events_and_cartology_campaign_info_v4_test_and_control`
    FROM gcp-wow-cart-data-dev-d4d7.davide.adobe_events_and_cartology_campaign_info_v4
    WHERE booking_id = "WOW20004382"
    AND crn = "1100000000085084482"
    ORDER BY date
    LIMIT 20
) 
--step_two AS (
    SELECT 
        booking_id,
        date,
        ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS sorted_skus
    FROM step_one, 
    UNNEST(SPLIT(skus, ",")) AS individual_product_string 
    GROUP BY 1,2
--)
/*
SELECT DISTINCT 
    booking_id, 
    ARRAY_TO_STRING(sorted_skus, ", ") AS sorted_skus_string 
FROM step_two */
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,date,sorted_skus
0,WOW20004382,2023-11-02,"[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 623033, 623034, 623069, 669379, 669683, 672966, 676160, 679121, 679122, 723452, 723453, 725866, 750733, 760280, 77938, ...]"
1,WOW20004382,2023-11-03,"[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 623033, 623034, 623069, 669379, 669683, 672966, 676160, 679121, 679122, 723452, 723453, 725866, 750733, 760280, 77938, ...]"
2,WOW20004382,2023-11-07,"[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 623033, 623034, 623069, 669379, 669683, 672966, 676160, 679121, 679122, 723452, 723453, 725866, 750733, 760280, 77938, ...]"
3,WOW20004382,2023-12-13,"[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 623033, 623034, 623069, 669379, 669683, 672966, 676160, 679121, 679122, 723452, 723453, 725866, 750733, 760280, 77938, ...]"
4,WOW20004382,2023-12-14,"[136874, 137848, 140090, 140092, 140640, 15917, 159796, 159797, 159798, 159799, 159801, 162961, 162962, 176210, 178132, 178416, 178822, 180910, 180941, 215509, 220986, 221011, 221494, 221678, 221713, 221774, 221822, 237909, 238400, 238548, 238684, 242089, 242317, 242611, 243692, 243760, 243795, 244282, 244370, 244826, 244886, 244993, 245007, 245069, 245582, 246323, 260366, 260368, 262843, 279908, 285384, 286997, 287016, 290133, 290344, 313019, 315221, 317387, 317388, 318562, 322924, 32731, 32812, 332730, 332732, 333023, 34691, 360462, 362262, 365186, 365189, 365190, 365548, 369725, 369795, 370757, 38121, 42605, 45533, 51557, 51569, 51570, 52547, 52558, 62170, 6

In [22]:
check = client.query( 
    """
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_two` AS 
WITH sorted_crn_string AS (
    SELECT * EXCEPT(sorted_crn_array), ARRAY_TO_STRING(sorted_crn_array, ", ") AS sorted_crn_string_array
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_one` 
),
unique_sorted_crn_string AS (
    SELECT DISTINCT * FROM sorted_crn_string
),
min_diff AS (
    SELECT 
        *,
        ABS(total_orders_brand_level_ccs - total_orders_brand_level) AS total_orders_brand_level_diff,
        ABS(brand_level_spend_ccs - brand_level_spend) AS brand_level_spend_diff, 
        ABS(rolling_avg_days_between_orders_brand_level_ccs - rolling_avg_days_between_orders_brand_level) AS rolling_avg_days_between_orders_brand_level_diff, 
        ABS(rolling_avg_brand_level_spend_per_order_brand_level_ccs - rolling_avg_brand_level_spend_per_order_brand_level) AS rolling_avg_brand_level_spend_per_order_brand_level_diff, 
        ABS(total_orders_sku_level_ccs - total_orders_sku_level) AS total_orders_sku_level_diff, 
        ABS(sku_level_spend_ccs - sku_level_spend) AS sku_level_spend_diff, 
        ABS(rolling_avg_days_between_orders_sku_level_ccs - rolling_avg_days_between_orders_sku_level) AS rolling_avg_days_between_orders_sku_level_diff, 
        ABS(rolling_avg_sku_level_spend_per_order_sku_level_ccs - rolling_avg_sku_level_spend_per_order_sku_level) AS rolling_avg_sku_level_spend_per_order_sku_level_diff 
    FROM unique_sorted_crn_string
),
ranked_diff AS (
    SELECT 
        *,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_brand_level_diff) AS total_orders_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY brand_level_spend_diff) AS brand_level_spend_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_brand_level_diff) AS rolling_avg_days_between_orders_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_brand_level_spend_per_order_brand_level_diff) AS rolling_avg_brand_level_spend_per_order_brand_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY total_orders_sku_level_diff) AS total_orders_sku_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY sku_level_spend_diff) AS sku_level_spend_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_days_between_orders_sku_level_diff) AS rolling_avg_days_between_orders_sku_level_diff_rank,
        ROW_NUMBER() OVER(PARTITION BY booking_id, exposure_crn ORDER BY rolling_avg_sku_level_spend_per_order_sku_level_diff) AS rolling_avg_sku_level_spend_per_order_sku_level_diff_rank
    FROM min_diff
),
rank_sum AS (
    SELECT 
        *,
        total_orders_brand_level_diff_rank + brand_level_spend_diff_rank + rolling_avg_days_between_orders_brand_level_diff_rank + rolling_avg_brand_level_spend_per_order_brand_level_diff_rank + 
        total_orders_sku_level_diff_rank + sku_level_spend_diff_rank + rolling_avg_days_between_orders_sku_level_diff_rank + rolling_avg_sku_level_spend_per_order_sku_level_diff_rank 
        AS sum_of_diff_ranks
    FROM ranked_diff 
)
SELECT * FROM rank_sum;

    """
).result()
check_df = check.to_dataframe()
check_df

""


In [23]:
check = client.query( 
    """
CREATE OR REPLACE TABLE `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_final` AS
WITH ranked_pairs AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY booking_id ORDER BY sum_of_diff_ranks) AS row_num
    FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_step_two`
),
deduplicated_pairs AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY candidate_comparison_shopper ORDER BY row_num) AS shopper_row_num
    FROM ranked_pairs
)
SELECT 
    booking_id,
    exposure_crn,
    candidate_comparison_shopper,
    total_orders_brand_level_diff,
    brand_level_spend_diff,
    rolling_avg_days_between_orders_brand_level_diff,
    rolling_avg_brand_level_spend_per_order_brand_level_diff,
    total_orders_sku_level_diff,
    sku_level_spend_diff,
    rolling_avg_days_between_orders_sku_level_diff,
    rolling_avg_sku_level_spend_per_order_sku_level_diff
FROM deduplicated_pairs
WHERE shopper_row_num = 1;

    """
).result()
check_df = check.to_dataframe()
check_df

""


In [24]:
check = client.query( 
    """
    SELECT * FROM `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history_test_vs_control_final` LIMIT 20;
    """
).result()
check_df = check.to_dataframe()
check_df

,booking_id,exposure_crn,candidate_comparison_shopper,total_orders_brand_level_diff,brand_level_spend_diff,rolling_avg_days_between_orders_brand_level_diff,rolling_avg_brand_level_spend_per_order_brand_level_diff,total_orders_sku_level_diff,sku_level_spend_diff,rolling_avg_days_between_orders_sku_level_diff,rolling_avg_sku_level_spend_per_order_sku_level_diff
0,WOW20005286,1100000000103680662,3300000000002660270,<NA>,None,NaN,None,<NA>,None,NaN,None
1,WOW20005286,1000000000002176969,1100000000166490731,<NA>,None,NaN,None,<NA>,None,NaN,None
2,WOW20005286,3300000000006204845,1100000000107922156,<NA>,None,NaN,None,<NA>,None,NaN,None
3,WOW20005286,1100000000098821685,1100000000010313147,<NA>,None,NaN,None,<NA>,None,NaN,None
4,WOW20005286,3300000000008121274,1100000000105812968,<NA>,None,NaN,None,<NA>,None,NaN,None
5,WOW20005286,3300000000005056279,1100000000001487314,<NA>,None,NaN,None,<NA>,None,NaN,None
6,WOW20005286,3300000000005776825,3300000000001287460,<NA>,None,NaN,None,<NA>,None,NaN,None
7,WOW20005286,1100000000118677818,3300000000008811601,<NA>,None,NaN,None,<NA>,None,NaN,None
8,WOW20005286,3300000000003202790,1000000000000557244,<NA>,None,NaN,None,<NA>,None,NaN,None
9,WOW20005286,3300000000002652854,1000000000001369482,<NA>,None,NaN,None,<NA>,None,NaN,None


In [19]:
check = client.query( 
    """
        with step_one AS (
            SELECT full_set.*, downsampled.CENTROID_ID
            FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` full_set 
            INNER JOIN `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled` downsampled 
                ON full_set.booking_id = downsampled.booking_id 
                AND full_set.exposure_crn = downsampled.exposure_crn 
            
            WHERE downsampled.exposure_crn IS NOT NULL
            AND full_set.cohort = "Exposed"
        ), 
        step_two AS ( 
        SELECT 
            CENTROID_ID, 
            COUNT(DISTINCT exposure_crn) AS n_exposed 
        FROM step_one 
        GROUP BY 1
    ), step_three AS (
            SELECT full_set.*, downsampled.CENTROID_ID
            FROM  `gcp-wow-food-wlx-digaspt-dev.playground.ces_exposure_plus_history` full_set
            INNER JOIN `gcp-wow-food-wlx-digaspt-dev.playground.clustered_data_sampled` downsampled 
                ON full_set.booking_id = downsampled.booking_id 
                AND full_set.exposure_crn = downsampled.exposure_crn 
            WHERE full_set.cohort = "Not Exposed"
    ), step_four AS (
        SELECT 
            CENTROID_ID, 
            COUNT(DISTINCT exposure_crn) AS n_not_exposed 
        FROM step_three
        GROUP BY 1
    )
    SELECT 
        step_two.CENTROID_ID, 
        n_exposed,
        n_not_exposed 
    FROM step_two 
    LEFT JOIN step_four 
        ON step_two.CENTROID_ID = step_four.CENTROID_ID
    """
).result()
check_df = check.to_dataframe()
check_df

,CENTROID_ID,n_exposed,n_not_exposed
0,10,2000,2000
1,7,481,2000
2,1,138,2000
3,2,13,72
4,5,380,2000
5,3,123,2000
6,9,965,2000
7,6,2000,2000
8,8,418,2000
9,4,35,459


In [114]:
check = client.query( 
    """
    SELECT 
        *
    FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_clustering_run_logs` 
    ORDER BY campaign_id, query_step
    LIMIT 36
    """
).result()
check_df = check.to_dataframe()
check_df

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20005286,1,Shopper Features,2024-06-27 07:04:48.632796,2024-06-27 07:04:54.499698,6,0.100000
1,WOW20005286,2,K-Means Model Training,2024-06-27 07:05:03.961847,2024-06-27 07:10:42.568489,339,5.650000
2,WOW20005286,3,Random Sampling from Clusters,2024-06-27 07:10:47.852258,2024-06-27 07:11:26.294533,39,0.650000
3,WOW20005286,4,Cross Join Test and Control,2024-06-27 07:11:30.196524,2024-06-27 07:12:16.620533,46,0.766667
4,WOW20005286,5,Test vs Control Similarity Ranking,2024-06-27 07:12:19.744640,2024-06-27 07:13:02.752582,43,0.716667
5,WOW20005286,6,Unique Pairs,2024-06-27 07:13:06.191207,2024-06-27 07:13:21.291532,15,0.250000
6,WOW20005286,7,Shopper Level Summary,2024-06-27 07:13:24.570460,2024-06-27 07:13:29.721566,5,0.083333
7,WOW20005286,8,Campaigm Level Summary,2024-06-27 07:13:32.923036,2024-06-27 07:13:40.430408,8,0.133333
8,WOW20005286,9,Total Runtime,2024-06-27 07:13:32.923036,2024-06-27 07:13:43.307719,535,8.916667
9,WOW20005757,1,Shopper Features,2024-06-27 05:01:33.450508,2024-06-27 05:01:39.166118,6,0.100000
